In [4]:
"""
Improved PubMed Search Script with Better Author Detection for Jupyter Lab

This script retrieves publications from PubMed based on a search query,
lists all author names, groups results by year, highlights specific authors,
provides links to the original papers, and includes a GitHub Pages header.

Features:
- Cleaner author search query structure
- Improved handling of author names with consistent formatting
- Better detection of collaborations
- Easy-to-update author list

Requirements:
- pip install biopython
- pip install IPython (should already be installed with Jupyter)
"""

import re
import sys
import ssl
import certifi
from collections import defaultdict
from datetime import datetime
from IPython.display import HTML, display, Markdown

try:
    from Bio import Entrez
    from Bio import Medline
    # Fix for SSL certificate verification issues
    import socket
    import urllib.request
    # Create a custom SSL context using certifi's certificates
    custom_context = ssl.create_default_context(cafile=certifi.where())
    
    # Monkey patch the default opener
    urllib.request.install_opener(
        urllib.request.build_opener(
            urllib.request.HTTPSHandler(context=custom_context)
        )
    )
except ImportError:
    print("This script requires Biopython and certifi. Please install them using:")
    print("!pip install biopython certifi")
    sys.exit(1)


def build_search_query(authors_list, affiliation, default_start_year, collaboration_groups=None):
    """
    Build a structured PubMed search query from a list of authors with individual date filters.
    
    Parameters:
    authors_list (list): List of author dictionaries with name, tag and optional start_year
    affiliation (str): Affiliation to search for (can be None to remove affiliation filter)
    default_start_year (str): Default start year for publication date filter
    collaboration_groups (list): List of research collaboration groups to include in search
    
    Returns:
    str: Formatted PubMed search query
    """
    # Process each author based on their tag type and start year
    author_queries = []
    
    for author in authors_list:
        name = author["name"]
        tag_type = author["tag"]
        
        # Get author-specific start year if available, otherwise use default
        start_year = author.get("start_year", default_start_year)
        
        # Create date filter for this author
        if start_year:
            date_filter = f"(\"{start_year}\"[Date - Publication] : \"3000\"[Date - Publication])"
        else:
            date_filter = ""  # No date filter for this author
        
        if tag_type == "author":
            # Standard author tag format
            author_query = f"({name}[Author])"
        elif tag_type == "custom":
            # Keep custom queries as-is
            author_query = f"({name})"
        else:
            # Default to author tag if not specified
            author_query = f"({name}[Author])"
            
        # Add date filter if specified
        if date_filter:
            author_query = f"({author_query} AND {date_filter})"
            
        author_queries.append(author_query)
    
    # Add collaboration group searches if provided
    if collaboration_groups:
        for group in collaboration_groups:
            author_queries.append(f"(\"{group}\"[Corporate Author])")
    
    # Join all author queries with OR
    authors_query = " OR ".join(author_queries)
    
    # Combine with affiliation if provided
    if affiliation:
        final_query = f"({affiliation}[Affiliation]) AND ({authors_query})"
    else:
        final_query = f"({authors_query})"  # No affiliation filter
    
    print(f"Generated query: {final_query}")
    
    return final_query
    
    return final_query


def search_pubmed(query, max_results=None, email="your.email@example.com", additional_pmids=None):
    """
    Search PubMed for the given query and return a list of PubMed IDs.
    
    Parameters:
    query (str): PubMed search query
    max_results (int, optional): Maximum number of results to retrieve (None for unlimited)
    email (str): Your email address (required by NCBI)
    additional_pmids (list): List of specific PMIDs to add to results
    
    Returns:
    list: List of PubMed records
    """
    Entrez.email = email
    all_records = []
    
    # First, directly fetch the specific PMIDs if provided
    if additional_pmids and len(additional_pmids) > 0:
        print(f"First retrieving {len(additional_pmids)} specific PMIDs...")
        # Start with these known publications
        specific_records = []
        for pmid in additional_pmids:
            try:
                handle = Entrez.efetch(db="pubmed", id=pmid, rettype="medline", retmode="text")
                specific_records.extend(list(Medline.parse(handle)))
                handle.close()
            except Exception as e:
                print(f"  Error retrieving PMID {pmid}: {str(e)}")
        print(f"Retrieved {len(specific_records)} specific publications")
        all_records.extend(specific_records)
    
    # Then, search PubMed to get the list of article IDs
    print(f"Searching PubMed for: {query}")
    try:
        # If max_results is None, use a very large number to get all results
        # PubMed API has a limit around 100,000 but we'll never reach that
        retmax = max_results if max_results is not None else 100000
        handle = Entrez.esearch(db="pubmed", term=query, retmax=retmax, sort="date")
        record = Entrez.read(handle)
        handle.close()
    except Exception as e:
        print(f"Error searching PubMed: {str(e)}")
        print("\nIf you're on macOS and having SSL certificate issues, try running this in a terminal:")
        print("python -m pip install --upgrade certifi")
        return all_records  # Return any records we already fetched directly
    
    id_list = record["IdList"]
    print(f"Found {len(id_list)} articles via query")
    
    # Don't fetch PMIDs we already have
    existing_pmids = [record.get("PMID", "") for record in all_records]
    id_list = [pmid for pmid in id_list if pmid not in existing_pmids]
    
    if not id_list:
        return all_records  # Return any records we already fetched directly
    
    # Then, fetch the full records for these IDs
    print(f"Retrieving details for {len(id_list)} additional articles...")
    try:
        # For large result sets, fetch in batches to avoid timeout issues
        batch_size = 500
        for i in range(0, len(id_list), batch_size):
            batch = id_list[i:i+batch_size]
            print(f"  Fetching batch {i//batch_size + 1} of {(len(id_list)-1)//batch_size + 1} ({len(batch)} articles)...")
            handle = Entrez.efetch(db="pubmed", id=batch, rettype="medline", retmode="text")
            batch_records = list(Medline.parse(handle))
            handle.close()
            all_records.extend(batch_records)
    except Exception as e:
        print(f"Error fetching article details: {str(e)}")
    
    print(f"Total publications found: {len(all_records)}")
    return all_records


def group_by_year(records):
    """Group records by publication year."""
    grouped = defaultdict(list)
    
    for record in records:
        # Extract year from the publication date
        if "DP" in record:
            # Publication date format can vary, so we extract just the year
            year_match = re.search(r'\b(19|20)\d{2}\b', record["DP"])
            if year_match:
                year = year_match.group(0)
                grouped[year].append(record)
            else:
                grouped["Unknown Year"].append(record)
        else:
            grouped["Unknown Year"].append(record)
    
    return grouped


def normalize_author_name(name):
    """
    Normalize author names to handle variations in capitalization and special characters.
    This helps with more consistent matching across different formats.
    """
    # Convert to lowercase for case-insensitive matching
    normalized = name.lower()
    # Replace hyphen with space to catch both hyphenated and non-hyphenated versions
    normalized = normalized.replace('-', ' ')
    return normalized


def is_team_member(author, authors_to_highlight):
    """
    Check if an author is a team member using normalized name matching.
    This handles different ways names might appear in PubMed.
    """
    # Normalize the author name for more consistent matching
    normalized_author = normalize_author_name(author)
    
    for highlight_name in authors_to_highlight:
        # Skip empty highlight names
        if not highlight_name:
            continue
            
        # Split the highlight name (which should be in LastName Initial format)
        highlight_parts = highlight_name.split() if isinstance(highlight_name, str) else []
        
        # Check for last name match (first part of the highlight name)
        if len(highlight_parts) > 0:
            last_name = normalize_author_name(highlight_parts[0])
            
            # Check if normalized last name is in the normalized author string
            if last_name in normalized_author:
                # If we have a last name match, check for initial if available
                if len(highlight_parts) > 1:
                    initial = highlight_parts[1][0].lower() if len(highlight_parts[1]) > 0 else ""
                    
                    # For initials, we look for them after the last name
                    if initial:
                        # Look for the initial after the last name mention
                        last_name_pos = normalized_author.find(last_name)
                        if last_name_pos >= 0:
                            remaining_text = normalized_author[last_name_pos + len(last_name):]
                            if initial.lower() in remaining_text.lower():
                                return True
                    else:
                        # If no initial provided, last name match is enough
                        return True
                else:
                    # If we only have a last name to match, that's enough
                    return True
    
    return False


def highlight_author(author, authors_to_highlight, format_type):
    """Highlight an author name if it appears in the list of authors to highlight."""
    if is_team_member(author, authors_to_highlight):
        if format_type == "text":
            return f"**{author}**"
        elif format_type == "html":
            return f'<span style="color: red; font-weight: bold;">{author}</span>'
        elif format_type == "markdown":
            return f"**{author}**"
    
    return author


def get_pubmed_link(pmid, format_type):
    """Generate a link to the PubMed article."""
    url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
    
    if format_type == "text":
        return url
    elif format_type == "html":
        return f'<a href="{url}" target="_blank" style="text-decoration: none;">{url}</a>'
    elif format_type == "markdown":
        # Using HTML inside markdown to remove the underline
        return f'<a href="{url}" style="text-decoration: none;">PubMed</a>'
    
    return url


def format_record(record, authors_to_highlight, format_type):
    """Format a single record with highlighted authors."""
    # Get basic citation information
    title = record.get("TI", "No Title Available")
    journal = record.get("JT", "Unknown Journal")
    volume = record.get("VI", "")
    issue = record.get("IP", "")
    pages = record.get("PG", "")
    pmid = record.get("PMID", "")
    
    # Format journal info
    journal_info = journal
    if volume:
        journal_info += f", {volume}"
    if issue:
        journal_info += f"({issue})"
    if pages:
        journal_info += f":{pages}"
    
    # Format authors
    if "AU" in record:
        authors = record["AU"]
        formatted_authors = []
        
        for author in authors:
            formatted_authors.append(highlight_author(author, authors_to_highlight, format_type))
        
        if format_type == "text" or format_type == "markdown":
            authors_text = ", ".join(formatted_authors)
        elif format_type == "html":
            authors_text = ", ".join(formatted_authors)
    else:
        authors_text = "No authors listed"
    
    # Create link to PubMed
    pubmed_link = get_pubmed_link(pmid, format_type)
    
    # Format the complete record
    if format_type == "text":
        return f"* {title}\n  {authors_text}. {journal_info}. {pubmed_link}\n"
    
    elif format_type == "html":
        return f"""
        <li>
            <strong>{title}</strong><br>
            {authors_text}. <em>{journal_info}</em>. {pubmed_link}
        </li>
        <br>
        """
    
    elif format_type == "markdown":
        return f"* **{title}**  \n  {authors_text}. *{journal_info}*. {pubmed_link}\n"


def generate_output(grouped_records, authors_to_highlight, format_type, github_pages_header=None):
    """Generate formatted output."""
    if format_type == "html":
        output = "<html>\n<head>\n<title>PubMed Search Results</title>\n</head>\n<body>\n"
    elif format_type == "markdown" and github_pages_header:
        output = github_pages_header + "\n\n"
    else:
        output = ""
    
    # Sort years in descending order (most recent first)
    for year in sorted(grouped_records.keys(), reverse=True):
        records = grouped_records[year]
        
        if format_type == "html":
            output += f"<h2>{year}</h2>\n<ul>\n"
        elif format_type == "markdown":
            output += f"## {year}\n\n"
        else:
            output += f"{year}\n" + "-" * 40 + "\n"
        
        for record in records:
            formatted_record = format_record(record, authors_to_highlight, format_type)
            output += formatted_record + "\n"
        
        if format_type == "html":
            output += "</ul>\n"
        else:
            output += "\n"
    
    if format_type == "html":
        output += "</body>\n</html>"
    
    return output


def pubmed_search(query=None, email=None, authors_list=None, affiliation=None, 
                 default_start_year=None, highlight=None, max_results=100, 
                 output_format="markdown", output_file=None, github_pages_header=None,
                 collaboration_groups=None, additional_pmids=None):
    """
    Main function for Jupyter notebooks to search PubMed and display results.
    
    Parameters:
    query (str, optional): PubMed search query (if provided, overrides authors_list and affiliation)
    email (str): Your email address (required by NCBI)
    authors_list (list): List of author dictionaries with name, tag, and optional start_year
    affiliation (str): Affiliation to search for
    default_start_year (str): Default start year for publication date filter
    highlight (list): List of author names to highlight
    max_results (int): Maximum number of results to retrieve
    output_format (str): Output format ('text', 'html', or 'markdown')
    output_file (str): Optional file path to save the results
    github_pages_header (str): Optional GitHub Pages front matter header
    collaboration_groups (list): List of research collaboration groups to include in search
    additional_pmids (list): List of specific PMIDs to include regardless of search results
    """
    if highlight is None:
        highlight = []
    
    # Build query if not provided
    if query is None and authors_list is not None:
        query = build_search_query(authors_list, affiliation, default_start_year, collaboration_groups)
    
    # Search PubMed
    records = search_pubmed(query, max_results, email, additional_pmids)
    
    if not records:
        print("No results found for the query or an error occurred")
        return
    
    # Group records by year
    grouped_records = group_by_year(records)
    
    # Generate output
    output = generate_output(grouped_records, highlight or authors_list, output_format, github_pages_header)
    
    # Save to file if requested
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(output)
        print(f"Results saved to {output_file}")
    
    # Display results based on format
    if output_format == "html":
        display(HTML(output))
    elif output_format == "markdown":
        display(Markdown(output))
    else:
        print(output)
    
    return grouped_records  # Return data for further processing if needed


# CONFIGURATION SECTION
# ===========================================
# Set your parameters here in an easy-to-edit format

# CONFIGURATION SETTINGS
# ===========================================

# Your email for NCBI
YOUR_EMAIL = "nora.bruns@uk-essen.de"

# Affiliation to search - set to None to search without affiliation filter
# Change to None to find publications without the city in the affiliation
AFFILIATION = None  # Was "Essen"

# No default start year - each author can have their own
DEFAULT_START_YEAR = None

# Maximum results to retrieve (set to None for unlimited)
MAX_RESULTS = None  # No limit, retrieve all matching publications

# OUTPUT SETTINGS
# ===========================================
OUTPUT_FORMAT = "markdown"  # Options: "text", "html", "markdown"
OUTPUT_FILE = "index.md"  # Set to None to disable file output

# List of authors to search for - easy to add/edit
# Each author is a dictionary with:
#   - name: Author name in "LastName FirstName" format for precise searching
#   - highlight: Name format for highlighting (typically "LastName Initial")
#   - tag: "author" for standard [Author] tag, "custom" for custom query
#   - start_year: (Optional) Start year to filter publications for this author
AUTHORS_LIST = [
    # Dohna-Schwake with different variations to catch hyphenation and capitalization issues
    {"name": "Dohna-Schwake Christian", "highlight": "Dohna-Schwake C", "tag": "author", "start_year": "2019"},
    {"name": "Dohna Schwake Christian", "highlight": "Dohna-Schwake C", "tag": "author", "start_year": "2019"},
    {"name": "dohna-schwake c", "highlight": "Dohna-Schwake C", "tag": "custom", "start_year": "2019"},
    
    # Bruns variations
    {"name": "Bruns Nora", "highlight": "Bruns N", "tag": "author"},
    {"name": "bruns n essen", "highlight": "Bruns N", "tag": "custom"},  # Kept as requested
    
    # Other team members
    {"name": "Schwarz Simone", "highlight": "Schwarz S", "tag": "author"},
    
    # Goretzki with variations to catch collaborative papers
    {"name": "Goretzki Sarah", "highlight": "Goretzki S", "tag": "author"},
    {"name": "Goretzki Sarah Charlotte", "highlight": "Goretzki SC", "tag": "author"},
    {"name": "Goretzki S", "highlight": "Goretzki S", "tag": "custom"},  # Without [Author] tag for better matching
    {"name": "Goretzki", "highlight": "Goretzki", "tag": "custom"},  # Added to catch partial matches
    
    # Remaining team members
    {"name": "Greve Sandra", "highlight": "Greve S", "tag": "author"},
    {"name": "Brensing Pia", "highlight": "Brensing P", "tag": "author"},
    {"name": "Paul Luisa", "highlight": "Paul L", "tag": "author"},
    {"name": "Hojeij Rayan", "highlight": "Hojeij R", "tag": "author"}
]

# Known collaborative groups that might include team members
COLLABORATION_GROUPS = [
    "German Neonatal Network",
    "GNN"
]

# Specific PMIDs that should be included regardless of search results
# Add any known papers that aren't being found automatically
ADDITIONAL_PMIDS = [
    "36670127",  # Goretzki S paper example
    "31444505"   # Dohna-Schwake C paper example
]

# Extract highlight names for the output
AUTHORS_TO_HIGHLIGHT = [author["highlight"] for author in AUTHORS_LIST]

# GitHub Pages YAML front matter header
GITHUB_HEADER = """---
type: page
title: Publications by our lab members
---"""

# OUTPUT SETTINGS
# ===========================================
OUTPUT_FORMAT = "markdown"  # Options: "text", "html", "markdown"
OUTPUT_FILE = "index.md"  # Set to None to disable file output

# RUN THE SEARCH
# ===========================================
# Build the search query and run the search
pubmed_search(
    email=YOUR_EMAIL,
    authors_list=AUTHORS_LIST,
    affiliation=AFFILIATION,
    default_start_year=DEFAULT_START_YEAR,
    highlight=AUTHORS_TO_HIGHLIGHT,
    max_results=MAX_RESULTS,
    output_format=OUTPUT_FORMAT,
    output_file=OUTPUT_FILE,
    github_pages_header=GITHUB_HEADER,
    collaboration_groups=COLLABORATION_GROUPS,
    additional_pmids=ADDITIONAL_PMIDS
)

Generated query: (((Dohna-Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((Dohna Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((dohna-schwake c) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR (Bruns Nora[Author]) OR (bruns n essen) OR (Schwarz Simone[Author]) OR (Goretzki Sarah[Author]) OR (Goretzki Sarah Charlotte[Author]) OR (Goretzki S) OR (Goretzki) OR (Greve Sandra[Author]) OR (Brensing Pia[Author]) OR (Paul Luisa[Author]) OR (Hojeij Rayan[Author]) OR ("German Neonatal Network"[Corporate Author]) OR ("GNN"[Corporate Author]))
First retrieving 2 specific PMIDs...
Retrieved 2 specific publications
Searching PubMed for: (((Dohna-Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((Dohna Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((dohna-schwake c) AND ("2019"[Date - Publication] : "

---
type: page
title: Publications by our lab members
---

## 2025

* **The Phoenix Sepsis Score in Pediatric Oncology Patients With Sepsis at PICU Admission: Test of Performance in a European Multicenter Cohort, 2018-2020.**  
  Wosten-van Asperen RM, la Roi-Teeuw HM, Tissing WJE, Jordan I, **Dohna-Schwake C**, Bottari G, Pappachan J, Crazzolara R, Amigoni A, Mizia-Malarz A, Moscatelli A, Sanchez-Martin M, Willems J, Schlapbach LJ. *Pediatric critical care medicine : a journal of the Society of Critical Care Medicine and the World Federation of Pediatric Intensive and Critical Care Societies, 26(2):e177-e185*. <a href="https://pubmed.ncbi.nlm.nih.gov/39982155/" style="text-decoration: none;">PubMed</a>

* **Structure and Dynamics of Macrophage Infectivity Potentiator Proteins from Pathogenic Bacteria and Protozoans Bound to Fluorinated Pipecolic Acid Inhibitors.**  
  Perez Carrillo VH, Whittaker JJ, Wiedemann C, Harder JM, Lohr T, Jamithireddy AK, Dajka M, **Goretzki B**, Joseph B, Guskov A, Harmer NJ, Holzgrabe U, Hellmich UA. *Journal of medicinal chemistry, 68(5):5926-5941*. <a href="https://pubmed.ncbi.nlm.nih.gov/39976355/" style="text-decoration: none;">PubMed</a>

* **Ultrasound Super-Resolution Imaging of Neonatal Cerebral Vascular Reorganization.**  
  **Schwarz S**, Denis L, Nedoschill E, Buehler A, Danko V, Hilger AC, Brevis Nunez F, Durr NR, Schlunz-Hendann M, Brassel F, Felderhoff-Muser U, Reutter H, Woelfle J, Jungert J, **Dohna-Schwake C**, **Bruns N**, Regensburger AP, Couture O, Mandelbaum H, Knieling F. *Advanced science (Weinheim, Baden-Wurttemberg, Germany):e2415235*. <a href="https://pubmed.ncbi.nlm.nih.gov/39899647/" style="text-decoration: none;">PubMed</a>

* **Performance of ICD-10-based injury severity scores in pediatric trauma patients using the ICD-AIS map and survival rate ratios.**  
  **Hojeij R**, **Brensing P**, Nonnemacher M, Kowall B, Felderhoff-Muser U, Dudda M, **Dohna-Schwake C**, Stang A, **Bruns N**. *Journal of clinical epidemiology, 178:111634*. <a href="https://pubmed.ncbi.nlm.nih.gov/39647538/" style="text-decoration: none;">PubMed</a>

* **Severity of Pediatric Inflammatory Multisystem Syndrome Temporally Associated with SARS-CoV-2 Diminished During Successive Waves of the COVID-19 Pandemic: Data from a Nationwide German Survey.**  
  Lohrmann F, Doenhardt M, Diffloth N, Jakob A, Hospach A, Schneider DT, Trotter A, Brunner J, **Goretzki S**, Arens S, Rank M, Mauer R, Armann J, Berner R, Hufnagel M. *The Journal of pediatrics, 278:114419*. <a href="https://pubmed.ncbi.nlm.nih.gov/39603520/" style="text-decoration: none;">PubMed</a>

* **Management and outcome of cutaneous diphtheria in adolescent refugees in Germany, June 2022 - October 2023.**  
  Zink A, Hofer J, Schneider C, Kessler F, Klenze H, Klauwer D, Maleki K, Muller A, **Goretzki S**, Wang S, Kobbe R, Ramirez AV, Bode S, Janda A, Fressle R, Remppis J, Henneke P, Rieg S, Berger A, Sing A, Hufnagel M, Spielberger BD. *Infection, 53(1):329-337*. <a href="https://pubmed.ncbi.nlm.nih.gov/39190269/" style="text-decoration: none;">PubMed</a>

* **An [illustrative] update on pediatric emergency medicine ultrasound: part 2 - abdominal and urogenital applications.**  
  Dong Y, Hoffmann B, **Schwarz S**, Nourkami-Tutdibi N, Huang YL, Chen S, Cekuolis A, Augustiniene R, Snelling PJ, Schreiber-Dietrich D, Grevelding L, Dietrich CF. *Medical ultrasonography, 27(1):63-72*. <a href="https://pubmed.ncbi.nlm.nih.gov/39078993/" style="text-decoration: none;">PubMed</a>


## 2024

* **Dual BACH1 regulation by complementary SCF-type E3 ligases.**  
  **Goretzki B**, Khoshouei M, Schroder M, Penner P, Egger L, Stephan C, Argoti D, Dierlamm N, Rada JM, Kapps S, Muller CS, Thiel Z, Mutlu M, Tschopp C, Furkert D, Freuler F, Haenni S, Tenaillon L, Knapp B, Hinniger A, Hoppe P, Schmidt E, Gutmann S, Iurlaro M, Ryzhakov G, Fernandez C. *Cell, 187(26):7585-7602.e25*. <a href="https://pubmed.ncbi.nlm.nih.gov/39657677/" style="text-decoration: none;">PubMed</a>

* **Case report: Acute liver failure during deferasirox therapy and the potential role of pharmacogenetics.**  
  Garcia-Farina B, Rink L, Santarini V, Westkemper M, **Dohna-Schwake C**, Mohlendick B. *Frontiers in pharmacology, 15:1477755*. <a href="https://pubmed.ncbi.nlm.nih.gov/39508042/" style="text-decoration: none;">PubMed</a>

* **Ultrasound Localization Microscopy for the Assessment of Microvascular Circulation in Ischemic Perinatal Stroke.**  
  Regensburger AP, Wachter F, Denis L, Mandelbaum H, Schey F, Buehler A, Siebenlist G, **Schwarz S**, Schulz-Heise S, Rompel O, Trollmann R, Woelfle J, Jungert J, Couture O, Reutter H, Hanslik G, Knieling F. *Stroke, 55(12):e323-e325*. <a href="https://pubmed.ncbi.nlm.nih.gov/39474686/" style="text-decoration: none;">PubMed</a>

* **The effect of live-performed music therapy with physical contact in preterm infants on parental perceived stress and salivary cortisol levels.**  
  Dewan MV, Ader M, Kleinbeck T, Dathe AK, Schedlowski M, Engler H, Felderhoff-Mueser U, **Bruns N**, Kobus S. *Frontiers in psychology, 15:1441824*. <a href="https://pubmed.ncbi.nlm.nih.gov/39434912/" style="text-decoration: none;">PubMed</a>

* **How to perform Point of Care Ultrasound at resuscitation and when it is useful.**  
  Wastl D, Blaivas M, Horn R, Michels G, Seibel A, **Schwarz S**, Hoffmann BH, Hoffmann O, Von Ow D, Dietrich CF. *Medical ultrasonography*. <a href="https://pubmed.ncbi.nlm.nih.gov/39420822/" style="text-decoration: none;">PubMed</a>

* **An [illustrative] update on pediatric emergency ultrasound: part 3 - cerebral, musculoskeletal and other applications.**  
  **Schwarz S**, Dong Y, Snelling PJ, Hoffmann B, Nourkami-Tutdibi N, Huang YL, Chen S, Cekuolis A, Augustiniene R, Schreiber-Dietrich D, Grevelding L, Dietrich CF. *Medical ultrasonography*. <a href="https://pubmed.ncbi.nlm.nih.gov/39420818/" style="text-decoration: none;">PubMed</a>

* **Telemedicine-based antibiotic stewardship program in pediatrics: study protocol of a stepped wedge cluster randomized trial-the TeleKasper study.**  
  Diexer S, Ihling A, Gomes D, Moritz S, Simon A, **Dohna-Schwake C**, Mikolajczyk R, Huebner J, von Both U. *Trials, 25(1):678*. <a href="https://pubmed.ncbi.nlm.nih.gov/39402608/" style="text-decoration: none;">PubMed</a>

* **Characterization of the Immune-Modulating Properties of Different beta-Glucans on Myeloid Dendritic Cells.**  
  Rainer H, **Goretzki A**, Lin YJ, Schiller HR, Krause M, Doring S, Strecker D, Junker AC, Wolfheimer S, Toda M, Scheurer S, Schulke S. *International journal of molecular sciences, 25(18)*. <a href="https://pubmed.ncbi.nlm.nih.gov/39337403/" style="text-decoration: none;">PubMed</a>

* **Oxadiazolines as Photoreleasable Labels for Drug Target Identification.**  
  Bon C, **Goretzki B**, Flamme M, Shelton C, Davis H, Lima F, Garcia F, Brittain S, Brocklehurst CE. *Journal of the American Chemical Society, 146(39):26759-26765*. <a href="https://pubmed.ncbi.nlm.nih.gov/39288302/" style="text-decoration: none;">PubMed</a>

* **Emergency surgery influences oncological outcome in small intestinal neuroendocrine tumors.**  
  Butz F, Supper L, Reinhard L, Dukaczewska A, Jann H, Fehrenbach U, Muller-Debus CF, Skachko T, Pratschke J, **Goretzki PE**, Mogl MT, Dobrindt EM. *Scandinavian journal of surgery : SJS : official organ for the Finnish Surgical Society and the Scandinavian Surgical Society, 113(4):303-313*. <a href="https://pubmed.ncbi.nlm.nih.gov/39230104/" style="text-decoration: none;">PubMed</a>

* **Feasibility, efficacy, and safety of animal-assisted activities with visiting dogs in inpatient pediatric oncology.**  
  Steff K, Grasemann M, Ostermann K, **Goretzki SC**, Rath PM, Reinhardt D, Schundeln MM. *World journal of pediatrics : WJP, 20(9):915-924*. <a href="https://pubmed.ncbi.nlm.nih.gov/39112809/" style="text-decoration: none;">PubMed</a>

* **Neonatal Sepsis Episodes and Retinopathy of Prematurity in Very Preterm Infants.**  
  Glaser K, Hartel C, Klingenberg C, Herting E, Fortmann MI, Speer CP, Stensvold HJ, Huncikova Z, Ronnestad AE, Nentwich MM, Stahl A, Dammann O, Gopel W. *JAMA network open, 7(7):e2423933*. <a href="https://pubmed.ncbi.nlm.nih.gov/39052290/" style="text-decoration: none;">PubMed</a>

* **Surgical Site Infections After Pediatric Liver Transplantation-Impact of a Change in Perioperative Prophylactic Antibiotic Protocol.**  
  **Bruns N**, Hauschild J, Lainka E, **Dohna-Schwake C**. *Pediatric transplantation, 28(5):e14830*. <a href="https://pubmed.ncbi.nlm.nih.gov/39030994/" style="text-decoration: none;">PubMed</a>

* **Short-term outcomes of asphyxiated neonates depending on requirement for transfer in the first 24 h of life.**  
  **Bruns N**, Feddahi N, **Hojeij R**, Rossi R, **Dohna-Schwake C**, Stein A, Kobus S, Stang A, Kowall B, Felderhoff-Muser U. *Resuscitation, 202:110309*. <a href="https://pubmed.ncbi.nlm.nih.gov/39002696/" style="text-decoration: none;">PubMed</a>

* **The effect of live music therapy on white matter microstructure in very preterm infants - A randomized controlled trial.**  
  Dewan MV, Jungilligens J, Kobus S, Diezel M, Dathe AK, Schweiger B, Huning B, Felderhoff-Muser U, **Bruns N**. *European journal of paediatric neurology : EJPN : official journal of the European Paediatric Neurology Society, 51:132-139*. <a href="https://pubmed.ncbi.nlm.nih.gov/38941879/" style="text-decoration: none;">PubMed</a>

* **Cerebral Doppler imaging in neonates: A guide for clinical application and diagnosis.**  
  Horsch S, **Schwarz S**, Arnaez J, Steggerda S, Arena R, Govaert P. *Developmental medicine and child neurology, 66(12):1570-1589*. <a href="https://pubmed.ncbi.nlm.nih.gov/38940604/" style="text-decoration: none;">PubMed</a>

* **Association of bilaterally suppressed EEG amplitudes and outcomes in critically ill children.**  
  **Paul L**, **Greve S**, Hegemann J, Gienger S, Loffelhardt VT, Della Marina A, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in neuroscience, 18:1411151*. <a href="https://pubmed.ncbi.nlm.nih.gov/38903601/" style="text-decoration: none;">PubMed</a>

* **Position statement on genuine physiotherapy research at German university hospitals.**  
  Klotz SGR, Bokel A, Friderichs-Nedohibchenko M, Stickdorn I, Vogel B, Doods B, Feldmann F, Ghiazza M, Giehl M, Hoberg A, Jansen L, Kohlhofer D, Leonhardt R, Meier SF, Muller C, Pannzek M, **Schwarz S**, Traut M, Urdahl M. *German medical science : GMS e-journal, 22:Doc06*. <a href="https://pubmed.ncbi.nlm.nih.gov/38883338/" style="text-decoration: none;">PubMed</a>

* **Small molecule induced STING degradation facilitated by the HECT ligase HERC4.**  
  Mutlu M, Schmidt I, Morrison AI, **Goretzki B**, Freuler F, Begue D, Simic O, Pythoud N, Ahrne E, Kapps S, Roest S, Bonenfant D, Jeanpierre D, Tran TT, Maher R, An S, Rietsch A, Nigsch F, Hofmann A, Reece-Hoyes J, Parker CN, Guerini D. *Nature communications, 15(1):4584*. <a href="https://pubmed.ncbi.nlm.nih.gov/38811577/" style="text-decoration: none;">PubMed</a>

* **An [illustrative] update on pediatric emergency medicine ultrasound: part 1 - trauma and thoracic applications.**  
  Dong Y, **Schwarz S**, Hoffmann B, Nourkami-Tutdibi N, Huang YL, Chen S, Cekuolis A, Augustiniene R, Snelling PJ, Schreiber-Dietrich D, Grevelding L, Dietrich CF. *Medical ultrasonography, 26(4):393-404*. <a href="https://pubmed.ncbi.nlm.nih.gov/38805622/" style="text-decoration: none;">PubMed</a>

* **Effects of an early transfer from incubator to a warming crib in very low birthweight preterm infants.**  
  **Greve S**, **Bruns N**, Dathe AK, Schuendeln MM, Felderhoff-Mueser U, Stein A. *BMC pediatrics, 24(1):319*. <a href="https://pubmed.ncbi.nlm.nih.gov/38724933/" style="text-decoration: none;">PubMed</a>

* **COMFORTneo scale in preterm infants during live performed music therapy-Difference between close physical contact and hand touch contact.**  
  Kobus S, Kleinbeck T, Ader M, Dewan MV, Dathe AK, Feddahi N, Felderhoff-Mueser U, **Bruns N**. *Frontiers in neuroscience, 18:1359769*. <a href="https://pubmed.ncbi.nlm.nih.gov/38606306/" style="text-decoration: none;">PubMed</a>

* **Correlation of Comfort Score and Narcotrend Index during Procedural Sedation with Midazolam and Propofol in Children.**  
  **Bruns N**, Joist CA, Joist CM, Daniels A, Felderhoff-Muser U, **Dohna-Schwake C**, Tschiedel E. *Journal of clinical medicine, 13(5)*. <a href="https://pubmed.ncbi.nlm.nih.gov/38592307/" style="text-decoration: none;">PubMed</a>

* **Adaption of Follow-Up Visits for Preterm and High-Risk Children for Video Consultations.**  
  Dathe AK, **Greve S**, Teschler U, Heuser-Spura KM, Cordier L, Albayrak B, Felderhoff-Mueser U, Huening BM. *Telemedicine journal and e-health : the official journal of the American Telemedicine Association, 30(7):e2004-e2012*. <a href="https://pubmed.ncbi.nlm.nih.gov/38466975/" style="text-decoration: none;">PubMed</a>

* **Pediatric Community-Acquired Brain Abscesses: A Single-center Retrospective Study.**  
  Lu KH, **Bruns N**, Pentek C, Della Marina A, Gangfuss A, Kolbel H, Dogan B, Dziobaka J, Ahmadipour Y, Damman P, Felderhoff-Muser U, **Dohna-Schwake C**, **Goretzki SC**. *The Pediatric infectious disease journal, 43(6):e214-e217*. <a href="https://pubmed.ncbi.nlm.nih.gov/38451881/" style="text-decoration: none;">PubMed</a>

* **Current practice of intracranial pressure monitoring in children with severe traumatic brain injury-a nationwide prospective surveillance study in Germany.**  
  **Brensing P**, **Greve S**, **Hojeij R**, Dammann P, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in pediatrics, 12:1355771*. <a href="https://pubmed.ncbi.nlm.nih.gov/38405592/" style="text-decoration: none;">PubMed</a>

* **Outbreak of severe community-acquired bacterial infections among children in North Rhine-Westphalia (Germany), October to December 2022.**  
  **Goretzki SC**, van der Linden M, Itzek A, Huhne T, Adelmann RO, Ala Eldin F, Albarouni M, Becker JC, Berghauser MA, Boesing T, Boeswald M, Brasche M, Brevis Nunez F, Camara R, Deibert C, Dohle F, Dolgner J, Dziobaka J, Eifinger F, Elting N, Endmann M, Engelmann G, Frenzke H, Gappa M, Gharavi B, Goletz C, Hahn E, Heidenreich Y, Heimann K, Hensel KO, Hoffmann HG, Hoppenz M, Horneff G, Klassen H, Koerner-Rettberg C, Langler A, Lenz P, Lohmeier K, Muller A, Niemann F, Paulussen M, Pentek F, Perez R, Pingel M, Repges P, Rothoeft T, Rubo J, Schade H, Schmitz R, Schonhoff P, Schwade JN, Schwarz T, Seiffert P, Selzer G, Spille U, Thiel C, Thimm A, Urgatz B, van den Heuvel A, van Hop T, Giesen V, Wirth S, Wollbrink T, Wuller D, Felderhoff-Muser U, **Dohna-Schwake C**, Lam TT, Claus H, **Bruns N**. *Infection, 52(3):1099-1111*. <a href="https://pubmed.ncbi.nlm.nih.gov/38366304/" style="text-decoration: none;">PubMed</a>

* **Morphological chest CT changes in cystic fibrosis and massive hemoptysis.**  
  Dohna M, Kuhl H, Sutharsan S, **Bruns N**, Vo Chieu VD, Hellms S, Kornemann N, Montag MJ. *Radiologie (Heidelberg, Germany), 64(Suppl 1):76-85*. <a href="https://pubmed.ncbi.nlm.nih.gov/38326635/" style="text-decoration: none;">PubMed</a>

* **Influence of Lymphatic, Microvascular and Perineural Invasion on Oncological Outcome in Patients with Neuroendocrine Tumors of the Small Intestine.**  
  Butz F, Dukaczewska A, Kunze CA, Kromer JM, Reinhard L, Jann H, Fehrenbach U, Muller-Debus CF, Skachko T, Pratschke J, **Goretzki PE**, Mogl MT, Dobrindt EM. *Cancers, 16(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/38254794/" style="text-decoration: none;">PubMed</a>

* **Clinical recommendations for the inpatient management of lower respiratory tract infections in children and adolescents with severe neurological impairment in Germany.**  
  Mauritz MD, von Both U, **Dohna-Schwake C**, Gille C, Hasan C, Huebner J, Hufnagel M, Knuf M, Liese JG, Renk H, Rudolph H, Schulze-Sturm U, Simon A, Stehling F, Tenenbaum T, Zernikow B. *European journal of pediatrics, 183(3):987-999*. <a href="https://pubmed.ncbi.nlm.nih.gov/38172444/" style="text-decoration: none;">PubMed</a>

* **[Increased rate of complications of pediatric acute otitis media and sinusitis in 2022/2023].**  
  Voss N, Sadok N, **Goretzki S**, **Dohna-Schwake C**, Meyer MF, Mattheis S, Lang S, Stahr K. *HNO, 72(2):83-89*. <a href="https://pubmed.ncbi.nlm.nih.gov/38108853/" style="text-decoration: none;">PubMed</a>

* **Targeted PCR of Mucorales in pediatric bronchoalveolar lavage samples indicates low prevalence of airway colonization and sample contamination.**  
  Kavvalou A, Stehling F, **Goretzki SC**, **Dohna-Schwake C**, Rath PM, Steindor M. *Infectious diseases (London, England), 56(3):255-258*. <a href="https://pubmed.ncbi.nlm.nih.gov/38015560/" style="text-decoration: none;">PubMed</a>

* **Endoscopic ultrasound-guided ethanol ablation versus surgical resection of insulinomas.**  
  Jurgensen C, Eckart M, Haberbosch L, Tacke F, Sandforth A, Birkenfeld AL, Overkamp D, Daniels M, Mogl M, **Goretzki P**, Strasburger C, Mai K, Spranger J, Jumpertz von Schwartzenberg R. *Ultraschall in der Medizin (Stuttgart, Germany : 1980), 45(5):519-527*. <a href="https://pubmed.ncbi.nlm.nih.gov/37931914/" style="text-decoration: none;">PubMed</a>

* **Aortic Steal Correlates with Acute Organ Dysfunction and Short-Term Outcomes in Neonates with Vein of Galen Malformation.**  
  **Schwarz S**, Brevis Nunez F, Durr NR, Schlunz-Hendann M, Brassel F, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Neonatology, 121(1):106-115*. <a href="https://pubmed.ncbi.nlm.nih.gov/37906988/" style="text-decoration: none;">PubMed</a>


## 2023

* **A comparison of pediatric inflammatory multisystem syndrome temporarily-associated with SARS-CoV-2 and Kawasaki disease.**  
  Hufnagel M, Armann J, Jakob A, Doenhardt M, Diffloth N, Hospach A, Schneider DT, Trotter A, Roessler M, Schmitt J, Berner R. *Scientific reports, 13(1):1173*. <a href="https://pubmed.ncbi.nlm.nih.gov/36670127/" style="text-decoration: none;">PubMed</a>

* **Pediatric intensive care unit admissions network-rationale, framework and method of operation of a nationwide collaborative pediatric intensive care research network in Germany.**  
  **Bruns N**, **Dohna-Schwake C**, Olivieri M, Urschitz MS, Blomenkamp S, Frosch C, Lieftuchter V, Tomidis Chatzimanouil MK, Hoffmann F, Brenner S. *Frontiers in pediatrics, 11:1254935*. <a href="https://pubmed.ncbi.nlm.nih.gov/38269291/" style="text-decoration: none;">PubMed</a>

* **Increased SARS-CoV-2 reactive low avidity T cells producing inflammatory cytokines in pediatric post-acute COVID-19 sequelae (PASC).**  
  Paniskaki K, **Goretzki S**, Anft M, Konik MJ, Meister TL, Pfaender S, Lechtenberg K, Vogl M, Dogan B, Dolff S, Westhoff TH, Rohn H, Felderhoff-Mueser U, Stervbo U, Witzke O, **Dohna-Schwake C**, Babel N. *Pediatric allergy and immunology : official publication of the European Society of Pediatric Allergy and Immunology, 34(12):e14060*. <a href="https://pubmed.ncbi.nlm.nih.gov/38146118/" style="text-decoration: none;">PubMed</a>

* **Review on Pediatric Malignant Focal Liver Lesions with Imaging Evaluation: Part II.**  
  Dong Y, Cekuolis A, Schreiber-Dietrich D, Augustiniene R, **Schwarz S**, Moller K, Nourkami-Tutdibi N, Chen S, Cao JY, Huang YL, Wang Y, Taut H, Grevelding L, Dietrich CF. *Diagnostics (Basel, Switzerland), 13(24)*. <a href="https://pubmed.ncbi.nlm.nih.gov/38132242/" style="text-decoration: none;">PubMed</a>

* **Determinants of human papillomavirus vaccine hesitancy among Lebanese parents.**  
  Zakhour R, Tamim H, Faytrouni F, Makki M, **Hojeij R**, Charafeddine L. *PloS one, 18(12):e0295644*. <a href="https://pubmed.ncbi.nlm.nih.gov/38091310/" style="text-decoration: none;">PubMed</a>

* **Review on Pediatric Malignant Focal Liver Lesions with Imaging Evaluation: Part I.**  
  Dong Y, Cekuolis A, Schreiber-Dietrich D, Augustiniene R, **Schwarz S**, Moller K, Nourkami-Tutdibi N, Chen S, Cao JY, Huang YL, Wang Y, Taut H, Grevelding L, Dietrich CF. *Diagnostics (Basel, Switzerland), 13(23)*. <a href="https://pubmed.ncbi.nlm.nih.gov/38066809/" style="text-decoration: none;">PubMed</a>

* **Fading SARS-CoV-2 humoral VOC cross-reactivity and sustained cellular immunity in convalescent children and adolescents.**  
  Paniskaki K, **Goretzki S**, Anft M, Konik MJ, Lechtenberg K, Vogl M, Meister TL, Pfaender S, Zettler M, Jager J, Dolff S, Westhoff TH, Rohn H, Felderhoff-Mueser U, Stervbo U, Witzke O, **Dohna-Schwake C**, Babel N. *BMC infectious diseases, 23(1):818*. <a href="https://pubmed.ncbi.nlm.nih.gov/37993788/" style="text-decoration: none;">PubMed</a>

* **Music therapy modulates mothers' perception of their preterm infants.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Marschik PB, Felderhoff-Mueser U, **Bruns N**. *Frontiers in psychology, 14:1231741*. <a href="https://pubmed.ncbi.nlm.nih.gov/37928582/" style="text-decoration: none;">PubMed</a>

* **Distinct clinical phenotypes in paediatric cancer patients with sepsis are associated with different outcomes-an international multicentre retrospective study.**  
  Wosten-van Asperen RM, la Roi-Teeuw HM, van Amstel RB, Bos LD, Tissing WJ, Jordan I, **Dohna-Schwake C**, Bottari G, Pappachan J, Crazzolara R, Comoretto RI, Mizia-Malarz A, Moscatelli A, Sanchez-Martin M, Willems J, Rogerson CM, Bennett TD, Luo Y, Atreya MR, Faustino EVS, Geva A, Weiss SL, Schlapbach LJ, Sanchez-Pinto LN. *EClinicalMedicine, 65:102252*. <a href="https://pubmed.ncbi.nlm.nih.gov/37842550/" style="text-decoration: none;">PubMed</a>

* **[Time to Antibiotics (TTA) - Reassessment from the German Working Group for Fever and Neutropenia in Children and Adolescents (DGPI/GPOH)].**  
  Simon A, Lehrnbecher T, Baltaci Y, **Dohna-Schwake C**, Groll A, Laws HJ, Potratz J, Hufnagel M, Bochennek K. *Klinische Padiatrie, 235(6):331-341*. <a href="https://pubmed.ncbi.nlm.nih.gov/37751768/" style="text-decoration: none;">PubMed</a>

* **Erythrocyte transfusions are associated with retinopathy of prematurity in extremely low gestational age newborns.**  
  Glaser K, Hartel C, Dammann O, Herting E, Andres O, Speer CP, Gopel W, Stahl A. *Acta paediatrica (Oslo, Norway : 1992), 112(12):2507-2515*. <a href="https://pubmed.ncbi.nlm.nih.gov/37667535/" style="text-decoration: none;">PubMed</a>

* **Legionella pneumophila macrophage infectivity potentiator protein appendage domains modulate protein dynamics and inhibitor binding.**  
  Wiedemann C, Whittaker JJ, Perez Carrillo VH, **Goretzki B**, Dajka M, Tebbe F, Harder JM, Krajczy PR, Joseph B, Hausch F, Guskov A, Hellmich UA. *International journal of biological macromolecules, 252:126366*. <a href="https://pubmed.ncbi.nlm.nih.gov/37633566/" style="text-decoration: none;">PubMed</a>

* **Outcome after conservative or surgical treatment for new-onset epilepsy in children with cerebral cavernous malformation.**  
  Santos AN, Rauschenbach L, Riess C, Georgiades I, Ficilar B, Gallardo EG, Quesada CM, Li Y, Tippelt S, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Siegel AM, Benet A, Wrede KH, Sure U, Dammann P. *Seizure, 111:23-29*. <a href="https://pubmed.ncbi.nlm.nih.gov/37494759/" style="text-decoration: none;">PubMed</a>

* **Early Prediction of Mortality after Birth Asphyxia with the nSOFA.**  
  Dathe AK, Stein A, **Bruns N**, Craciun ED, Tuda L, Bialas J, Brasseler M, Felderhoff-Mueser U, Huening BM. *Journal of clinical medicine, 12(13)*. <a href="https://pubmed.ncbi.nlm.nih.gov/37445355/" style="text-decoration: none;">PubMed</a>

* **Crosstalk between regulatory elements in disordered TRPV4 N-terminus modulates lipid-dependent channel activity.**  
  **Goretzki B**, Wiedemann C, McCray BA, Schafer SL, Jansen J, Tebbe F, Mitrovic SA, Noth J, Cabezudo AC, Donohue JK, Jeffries CM, Steinchen W, Stengel F, Sumner CJ, Hummer G, Hellmich UA. *Nature communications, 14(1):4165*. <a href="https://pubmed.ncbi.nlm.nih.gov/37443299/" style="text-decoration: none;">PubMed</a>

* **Prognostic differences in grading and metastatic lymph node pattern in patients with small bowel neuroendocrine tumors.**  
  Reinhard L, Mogl MT, Benz F, Dukaczewska A, Butz F, Dobrindt EM, Tacke F, Pratschke J, **Goretzki PE**, Jann H. *Langenbeck's archives of surgery, 408(1):237*. <a href="https://pubmed.ncbi.nlm.nih.gov/37332044/" style="text-decoration: none;">PubMed</a>

* **Development of an Ultrasound Scoring System to Describe Brain Maturation in Preterm Infants.**  
  Stein A, Sody E, **Bruns N**, Felderhoff-Muser U. *AJNR. American journal of neuroradiology, 44(7):846-852*. <a href="https://pubmed.ncbi.nlm.nih.gov/37321856/" style="text-decoration: none;">PubMed</a>

* **[Special features of the diagnostics and treatment of hereditary primary hyperparathyroidism].**  
  Mogl MT, **Goretzki PE**. *Chirurgie (Heidelberg, Germany), 94(7):586-594*. <a href="https://pubmed.ncbi.nlm.nih.gov/37291366/" style="text-decoration: none;">PubMed</a>

* **Doppler Ultrasound Flow Reversal in the Superior Sagittal Sinus to Detect Cerebral Venous Congestion in Vein of Galen Malformation.**  
  **Schwarz S**, Brevis Nunez F, Durr NR, Brassel F, Schlunz-Hendann M, Feldkamp A, Rosenbaum T, Felderhoff-Muser U, Schulz K, **Dohna-Schwake C**, **Bruns N**. *AJNR. American journal of neuroradiology, 44(6):707-715*. <a href="https://pubmed.ncbi.nlm.nih.gov/37230540/" style="text-decoration: none;">PubMed</a>

* **Surgical Approach to Liver Metastases in GEP-NET in a Tertiary Reference Center.**  
  Butz F, Dukaczewska A, Jann H, Dobrindt EM, Reinhard L, Lurje G, Pratschke J, **Goretzki PE**, Schoning W, Mogl MT. *Cancers, 15(7)*. <a href="https://pubmed.ncbi.nlm.nih.gov/37046708/" style="text-decoration: none;">PubMed</a>

* **A flagellin-conjugate protein induces dual NLRC4- and NLRP3-inflammasome activation which modulates inflammatory cytokine secretion from macrophages.**  
  Lin YJ, Jamin A, Wolfheimer S, Fiedler A, Junker AC, **Goretzki A**, Scheurer S, Schulke S. *Frontiers in immunology, 14:1136669*. <a href="https://pubmed.ncbi.nlm.nih.gov/37026001/" style="text-decoration: none;">PubMed</a>

* **Giant Cavernous Malformation Mimicking an Infiltrative Intracranial Neoplasm in Children-Case Report and Systematic Review of the Literature.**  
  Gonzalez-Gallardo E, Rauschenbach L, Santos AN, Riess C, Li Y, Tippelt S, Della Marina A, **Dohna-Schwake C**, Sure U, Dammann P. *World neurosurgery, 174:30-41*. <a href="https://pubmed.ncbi.nlm.nih.gov/36889633/" style="text-decoration: none;">PubMed</a>

* **High Prevalence of Alternative Diagnoses in Children and Adolescents with Suspected Long COVID-A Single Center Cohort Study.**  
  **Goretzki SC**, Brasseler M, Dogan B, Huhne T, Bernard D, Schonecker A, Steindor M, Gangfuss A, Della Marina A, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Viruses, 15(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/36851793/" style="text-decoration: none;">PubMed</a>

* **A European International Multicentre Survey on the Current Practice of Perioperative Antibiotic Prophylaxis for Paediatric Liver Transplantations.**  
  Hauschild J, **Bruns N**, Lainka E, **Dohna-Schwake C**. *Antibiotics (Basel, Switzerland), 12(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/36830202/" style="text-decoration: none;">PubMed</a>

* **Functional outcome after initial and multiple intracerebral hemorrhages in children with cerebral cavernous malformations.**  
  Santos AN, Rauschenbach L, Gull HH, Dinger TF, Chihi M, Li Y, Tippelt S, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Wrede KH, Sure U, Dammann P. *European journal of neurology, 30(5):1364-1370*. <a href="https://pubmed.ncbi.nlm.nih.gov/36789485/" style="text-decoration: none;">PubMed</a>

* **IgD shapes the pre-immune naive B cell compartment in humans.**  
  Dirks J, Andres O, **Paul L**, Manukjan G, Schulze H, Morbach H. *Frontiers in immunology, 14:1096019*. <a href="https://pubmed.ncbi.nlm.nih.gov/36776874/" style="text-decoration: none;">PubMed</a>

* **Functional outcome after pediatric cerebral cavernous malformation surgery.**  
  Rauschenbach L, Santos AN, Dinger TF, Darkwah Oppong M, Li Y, Tippelt S, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Wrede KH, Sure U, Dammann P. *Scientific reports, 13(1):2286*. <a href="https://pubmed.ncbi.nlm.nih.gov/36759693/" style="text-decoration: none;">PubMed</a>

* **Pulmonary Sonography - Neonatal Diagnosis Part 2.**  
  **Schwarz S**. *Ultraschall in der Medizin (Stuttgart, Germany : 1980), 44(3):240-268*. <a href="https://pubmed.ncbi.nlm.nih.gov/36669528/" style="text-decoration: none;">PubMed</a>

* **[Pediatric neurocritical care].**  
  **Bruns N**, Schara-Schmidt U, **Dohna-Schwake C**. *Der Nervenarzt, 94(2):75-83*. <a href="https://pubmed.ncbi.nlm.nih.gov/36645451/" style="text-decoration: none;">PubMed</a>

* **Re: "Watch Out for Drug-Induced Coma and Burst Suppression Pattern in Infants and Children Mimicking Severe Neurological Disease" by Falsaperla et al.**  
  **Bruns N**. *Therapeutic hypothermia and temperature management, 13(1):42-43*. <a href="https://pubmed.ncbi.nlm.nih.gov/36576494/" style="text-decoration: none;">PubMed</a>

* **Immune Metabolism in TH2 Responses: New Opportunities to Improve Allergy Treatment - Cell Type-Specific Findings (Part 2).**  
  Lin YJ, **Goretzki A**, Rainer H, Zimmermann J, Schulke S. *Current allergy and asthma reports, 23(1):41-52*. <a href="https://pubmed.ncbi.nlm.nih.gov/36520269/" style="text-decoration: none;">PubMed</a>

* **Immune Metabolism in TH2 Responses: New Opportunities to Improve Allergy Treatment - Disease-Specific Findings (Part 1).**  
  **Goretzki A**, Zimmermann J, Rainer H, Lin YJ, Schulke S. *Current allergy and asthma reports, 23(1):29-40*. <a href="https://pubmed.ncbi.nlm.nih.gov/36441389/" style="text-decoration: none;">PubMed</a>

* **Feasibility of Video Consultation for Preterm Neurodevelopmental Follow-up Care During the COVID-19 Pandemic: Cohort Study.**  
  Albayrak B, Cordier LJ, **Greve S**, Teschler U, Dathe AK, Felderhoff-Muser U, Huning BM. *JMIR pediatrics and parenting, 6:e40940*. <a href="https://pubmed.ncbi.nlm.nih.gov/36409307/" style="text-decoration: none;">PubMed</a>

* **Stimulation of naive B cells with a fusion protein consisting of FlaA and Bet v 1 induces regulatory B cells ex vivo.**  
  **Goretzki A**, Lin YJ, Meier C, Dorn B, Wolfheimer S, Jamin A, Schott M, Wangorsch A, Vieths S, Jakob T, Scheurer S, Schulke S. *Allergy, 78(3):663-681*. <a href="https://pubmed.ncbi.nlm.nih.gov/36196479/" style="text-decoration: none;">PubMed</a>

* **Pulmonary Sonography - Neonatal Diagnosis Part 1.**  
  **Schwarz S**. *Ultraschall in der Medizin (Stuttgart, Germany : 1980), 44(1):14-35*. <a href="https://pubmed.ncbi.nlm.nih.gov/36075237/" style="text-decoration: none;">PubMed</a>

* **[Surgery of hyperparathyroidism].**  
  Mogl MT, **Goretzki PE**. *Chirurgie (Heidelberg, Germany), 94(2):177-188*. <a href="https://pubmed.ncbi.nlm.nih.gov/35925141/" style="text-decoration: none;">PubMed</a>

* **Dissection of the Brachiocephalic Trunk with Subsequent Occlusion of Both Common Carotid Arteries in a Newborn with an Aortic Arch Variant - From Cerebral Doppler Ultrasound to Diagnosis.**  
  **Schwarz S**, Tarusinov G, Brevis F. *Ultraschall in der Medizin (Stuttgart, Germany : 1980), 44(3):e164-e167*. <a href="https://pubmed.ncbi.nlm.nih.gov/35211943/" style="text-decoration: none;">PubMed</a>


## 2022

* **Music Therapy in Preterm Infants Reduces Maternal Distress.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Marschik PB, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 20(1)*. <a href="https://pubmed.ncbi.nlm.nih.gov/36613052/" style="text-decoration: none;">PubMed</a>

* **Organizational characteristics of European pediatric onco-critical care: An international cross-sectional survey.**  
  Nielsen JSA, Agbeko R, Bate J, Jordan I, **Dohna-Schwake C**, Potratz J, Moscatelli A, Bottari G, Pappachan J, Witt V, Crazzolara R, Amigoni A, Mizia-Malarz A, Sanchez Martin M, Willems J, van den Heuvel-Eibrink MM, Schlapbach LJ, Wosten-van Asperen RM. *Frontiers in pediatrics, 10:1024273*. <a href="https://pubmed.ncbi.nlm.nih.gov/36533247/" style="text-decoration: none;">PubMed</a>

* **Development of restrictive eating disorders in children and adolescents with long-COVID-associated smell and taste dysfunction.**  
  Brasseler M, Schonecker A, Steindor M, Della Marina A, **Bruns N**, Dogan B, Felderhoff-Muser U, Hebebrand J, **Dohna-Schwake C**, **Goretzki SC**. *Frontiers in pediatrics, 10:1022669*. <a href="https://pubmed.ncbi.nlm.nih.gov/36518782/" style="text-decoration: none;">PubMed</a>

* **Next Generation Sequencing of Free Microbial DNA for Rapid Identification of Pathogens in Critically Ill Children with Systemic Inflammatory Response Syndrome (SIRS).**  
  **Goretzki SC**, Schafer M, Dogan B, **Bruns N**, Tschiedel E, Rath PM, Voigt S, Grumaz S, Horvatek P, Schonberger S, Stehling F, Brenner T, **Dohna-Schwake C**. *Frontiers in bioscience (Landmark edition), 27(11):302*. <a href="https://pubmed.ncbi.nlm.nih.gov/36472101/" style="text-decoration: none;">PubMed</a>

* **Bronchial artery diameter in massive hemoptysis in cystic fibrosis.**  
  Dohna M, Kuhl H, Sutharsan S, **Dohna-Schwake C**, Vo Chieu VD, Hellms S, Kornemann N, Renz DM, Montag MJ. *BMC pulmonary medicine, 22(1):424*. <a href="https://pubmed.ncbi.nlm.nih.gov/36397043/" style="text-decoration: none;">PubMed</a>

* **Recurrent Late-Onset Sepsis in Extremely Low Birth Weight Infants Is Associated with Motor Deficits in Early School Age.**  
  Humberg A, Fortmann MI, Spiegler J, Rausch TK, Siller B, Silwedel C, Marissen J, Herting E, Gopel W, Hartel C. *Neonatology, 119(6):695-702*. <a href="https://pubmed.ncbi.nlm.nih.gov/36327925/" style="text-decoration: none;">PubMed</a>

* **Role of Glycolysis and Fatty Acid Synthesis in the Activation and T Cell-Modulating Potential of Dendritic Cells Stimulated with a TLR5-Ligand Allergen Fusion Protein.**  
  **Goretzki A**, Lin YJ, Zimmermann J, Rainer H, Junker AC, Wolfheimer S, Vieths S, Scheurer S, Schulke S. *International journal of molecular sciences, 23(20)*. <a href="https://pubmed.ncbi.nlm.nih.gov/36293550/" style="text-decoration: none;">PubMed</a>

* **Administrative data in pediatric critical care research-Potential, challenges, and future directions.**  
  **Bruns N**, Sorg AL, Felderhoff-Muser U, **Dohna-Schwake C**, Stang A. *Frontiers in pediatrics, 10:1014094*. <a href="https://pubmed.ncbi.nlm.nih.gov/36245724/" style="text-decoration: none;">PubMed</a>

* **The Disordered MAX N-terminus Modulates DNA Binding of the Transcription Factor MYC:MAX.**  
  Schutz S, Bergsdorf C, **Goretzki B**, Lingel A, Renatus M, Gossert AD, Jahnke W. *Journal of molecular biology, 434(22):167833*. <a href="https://pubmed.ncbi.nlm.nih.gov/36174765/" style="text-decoration: none;">PubMed</a>

* **The impact of age and electrode position on amplitude-integrated EEGs in children from 1 month to 17 years of age.**  
  **Greve S**, Loffelhardt VT, Della Marina A, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in neurology, 13:952193*. <a href="https://pubmed.ncbi.nlm.nih.gov/36090865/" style="text-decoration: none;">PubMed</a>

* **Modulation of dendritic cell metabolism by an MPLA-adjuvanted allergen product for specific immunotherapy.**  
  Zimmermann J, **Goretzki A**, Meier C, Wolfheimer S, Lin YJ, Rainer H, Krause M, Wedel S, Spies G, Fuhrer F, Vieths S, Scheurer S, Schulke S. *Frontiers in immunology, 13:916491*. <a href="https://pubmed.ncbi.nlm.nih.gov/36059475/" style="text-decoration: none;">PubMed</a>

* **Sleep problems in infancy and early school age in very preterm infants.**  
  Neitmann J, Hanke K, Humberg A, Siller B, Spiegler J, Juhnke K, Gilmore J, Odendahl R, Herting E, Gopel W, Hartel C, Fortmann I. *Early human development, 173:105656*. <a href="https://pubmed.ncbi.nlm.nih.gov/35987047/" style="text-decoration: none;">PubMed</a>

* **Impact of Physical Contact on Preterm Infants' Vital Sign Response to Live Music Therapy.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 19(15)*. <a href="https://pubmed.ncbi.nlm.nih.gov/35954880/" style="text-decoration: none;">PubMed</a>

* **Association of Administration of Surfactant Using Less Invasive Methods With Outcomes in Extremely Preterm Infants Less Than 27 Weeks of Gestation.**  
  Hartel C, Herting E, Humberg A, Hanke K, Mehler K, Keller T, Mauer I, Frieauff E, Meyer S, Thome UH, Wieg C, Schmidtke S, Kribs A, Gopel W. *JAMA network open, 5(8):e2225810*. <a href="https://pubmed.ncbi.nlm.nih.gov/35943742/" style="text-decoration: none;">PubMed</a>

* **Prevalence of osteopathologies in a single center cohort of survivors of childhood primary brain tumor.**  
  Schundeln MM, Fritzemeier S, **Goretzki SC**, Hauffa PK, Munteanu M, Kiewert C, Hauffa BP, Fleischhack G, Tippelt S, Grasemann C. *Frontiers in pediatrics, 10:913343*. <a href="https://pubmed.ncbi.nlm.nih.gov/35923779/" style="text-decoration: none;">PubMed</a>

* **[Differentiated Strategies for the Therapy of Liver Metastases in Gastro-entero-pancreatic Neuroendocrine Neoplasia].**  
  Mogl MT, Ollinger R, Jann H, Gebauer B, Fehrenbach U, Amthauer H, Wetz C, Schmelzle M, Raschzok N, Krenzien F, **Goretzki PE**, Pratschke J, Schoening W. *Zentralblatt fur Chirurgie, 147(3):270-280*. <a href="https://pubmed.ncbi.nlm.nih.gov/35705089/" style="text-decoration: none;">PubMed</a>

* **Extent of intrinsic disorder and NMR chemical shift assignments of the distal N-termini from human TRPV1, TRPV2 and TRPV3 ion channels.**  
  Wiedemann C, **Goretzki B**, Merz ZN, Tebbe F, Schmitt P, Hellmich UA. *Biomolecular NMR assignments, 16(2):289-296*. <a href="https://pubmed.ncbi.nlm.nih.gov/35666427/" style="text-decoration: none;">PubMed</a>

* **Pediatric ICU Admissions After Adolescent Suicide Attempts During the Pandemic.**  
  **Bruns N**, Willemsen L, Stang A, Kowall B, Holtkamp K, Kamp O, Dudda M, Hey F, Hoffmann F, Blankenburg J, Eifinger F, Fuchs H, Haase R, Baier J, Andree C, Heldmann M, Maldera V, Potratz J, Kurz D, Mand N, Doerfel C, Rothoeft T, Schultz M, Ohlert M, Silkenbaumer K, Boesing T, Indraswari F, Niemann F, Jahn P, Merker M, Braun N, Nunez FB, Engler M, Heimann K, Brasche M, Wolf G, Freymann H, Dercks M, Hoppenz M, Felderhoff-Muser U, **Dohna-Schwake C**. *Pediatrics, 150(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/35534988/" style="text-decoration: none;">PubMed</a>

* **Backbone NMR assignments of the extensive human and chicken TRPV4 N-terminal intrinsically disordered regions as important players in ion channel regulation.**  
  **Goretzki B**, Tebbe F, Mitrovic SA, Hellmich UA. *Biomolecular NMR assignments, 16(2):205-212*. <a href="https://pubmed.ncbi.nlm.nih.gov/35451798/" style="text-decoration: none;">PubMed</a>

* **Immune Metabolism-An Opportunity to Better Understand Allergic Pathology and Improve Treatment of Allergic Diseases?**  
  **Goretzki A**, Zimmermann J, Lin YJ, Schulke S. *Frontiers in allergy, 3:825931*. <a href="https://pubmed.ncbi.nlm.nih.gov/35386646/" style="text-decoration: none;">PubMed</a>

* **Impact of the First COVID Lockdown on Accident- and Injury-Related Pediatric Intensive Care Admissions in Germany-A Multicenter Study.**  
  **Bruns N**, Willemsen LY, Holtkamp K, Kamp O, Dudda M, Kowall B, Stang A, Hey F, Blankenburg J, Sabir H, Eifinger F, Fuchs H, Haase R, Andree C, Heldmann M, Potratz J, Kurz D, Schumann A, Muller-Knapp M, Mand N, Doerfel C, Dahlem P, Rothoeft T, Ohlert M, Silkenbaumer K, Dohle F, Indraswari F, Niemann F, Jahn P, Merker M, Braun N, Brevis Nunez F, Engler M, Heimann K, Wolf GK, Wulf D, Hankel S, Freymann H, Allgaier N, Knirsch F, Dercks M, Reinhard J, Hoppenz M, Felderhoff-Muser U, **Dohna-Schwake C**. *Children (Basel, Switzerland), 9(3)*. <a href="https://pubmed.ncbi.nlm.nih.gov/35327736/" style="text-decoration: none;">PubMed</a>

* **Multiubiquitination of TRPV4 reduces channel activity independent of surface localization.**  
  Aisenberg WH, McCray BA, Sullivan JM, Diehl E, DeVine LR, Alevy J, Bagnell AM, Carr P, Donohue JK, **Goretzki B**, Cole RN, Hellmich UA, Sumner CJ. *The Journal of biological chemistry, 298(4):101826*. <a href="https://pubmed.ncbi.nlm.nih.gov/35300980/" style="text-decoration: none;">PubMed</a>

* **TRPV4 mutations causing mixed neuropathy and skeletal phenotypes result in severe gain of function.**  
  Taga A, Peyton MA, **Goretzki B**, Gallagher TQ, Ritter A, Harper A, Crawford TO, Hellmich UA, Sumner CJ, McCray BA. *Annals of clinical and translational neurology, 9(3):375-391*. <a href="https://pubmed.ncbi.nlm.nih.gov/35170874/" style="text-decoration: none;">PubMed</a>

* **Role of extracorporeal membrane oxygenation in pediatric cancer patients: a systematic review and meta-analysis of observational studies.**  
  Slooff V, Hoogendoorn R, Nielsen JSA, Pappachan J, Amigoni A, Caramelli F, Aziz O, Wildschut E, Verbruggen S, Crazzolara R, **Dohna-Schwake C**, Potratz J, Willems J, Llevadias J, Moscatelli A, Montaguti A, Bottari G, Di Nardo M, Schlapbach L, Wosten-van Asperen R. *Annals of intensive care, 12(1):8*. <a href="https://pubmed.ncbi.nlm.nih.gov/35092500/" style="text-decoration: none;">PubMed</a>

* **Hyperkalemia Following Parathyroidectomy in Patients with Renal Hyperparathyroidism-New Thresholds for Urgent Perioperative Dialysis.**  
  Bures C, Uluk Y, Besmens M, Akca A, Dobrindt EM, Pratschke J, **Goretzki P**, Mogl M, Uluk D. *Journal of clinical medicine, 11(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/35054103/" style="text-decoration: none;">PubMed</a>

* **Breastfeeding and Readmission for Hyperbilirubinemia in Late Preterm and Term Infants in Beirut, Lebanon.**  
  Hanin EA, Rayan H, Hani T, Taleb J, Dany AH, Lama C. *Indian pediatrics, 59(3):218-221*. <a href="https://pubmed.ncbi.nlm.nih.gov/34969942/" style="text-decoration: none;">PubMed</a>

* **Functional Short-Term Outcomes and Mortality in Children with Severe Traumatic Brain Injury: Comparing Decompressive Craniectomy and Medical Management.**  
  **Bruns N**, Kamp O, Lange K, Lefering R, Felderhoff-Muser U, Dudda M, **Dohna-Schwake C**. *Journal of neurotrauma, 39(13-14):944-953*. <a href="https://pubmed.ncbi.nlm.nih.gov/34877889/" style="text-decoration: none;">PubMed</a>

* **Antibiotics in critically ill children-a narrative review on different aspects of a rational approach.**  
  **Bruns N**, **Dohna-Schwake C**. *Pediatric research, 91(2):440-446*. <a href="https://pubmed.ncbi.nlm.nih.gov/34873285/" style="text-decoration: none;">PubMed</a>

* **Natural Course of Cerebral Cavernous Malformations in Children: A Five-Year Follow-Up Study.**  
  Santos AN, Rauschenbach L, Saban D, Chen B, Herten A, Dinger TF, Li Y, Tippelt S, Della Marina A, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Wrede KH, Sure U, Dammann P. *Stroke, 53(3):817-824*. <a href="https://pubmed.ncbi.nlm.nih.gov/34706565/" style="text-decoration: none;">PubMed</a>

* **Association of ApoE Genotypes and Recovery From Intracerebral Hemorrhage in Very Low Birth Weight Infants.**  
  Humberg A, Dzietko M, Schulz S, Felderhoff-Muser U, Huning B, Stein A, Fortmann MI, Marissen J, Rausch TK, Herting E, Hartel C, Gopel W. *Stroke, 53(2):514-522*. <a href="https://pubmed.ncbi.nlm.nih.gov/34619984/" style="text-decoration: none;">PubMed</a>

* **Latency duration of preterm premature rupture of membranes and neonatal outcome: a retrospective single-center experience.**  
  Muller H, Stahling AC, **Bruns N**, Weiss C, Ai M, Koninger A, Felderhoff-Muser U. *European journal of pediatrics, 181(2):801-811*. <a href="https://pubmed.ncbi.nlm.nih.gov/34605998/" style="text-decoration: none;">PubMed</a>


## 2021

* **What if... I Asked Cancer Patients About Psychological Distress? Barriers in Psycho-Oncological Screening From the Perspective of Nurses-A Qualitative Analysis.**  
  Dreismann L, **Goretzki A**, Ginger V, Zimmermann T. *Frontiers in psychiatry, 12:786691*. <a href="https://pubmed.ncbi.nlm.nih.gov/35153856/" style="text-decoration: none;">PubMed</a>

* **Characterization of aEEG During Sleep and Wakefulness in Healthy Children.**  
  Loffelhardt VT, Della Marina A, **Greve S**, Muller H, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in pediatrics, 9:773188*. <a href="https://pubmed.ncbi.nlm.nih.gov/35127587/" style="text-decoration: none;">PubMed</a>

* **The Fusion Protein rFlaA:Betv1 Modulates DC Responses by a p38-MAPK and COX2-Dependent Secretion of PGE(2) from Epithelial Cells.**  
  Lin YJ, Flaczyk A, Wolfheimer S, **Goretzki A**, Jamin A, Wangorsch A, Vieths S, Scheurer S, Schulke S. *Cells, 10(12)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34943923/" style="text-decoration: none;">PubMed</a>

* **Immune Metabolism of IL-4-Activated B Cells and Th2 Cells in the Context of Allergic Diseases.**  
  Lin YJ, **Goretzki A**, Schulke S. *Frontiers in immunology, 12:790658*. <a href="https://pubmed.ncbi.nlm.nih.gov/34925372/" style="text-decoration: none;">PubMed</a>

* **Parents' Perception of Family-Centered Music Therapy with Stable Preterm Infants.**  
  Kobus S, Diezel M, Huening B, Dewan MV, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 18(23)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34886539/" style="text-decoration: none;">PubMed</a>

* **The Flagellin:Allergen Fusion Protein rFlaA:Betv1 Induces a MyD88- and MAPK-Dependent Activation of Glucose Metabolism in Macrophages.**  
  Lin YJ, Papp G, Miskey C, Fiedler A, **Goretzki A**, Wolfheimer S, Zimmermann J, Crauwels P, Ivics Z, van Zandbergen G, Vieths S, Scheurer S, Schulke S. *Cells, 10(10)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34685593/" style="text-decoration: none;">PubMed</a>

* **Hospitalization and Morbidity Rates After Pediatric Traumatic Brain Injury: A Nation-Wide Population-Based Analysis.**  
  **Bruns N**, Trocchi P, Felderhoff-Muser U, **Dohna-Schwake C**, Stang A. *Frontiers in pediatrics, 9:747743*. <a href="https://pubmed.ncbi.nlm.nih.gov/34660495/" style="text-decoration: none;">PubMed</a>

* **Music Therapy Is Effective during Sleep in Preterm Infants.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 18(16)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34443994/" style="text-decoration: none;">PubMed</a>

* **Coil embolisation for massive haemoptysis in cystic fibrosis.**  
  Dohna M, Renz DM, Stehling F, **Dohna-Schwake C**, Sutharsan S, Neurohr C, Wirtz H, Eickmeier O, Grosse-Onnebrink J, Sauerbrey A, Soditt V, Poplawska K, Wacker F, Montag MJ. *BMJ open respiratory research, 8(1)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34385150/" style="text-decoration: none;">PubMed</a>

* **Next-generation sequencing diagnostics of bacteremia in pediatric sepsis.**  
  Schmoch T, Westhoff JH, Decker SO, Skarabis A, Hoffmann GF, **Dohna-Schwake C**, Felderhoff-Muser U, Skolik C, Feisst M, Klose C, Bruckner T, Luntz S, Weigand MA, Sohn K, Brenner T. *Medicine, 100(25):e26403*. <a href="https://pubmed.ncbi.nlm.nih.gov/34160425/" style="text-decoration: none;">PubMed</a>

* **Epidemiology, Diagnostics, and Management of Vein of Galen Malformation.**  
  Brevis Nunez F, **Dohna-Schwake C**. *Pediatric neurology, 119:50-55*. <a href="https://pubmed.ncbi.nlm.nih.gov/33895584/" style="text-decoration: none;">PubMed</a>

* **Immune metabolism in allergies, does it matter?-A review of immune metabolic basics and adaptations associated with the activation of innate immune cells in allergy.**  
  **Goretzki A**, Lin YJ, Schulke S. *Allergy, 76(11):3314-3331*. <a href="https://pubmed.ncbi.nlm.nih.gov/33811351/" style="text-decoration: none;">PubMed</a>

* **Unstructural Biology of TRP Ion Channels: The Role of Intrinsically Disordered Regions in Channel Function and Regulation.**  
  **Goretzki B**, Guhl C, Tebbe F, Harder JM, Hellmich UA. *Journal of molecular biology, 433(17):166931*. <a href="https://pubmed.ncbi.nlm.nih.gov/33741410/" style="text-decoration: none;">PubMed</a>

* **NAT2 polymorphisms as a cause of metamizole-induced agranulocytosis.**  
  Radulovic I, Dogan B, **Dohna-Schwake C**, Schundeln MM, Siffert W, Mohlendick B. *Pharmacogenetics and genomics, 31(6):140-143*. <a href="https://pubmed.ncbi.nlm.nih.gov/33675325/" style="text-decoration: none;">PubMed</a>

* **Neuropathy-causing TRPV4 mutations disrupt TRPV4-RhoA interactions and impair neurite extension.**  
  McCray BA, Diehl E, Sullivan JM, Aisenberg WH, Zaccor NW, Lau AR, Rich DJ, **Goretzki B**, Hellmich UA, Lloyd TE, Sumner CJ. *Nature communications, 12(1):1444*. <a href="https://pubmed.ncbi.nlm.nih.gov/33664271/" style="text-decoration: none;">PubMed</a>

* **Insulinoma in pregnancy (a case presentation and systematic review of the literature).**  
  Dobrindt EM, Mogl M, **Goretzki PE**, Pratschke J, Dukaczewska AK. *Rare tumors, 13:2036361320986647*. <a href="https://pubmed.ncbi.nlm.nih.gov/33613925/" style="text-decoration: none;">PubMed</a>

* **EVALI (E-cigarette or vaping product use associated lung injury): First case report of an adolescent in Europe.**  
  Schafer M, Steindor M, Stehling F, **Dohna-Schwake C**. *Pediatric pulmonology, 56(5):1274-1275*. <a href="https://pubmed.ncbi.nlm.nih.gov/33497507/" style="text-decoration: none;">PubMed</a>

* **A rare case of a patient with a high grade neuroendocrine tumor developing neutropenic sepsis after receiving PRRT combined with Capecitabine or Temozolomide: A case report.**  
  Ozdirik B, Amthauer H, Schatka I, **Goretzki PE**, Mogl MT, Fehrenbach U, Tacke F, Jann H, Roderburg C. *Molecular and clinical oncology, 14(1):20*. <a href="https://pubmed.ncbi.nlm.nih.gov/33363730/" style="text-decoration: none;">PubMed</a>

* **Sedation for bronchoscopy in children: A prospective randomized double-blinded trial.**  
  Tschiedel E, Eckerland M, Felderhoff-Mueser U, **Dohna-Schwake C**, Stehling F. *Pediatric pulmonology, 56(5):1221-1229*. <a href="https://pubmed.ncbi.nlm.nih.gov/33336889/" style="text-decoration: none;">PubMed</a>

* **aEEG as a useful tool for neuromonitoring in critically ill children - Current evidence and knowledge gaps.**  
  **Bruns N**, Felderhoff-Muser U, **Dohna-Schwake C**. *Acta paediatrica (Oslo, Norway : 1992), 110(4):1132-1140*. <a href="https://pubmed.ncbi.nlm.nih.gov/33210762/" style="text-decoration: none;">PubMed</a>

* **Aminoglycosides were associated with higher rates of surgical patent ductus arteriosus closure in preterm infants.**  
  Marissen J, Erdmann H, Bockenholt K, Hoppenz M, Rausch TK, Hartel C, Herting E, Gopel W. *Acta paediatrica (Oslo, Norway : 1992), 110(3):826-832*. <a href="https://pubmed.ncbi.nlm.nih.gov/32810301/" style="text-decoration: none;">PubMed</a>


## 2020

* **Iatrogenic severe hyperglycemia due to parenteral administration of glucose in children - a case series.**  
  **Bruns N**, Grosse Lordemann A, Rasche T, Meyburg J, Kruger M, Wieg C, Gratopp A, Hoppenz M, Heitmann F, Hoppen T, Loffler G, Felderhoff-Muser U, **Dohna-Schwake C**. *Italian journal of pediatrics, 46(1):179*. <a href="https://pubmed.ncbi.nlm.nih.gov/33261643/" style="text-decoration: none;">PubMed</a>

* **Unusual Presentation of Polyautoimmunity and Renal Tubular Acidosis in an Adolescent With Hashimoto's Thyroiditis and Central Pontine Myelinolysis.**  
  **Bruns N**, Finkelberg I, Al-Attrach I, Hoyer PF, Buscher R. *Frontiers in endocrinology, 11:548877*. <a href="https://pubmed.ncbi.nlm.nih.gov/33162932/" style="text-decoration: none;">PubMed</a>

* **More Than 500 Kids Could Be Saved Each Year! Ten Consensus Actions to Improve Quality of Pediatric Resuscitation in DACH-Countries (Austria, Germany, and Switzerland).**  
  Jung P, Brenner S, Bachmann I, Both C, Cardona F, **Dohna-Schwake C**, Eich C, Eifinger F, Huth R, Heimberg E, Landsleitner B, Olivieri M, Sasse M, Weisner T, Wagner M, Warnke G, Ziegler B, Boettiger BW, Nadkarni V, Hoffmann F. *Frontiers in pediatrics, 8:549710*. <a href="https://pubmed.ncbi.nlm.nih.gov/33117762/" style="text-decoration: none;">PubMed</a>

* **Further evidence for POMK as candidate gene for WWS with meningoencephalocele.**  
  **Paul L**, Rupprich K, Della Marina A, Stein A, Elgizouli M, Kaiser FJ, Schweiger B, Koninger A, Iannaccone A, Hehr U, Kolbel H, Roos A, Schara-Schmidt U, Kuechler A. *Orphanet journal of rare diseases, 15(1):242*. <a href="https://pubmed.ncbi.nlm.nih.gov/32907597/" style="text-decoration: none;">PubMed</a>

* **How to Administer Near-Infrared Spectroscopy in Critically ill Neonates, Infants, and Children.**  
  **Bruns N**, Moosmann J, Munch F, **Dohna-Schwake C**, Woelfle J, Cesnjevar R, Dittrich S, Felderhoff-Muser U, Muller H. *Journal of visualized experiments : JoVE(162)*. <a href="https://pubmed.ncbi.nlm.nih.gov/32894267/" style="text-decoration: none;">PubMed</a>

* **Preterm birth and sustained inflammation: consequences for the neonate.**  
  Humberg A, Fortmann I, Siller B, Kopp MV, Herting E, Gopel W, Hartel C. *Seminars in immunopathology, 42(4):451-468*. <a href="https://pubmed.ncbi.nlm.nih.gov/32661735/" style="text-decoration: none;">PubMed</a>

* **[Thyroid nodules: The Guidelines of the American Thyroid Association from a European Perspective].**  
  Derwahl KM, **Goretzki P**. *Deutsche medizinische Wochenschrift (1946), 145(17):1227-1235*. <a href="https://pubmed.ncbi.nlm.nih.gov/32634841/" style="text-decoration: none;">PubMed</a>

* **Influence of Parathyroidectomy on Kidney Graft Function in Secondary and Tertiary Hyperparathyroidism.**  
  Bures C, Seika P, Skachko T, Dobrindt EM, Rayes N, Pratschke J, **Goretzki PE**, Mogl MT. *Transplantation proceedings, 52(10):3134-3143*. <a href="https://pubmed.ncbi.nlm.nih.gov/32402458/" style="text-decoration: none;">PubMed</a>

* **Long-term outcome of surgical resection in patients with gastroenteropancreatic neuroendocrine neoplasia: results from a German nation-wide multi-centric registry.**  
  Begum N, Maasberg S, Pascher A, Plockinger U, Gress TM, Wurst C, Weber F, Raffel A, Krausch M, Holzer K, Bartsch DK, Musholt TJ, Keck T, Anlauf M, Rinke A, Pape UF, **Goretzki PE**. *Langenbeck's archives of surgery, 405(2):145-154*. <a href="https://pubmed.ncbi.nlm.nih.gov/32372309/" style="text-decoration: none;">PubMed</a>

* **Postnatal serum creatinine is elevated in preterm infants with PPROM-induced anhydramnios.**  
  **Bruns N**, Stahling AC, **Greve S**, Weiss C, Koninger A, Felderhoff-Muser U, Muller H. *Pediatrics and neonatology, 61(4):414-419*. <a href="https://pubmed.ncbi.nlm.nih.gov/32278744/" style="text-decoration: none;">PubMed</a>

* **The Impact of Gender on Diagnosis and Treatment in Endocrine Surgery.**  
  **Goretzki PE**. *Visceral medicine, 36(1):2*. <a href="https://pubmed.ncbi.nlm.nih.gov/32110649/" style="text-decoration: none;">PubMed</a>

* **Two-year outcome data suggest that less invasive surfactant administration (LISA) is safe. Results from the follow-up of the randomized controlled AMV (avoid mechanical ventilation) study.**  
  Herting E, Kribs A, Hartel C, von der Wense A, Weller U, Hoehn T, Vochem M, Moller J, Wieg C, Roth B, Gopel W. *European journal of pediatrics, 179(8):1309-1313*. <a href="https://pubmed.ncbi.nlm.nih.gov/32067100/" style="text-decoration: none;">PubMed</a>

* **Surgical necrotizing enterocolitis but not spontaneous intestinal perforation is associated with adverse neurological outcome at school age.**  
  Humberg A, Spiegler J, Fortmann MI, Zemlin M, Marissen J, Swoboda I, Rausch TK, Herting E, Gopel W, Hartel C. *Scientific reports, 10(1):2373*. <a href="https://pubmed.ncbi.nlm.nih.gov/32047169/" style="text-decoration: none;">PubMed</a>

* **aEEG Use in Pediatric Critical Care-An Online Survey.**  
  **Bruns N**, Felderhoff-Muser U, **Dohna-Schwake C**, Woelfle J, Muller H. *Frontiers in pediatrics, 8:3*. <a href="https://pubmed.ncbi.nlm.nih.gov/32039124/" style="text-decoration: none;">PubMed</a>

* **NMR assignments of a dynamically perturbed and dimerization inhibited N-terminal domain variant of a spider silk protein from E. australis.**  
  **Goretzki B**, Heiby JC, Hacker C, Neuweiler H, Hellmich UA. *Biomolecular NMR assignments, 14(1):67-71*. <a href="https://pubmed.ncbi.nlm.nih.gov/31786743/" style="text-decoration: none;">PubMed</a>

* **Correlation of lateral ventricular size and deep gray matter volume in MRI at term equivalent age with neurodevelopmental outcome at a corrected age of 24 months and with handedness in preterm infants.**  
  Storbeck T, **Bruns N**, Weiss C, Felderhoff-Muser U, Muller H. *European journal of pediatrics, 179(2):271-278*. <a href="https://pubmed.ncbi.nlm.nih.gov/31724086/" style="text-decoration: none;">PubMed</a>

* **Bacterial infections in children after liver transplantation: A single-center surveillance study of 345 consecutive transplantations.**  
  **Dohna Schwake C**, Guiddir T, Cuzon G, Benissa MR, Dubois C, Miatello J, Merchaoui Z, Durand P, Tissieres P. *Transplant infectious disease : an official journal of the Transplantation Society, 22(1):e13208*. <a href="https://pubmed.ncbi.nlm.nih.gov/31693773/" style="text-decoration: none;">PubMed</a>

* **Early Postoperative Fasting Serum Glucose Levels are Useful in Depicting Future Diabetes Mellitus in Patients with Curative Insulinoma Surgery.**  
  Akca A, Starke AAR, Dobek A, Ulrich A, **Goretzki PE**. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 128(3):158-163*. <a href="https://pubmed.ncbi.nlm.nih.gov/31039599/" style="text-decoration: none;">PubMed</a>

* **Neurological outcome at 24 months corrected age of prematurely born infants after preterm premature rupture of membranes (PPROM) of at least 7 days: a two-center experience in Germany.**  
  Muller H, Storbeck T, Katzer D, **Bruns N**, Wossner-Stegmann G, Ai M, Koninger A, Muller A, Felderhoff-Muser U, Bagci S. *The journal of maternal-fetal & neonatal medicine : the official journal of the European Association of Perinatal Medicine, the Federation of Asia and Oceania Perinatal Societies, the International Society of Perinatal Obstetricians, 33(8):1315-1320*. <a href="https://pubmed.ncbi.nlm.nih.gov/30153748/" style="text-decoration: none;">PubMed</a>


## 2019

* **Research priorities in pediatric onco-critical care: an international Delphi consensus study.**  
  Soeteman M, Potratz J, Nielsen JSA, Willems J, Valla FV, Brierley J, Wosten-van Asperen RM. *Intensive care medicine, 45(11):1681-1683*. <a href="https://pubmed.ncbi.nlm.nih.gov/31444505/" style="text-decoration: none;">PubMed</a>

* **Total Versus Near-total Thyroidectomy in Graves Disease: Results of the Randomized Controlled Multicenter TONIG-trial.**  
  Maurer E, Maschuw K, Reuss A, Zieren HU, Zielke A, **Goretzki P**, Simon D, Dotzenrath C, Steinmuller T, Jahne J, Kemen M, Coerper S, Leister I, Nies C, Hartel M, Turler A, Holzer K, Agha A, Knoop M, Musholt T, Aminossadati B, Bartsch DK. *Annals of surgery, 270(5):755-761*. <a href="https://pubmed.ncbi.nlm.nih.gov/31634179/" style="text-decoration: none;">PubMed</a>

* **Methionine in a protein hydrophobic core drives tight interactions required for assembly of spider silk.**  
  Heiby JC, **Goretzki B**, Johnson CM, Hellmich UA, Neuweiler H. *Nature communications, 10(1):4378*. <a href="https://pubmed.ncbi.nlm.nih.gov/31558722/" style="text-decoration: none;">PubMed</a>

* **Amplitude-Integrated EEG for Neurological Assessment and Seizure Detection in a German Pediatric Intensive Care Unit.**  
  **Bruns N**, Sanchez-Albisua I, Weiss C, Tschiedel E, **Dohna-Schwake C**, Felderhoff-Muser U, Muller H. *Frontiers in pediatrics, 7:358*. <a href="https://pubmed.ncbi.nlm.nih.gov/31555625/" style="text-decoration: none;">PubMed</a>

* **PICU mortality of children with cancer admitted to pediatric intensive care unit a systematic review and meta-analysis.**  
  Wosten-van Asperen RM, van Gestel JPJ, van Grotel M, Tschiedel E, **Dohna-Schwake C**, Valla FV, Willems J, Angaard Nielsen JS, Krause MF, Potratz J, van den Heuvel-Eibrink MM, Brierley J. *Critical reviews in oncology/hematology, 142:153-163*. <a href="https://pubmed.ncbi.nlm.nih.gov/31404827/" style="text-decoration: none;">PubMed</a>

* **Adjuvant Allergen Fusion Proteins as Novel Tools for the Treatment of Type I Allergies.**  
  Blanco-Perez F, Papp G, **Goretzki A**, Moller T, Anzaghe M, Schulke S. *Archivum immunologiae et therapiae experimentalis, 67(5):273-293*. <a href="https://pubmed.ncbi.nlm.nih.gov/31222414/" style="text-decoration: none;">PubMed</a>

* **A DELPHI study on aspects of study design to overcome knowledge gaps on the burden of disease caused by serogroup B invasive meningococcal disease.**  
  Marten O, Koerber F, Bloom D, Bullinger M, Buysse C, Christensen H, De Wals P, **Dohna-Schwake C**, Henneke P, Kirchner M, Knuf M, Lawrenz B, Monteiro AL, Sevilla JP, Van de Velde N, Welte R, Wright C, Greiner W. *Health and quality of life outcomes, 17(1):87*. <a href="https://pubmed.ncbi.nlm.nih.gov/31118091/" style="text-decoration: none;">PubMed</a>

* **19F NMR as a versatile tool to study membrane protein structure and dynamics.**  
  Rose-Sperling D, Tran MA, Lauth LM, **Goretzki B**, Hellmich UA. *Biological chemistry, 400(10):1277-1288*. <a href="https://pubmed.ncbi.nlm.nih.gov/31004560/" style="text-decoration: none;">PubMed</a>

* **NFkappaB- and MAP-Kinase Signaling Contribute to the Activation of Murine Myeloid Dendritic Cells by a Flagellin A:Allergen Fusion Protein.**  
  Moeller T, Wolfheimer S, **Goretzki A**, Scheurer S, Schulke S. *Cells, 8(4)*. <a href="https://pubmed.ncbi.nlm.nih.gov/30991709/" style="text-decoration: none;">PubMed</a>

* **Surgical therapy of adrenal tumors: guidelines from the German Association of Endocrine Surgeons (CAEK).**  
  Lorenz K, Langer P, Niederle B, Alesina P, Holzer K, Nies C, Musholt T, **Goretzki PE**, Rayes N, Quinkler M, Waldmann J, Simon D, Trupka A, Ladurner R, Hallfeldt K, Zielke A, Saeger D, Poppel T, Kukuk G, Hotker A, Schabram P, Schopf S, Dotzenrath C, Riss P, Steinmuller T, Kopp I, Vorlander C, Walz MK, Bartsch DK. *Langenbeck's archives of surgery, 404(4):385-401*. <a href="https://pubmed.ncbi.nlm.nih.gov/30937523/" style="text-decoration: none;">PubMed</a>

* **The vaccine adjuvant MPLA activates glycolytic metabolism in mouse mDC by a JNK-dependent activation of mTOR-signaling.**  
  Blanco-Perez F, **Goretzki A**, Wolfheimer S, Schulke S. *Molecular immunology, 106:159-169*. <a href="https://pubmed.ncbi.nlm.nih.gov/30623816/" style="text-decoration: none;">PubMed</a>

* **Visual Distraction During Sleep Induction with Propofol in Pediatric Patients - Can Cartoon Watching Influence the Sedative Dose?**  
  Tschiedel E, Beckers K, Grosse-Lordemann A, Basu O, Felderhoff-Muser U, **Dohna-Schwake C**. *Klinische Padiatrie, 231(1):35-40*. <a href="https://pubmed.ncbi.nlm.nih.gov/30231262/" style="text-decoration: none;">PubMed</a>


## 2018

* **[Not Available].**  
  Mogl MT, **Goretzki PE**. *Zentralblatt fur Chirurgie, 143(5):451-454*. <a href="https://pubmed.ncbi.nlm.nih.gov/30357782/" style="text-decoration: none;">PubMed</a>

* **[Neuroendocrine Neoplasia within the German NET Registry].**  
  No authors listed. *Zeitschrift fur Gastroenterologie, 56(10):1237-1246*. <a href="https://pubmed.ncbi.nlm.nih.gov/30304748/" style="text-decoration: none;">PubMed</a>

* **International neuromonitoring study group guidelines 2018: Part II: Optimal recurrent laryngeal nerve management for invasive thyroid cancer-incorporation of surgical, laryngeal, and neural electrophysiologic data.**  
  Wu CW, Dionigi G, Barczynski M, Chiang FY, Dralle H, Schneider R, Al-Quaryshi Z, Angelos P, Brauckhoff K, Brooks JA, Cernea CR, Chaplin J, Chen AY, Davies L, Diercks GR, Duh QY, Fundakowski C, **Goretzki PE**, Hales NW, Hartl D, Kamani D, Kandil E, Kyriazidis N, Liddy W, Miyauchi A, Orloff L, Rastatter JC, Scharpf J, Serpell J, Shin JJ, Sinclair CF, Stack BC Jr, Tolley NS, Slycke SV, Snyder SK, Urken ML, Volpi E, Witterick I, Wong RJ, Woodson G, Zafereo M, Randolph GW. *The Laryngoscope, 128 Suppl 3:S18-S27*. <a href="https://pubmed.ncbi.nlm.nih.gov/30291765/" style="text-decoration: none;">PubMed</a>

* **International neural monitoring study group guideline 2018 part I: Staging bilateral thyroid surgery with monitoring loss of signal.**  
  Schneider R, Randolph GW, Dionigi G, Wu CW, Barczynski M, Chiang FY, Al-Quaryshi Z, Angelos P, Brauckhoff K, Cernea CR, Chaplin J, Cheetham J, Davies L, **Goretzki PE**, Hartl D, Kamani D, Kandil E, Kyriazidis N, Liddy W, Orloff L, Scharpf J, Serpell J, Shin JJ, Sinclair CF, Singer MC, Snyder SK, Tolley NS, Van Slycke S, Volpi E, Witterick I, Wong RJ, Woodson G, Zafereo M, Dralle H. *The Laryngoscope, 128 Suppl 3:S1-S17*. <a href="https://pubmed.ncbi.nlm.nih.gov/30289983/" style="text-decoration: none;">PubMed</a>

* **Curative and palliative surgery in patients with neuroendocrine tumors of the gastro-entero-pancreatic (GEP) tract.**  
  **Goretzki PE**, Mogl MT, Akca A, Pratschke J. *Reviews in endocrine & metabolic disorders, 19(2):169-178*. <a href="https://pubmed.ncbi.nlm.nih.gov/30280290/" style="text-decoration: none;">PubMed</a>

* **Structural Basis of TRPV4 N Terminus Interaction with Syndapin/PACSIN1-3 and PIP(2).**  
  **Goretzki B**, Glogowski NA, Diehl E, Duchardt-Ferner E, Hacker C, Gaudet R, Hellmich UA. *Structure (London, England : 1993), 26(12):1583-1593.e5*. <a href="https://pubmed.ncbi.nlm.nih.gov/30244966/" style="text-decoration: none;">PubMed</a>

* **[Practice guideline neuroendocrine tumors - AWMF-Reg. 021-27].**  
  No authors listed. *Zeitschrift fur Gastroenterologie, 56(6):583-681*. <a href="https://pubmed.ncbi.nlm.nih.gov/29890561/" style="text-decoration: none;">PubMed</a>

* **[Update of the S2k guidelines : Surgical treatment of benign thyroid diseases].**  
  Musholt TJ, Bockisch A, Clerici T, Dotzenrath C, Dralle H, **Goretzki PE**, Hermann M, Holzer K, Karges W, Krude H, Kussmann J, Lorenz K, Luster M, Niederle B, Nies C, Riss P, Schabram J, Schabram P, Schmid KW, Simon D, Spitzweg C, Steinmuller T, Trupka A, Vorlander C, Weber T, Bartsch DK. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 89(9):699-709*. <a href="https://pubmed.ncbi.nlm.nih.gov/29876616/" style="text-decoration: none;">PubMed</a>

* **Relationship between brain function (aEEG) and brain structure (MRI) and their predictive value for neurodevelopmental outcome of preterm infants.**  
  Huning B, Storbeck T, **Bruns N**, Dransfeld F, Hobrecht J, Karpienski J, Sirin S, Schweiger B, Weiss C, Felderhoff-Muser U, Muller H. *European journal of pediatrics, 177(8):1181-1189*. <a href="https://pubmed.ncbi.nlm.nih.gov/29789947/" style="text-decoration: none;">PubMed</a>

* **In Reply.**  
  Bartsch DK, Luster M, **Goretzki PE**. *Deutsches Arzteblatt international, 115(13):221*. <a href="https://pubmed.ncbi.nlm.nih.gov/29669680/" style="text-decoration: none;">PubMed</a>

* **Antifungal Treatment and Outcome in Very Low Birth Weight Infants: A Population-based Observational Study of the German Neonatal Network.**  
  Fortmann I, Hartz A, Paul P, Pulzer F, Muller A, Bottger R, Proquitte H, Dawczynski K, Simon A, Rupp J, Herting E, Gopel W, Hartel C. *The Pediatric infectious disease journal, 37(11):1165-1171*. <a href="https://pubmed.ncbi.nlm.nih.gov/29601449/" style="text-decoration: none;">PubMed</a>

* **Indications for the Surgical Management of Benign Goiter in Adults.**  
  Bartsch DK, Luster M, Buhr HJ, Lorenz D, Germer CT, **Goretzki PE**. *Deutsches Arzteblatt international, 115(1-02):1-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/29345225/" style="text-decoration: none;">PubMed</a>

* **[Rare renal tumors].**  
  Akca A, Otto T, Hinsch N, Lammers BJ, Starke AAR, **Goretzki PE**. *Der Urologe. Ausg. A, 57(2):193-197*. <a href="https://pubmed.ncbi.nlm.nih.gov/29242949/" style="text-decoration: none;">PubMed</a>

* **Measurement of Basal Serum Calcitonin for the Diagnosis of Medullary Thyroid Cancer.**  
  Allelein S, Ehlers M, Morneau C, Schwartz K, **Goretzki PE**, Seppel T, Feldkamp J, Krieg A, Knoefel WT, Kuebart A, Haase M, Dringenberg T, Schmid C, Schott M. *Hormone and metabolic research = Hormon- und Stoffwechselforschung = Hormones et metabolisme, 50(1):23-28*. <a href="https://pubmed.ncbi.nlm.nih.gov/29169190/" style="text-decoration: none;">PubMed</a>


## 2017

* **Health of VLBW infants in Germany at five years of age: What do parents describe?**  
  Spiegler J, Stichtenoth G, Konig IR, Herting E, Gopel W. *Early human development, 115:88-92*. <a href="https://pubmed.ncbi.nlm.nih.gov/29024833/" style="text-decoration: none;">PubMed</a>

* **Application of an Amplitude-integrated EEG Monitor (Cerebral Function Monitor) to Neonates.**  
  **Bruns N**, Blumenthal S, Meyer I, Klose-Verschuur S, Felderhoff-Muser U, Muller H. *Journal of visualized experiments : JoVE(127)*. <a href="https://pubmed.ncbi.nlm.nih.gov/28930990/" style="text-decoration: none;">PubMed</a>

* **The association of mannose-binding lectin 2 polymorphisms with outcome in very low birth weight infants.**  
  Hartz A, Pagel J, Humberg A, Preuss M, Schreiter L, Rupp J, Figge J, Karsten CM, Nurnberg P, Herting E, Gopel W, Hartel C. *PloS one, 12(5):e0178032*. <a href="https://pubmed.ncbi.nlm.nih.gov/28558032/" style="text-decoration: none;">PubMed</a>

* **Delivery mode and intraventricular hemorrhage risk in very-low-birth-weight infants: Observational data of the German Neonatal Network.**  
  Humberg A, Hartel C, Paul P, Hanke K, Bossung V, Hartz A, Fasel L, Rausch TK, Rody A, Herting E, Gopel W. *European journal of obstetrics, gynecology, and reproductive biology, 212:144-149*. <a href="https://pubmed.ncbi.nlm.nih.gov/28363188/" style="text-decoration: none;">PubMed</a>

* **Comparison of two common aEEG classifications for the prediction of neurodevelopmental outcome in preterm infants.**  
  **Bruns N**, Dransfeld F, Huning B, Hobrecht J, Storbeck T, Weiss C, Felderhoff-Muser U, Muller H. *European journal of pediatrics, 176(2):163-171*. <a href="https://pubmed.ncbi.nlm.nih.gov/27924356/" style="text-decoration: none;">PubMed</a>

* **Glucocorticoid Receptor Gene Variants and Neonatal Outcome in Very-Low-Birth-Weight Preterm Infants.**  
  Schreiner C, Schreiner F, Hartel C, Heckmann M, Heep A, Bartmann P, Woelfle J, Muller A, Herting E, Gopel W. *Neonatology, 111(1):22-29*. <a href="https://pubmed.ncbi.nlm.nih.gov/27509264/" style="text-decoration: none;">PubMed</a>


## 2016

* **Clinical Evaluation of the First Automated Assay for the Detection of Stimulating TSH Receptor Autoantibodies.**  
  Allelein S, Ehlers M, **Goretzki S**, Hermsen D, Feldkamp J, Haase M, Dringenberg T, Schmid C, Hautzel H, Schott M. *Hormone and metabolic research = Hormon- und Stoffwechselforschung = Hormones et metabolisme, 48(12):795-801*. <a href="https://pubmed.ncbi.nlm.nih.gov/27923250/" style="text-decoration: none;">PubMed</a>

* **Evaluation of malignant parathyroid tumours in two European cohorts of patients with sporadic primary hyperparathyroidism.**  
  Ozolins A, Narbuts Z, Vanags A, Simtniece Z, Visnevska Z, Akca A, Wirowski D, Gardovskis J, Strumfa I, **Goretzki PE**. *Langenbeck's archives of surgery, 401(7):943-951*. <a href="https://pubmed.ncbi.nlm.nih.gov/26658808/" style="text-decoration: none;">PubMed</a>

* **Total Parathyroidectomy With Routine Thymectomy and Autotransplantation Versus Total Parathyroidectomy Alone for Secondary Hyperparathyroidism: Results of a Nonconfirmatory Multicenter Prospective Randomized Controlled Pilot Trial.**  
  Schlosser K, Bartsch DK, Diener MK, Seiler CM, Bruckner T, Nies C, Meyer M, Neudecker J, **Goretzki PE**, Glockzin G, Konopke R, Rothmund M. *Annals of surgery, 264(5):745-753*. <a href="https://pubmed.ncbi.nlm.nih.gov/27741007/" style="text-decoration: none;">PubMed</a>

* **Media Stories on NICU Outbreaks Lead to an Increased Prescription Rate of Third-Line Antibiotics in the Community of Neonatal Care.**  
  Hartel C, Hartz A, Bahr L, Gille C, Gortner L, Simon A, Orlikowsky T, Muller A, Korner T, Henneke P, Haase R, Zemlin M, Viemann D, Gebauer C, Thome U, Ziegler A, Rupp J, Herting E, Gopel W. *Infection control and hospital epidemiology, 37(8):924-930*. <a href="https://pubmed.ncbi.nlm.nih.gov/27143176/" style="text-decoration: none;">PubMed</a>

* **NOD2 Loss-of-Function Mutations and Risks of Necrotizing Enterocolitis or Focal Intestinal Perforation in Very Low-birth-weight Infants.**  
  Hartel C, Hartz A, Pagel J, Rupp J, Stein A, Kribs A, Muller A, Haase R, Gille C, Bottger R, Kittel J, Jensen R, Wieg C, Herting E, Gopel W. *Inflammatory bowel diseases, 22(2):249-56*. <a href="https://pubmed.ncbi.nlm.nih.gov/26752461/" style="text-decoration: none;">PubMed</a>

* **Does Breastmilk Influence the Development of Bronchopulmonary Dysplasia?**  
  Spiegler J, Preuss M, Gebauer C, Bendiks M, Herting E, Gopel W. *The Journal of pediatrics, 169:76-80.e4*. <a href="https://pubmed.ncbi.nlm.nih.gov/26621048/" style="text-decoration: none;">PubMed</a>


## 2015

* **Short-term outcome of very-low-birthweight infants with arterial hypotension in the first 24 h of life.**  
  Faust K, Hartel C, Preuss M, Rabe H, Roll C, Emeis M, Wieg C, Szabo M, Herting E, Gopel W. *Archives of disease in childhood. Fetal and neonatal edition, 100(5):F388-92*. <a href="https://pubmed.ncbi.nlm.nih.gov/26199082/" style="text-decoration: none;">PubMed</a>

* **Resection of Ileoinguinal and Ileohypogastric Nerves Combined with Gluing in Modified Lichtenstein Repair.**  
  Baer A, Bohnert N, **Goretzki PE**, Lammers BJ. *Surgical technology international, 26:143-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/26055002/" style="text-decoration: none;">PubMed</a>

* **Electrocortical Activity at 7 Days of Life is Affected in Extremely Premature Infants with Patent Ductus Arteriosus.**  
  **Bruns N**, Metze B, Buhrer C, Felderhoff-Muser U, Huseman D. *Klinische Padiatrie, 227(5):264-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/25875399/" style="text-decoration: none;">PubMed</a>

* **Length and weight of very low birth weight infants in Germany at 2 years of age: does it matter at what age they start complementary food?**  
  Spiegler J, Eisemann N, Ehlers S, Orlikowsky T, Kannt O, Herting E, Gopel W. *European journal of clinical nutrition, 69(6):662-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/25872910/" style="text-decoration: none;">PubMed</a>

* **Preterm prelabor rupture of membranes and outcome of very-low-birth-weight infants in the German Neonatal Network.**  
  Hanke K, Hartz A, Manz M, Bendiks M, Heitmann F, Orlikowsky T, Muller A, Olbertz D, Kuhn T, Siegel J, von der Wense A, Wieg C, Kribs A, Stein A, Pagel J, Herting E, Gopel W, Hartel C. *PloS one, 10(4):e0122564*. <a href="https://pubmed.ncbi.nlm.nih.gov/25856083/" style="text-decoration: none;">PubMed</a>

* **Less invasive surfactant administration is associated with improved pulmonary outcomes in spontaneously breathing preterm infants.**  
  Gopel W, Kribs A, Hartel C, Avenarius S, Teig N, Groneck P, Olbertz D, Roll C, Vochem M, Weller U, von der Wense A, Wieg C, Wintgens J, Preuss M, Ziegler A, Roth B, Herting E. *Acta paediatrica (Oslo, Norway : 1992), 104(3):241-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/25474712/" style="text-decoration: none;">PubMed</a>

* **Local recurrence in the neck and survival after thyroidectomy for metastatic renal cell carcinoma.**  
  Iesalnieks I, Machens A, Bures C, Krenz D, Winter H, Vorlander C, Bareck E, Alesina PF, Musholt T, Steinmuller T, Anthuber M, **Goretzki P**, Trupka A, Mayr M, Weber T, Schlitt HJ, Dralle H, Hermann M, Agha A. *Annals of surgical oncology, 22(6):1798-805*. <a href="https://pubmed.ncbi.nlm.nih.gov/25472649/" style="text-decoration: none;">PubMed</a>


## 2014

* **Impairment of bone health in pediatric patients with hemolytic anemia.**  
  Schundeln MM, **Goretzki SC**, Hauffa PK, Wieland R, Bauer J, Baeder L, Eggert A, Hauffa BP, Grasemann C. *PloS one, 9(10):e108400*. <a href="https://pubmed.ncbi.nlm.nih.gov/25299063/" style="text-decoration: none;">PubMed</a>

* **Mitochondrial mutation m.1555A>G as a risk factor for failed newborn hearing screening in a large cohort of preterm infants.**  
  Gopel W, Berkowski S, Preuss M, Ziegler A, Kuster H, Felderhoff-Muser U, Gortner L, Mogel M, Hartel C, Herting E. *BMC pediatrics, 14:210*. <a href="https://pubmed.ncbi.nlm.nih.gov/25155176/" style="text-decoration: none;">PubMed</a>

* **Encapsulated cells expressing a chemotherapeutic activating enzyme allow the targeting of subtoxic chemotherapy and are safe and efficacious: data from two clinical trials in pancreatic cancer.**  
  Lohr JM, Haas SL, Kroger JC, Friess HM, Hoft R, **Goretzki PE**, Peschel C, Schweigert M, Salmons B, Gunzburg WH. *Pharmaceutics, 6(3):447-66*. <a href="https://pubmed.ncbi.nlm.nih.gov/25116885/" style="text-decoration: none;">PubMed</a>

* **Prophylactic use of Lactobacillus acidophilus/Bifidobacterium infantis probiotics and outcome in very low birth weight infants.**  
  Hartel C, Pagel J, Rupp J, Bendiks M, Guthmann F, Rieger-Fackeldey E, Heckmann M, Franz A, Schiffmann JH, Zimmermann B, Hepping N, von der Wense A, Wieg C, Herting E, Gopel W. *The Journal of pediatrics, 165(2):285-289.e1*. <a href="https://pubmed.ncbi.nlm.nih.gov/24880888/" style="text-decoration: none;">PubMed</a>

* **Postoperative vocal cord dysfunction despite normal intraoperative neuromonitoring: an unexpected complication with the risk of bilateral palsy.**  
  Melin M, Schwarz K, Pearson MD, Lammers BJ, **Goretzki PE**. *World journal of surgery, 38(10):2597-602*. <a href="https://pubmed.ncbi.nlm.nih.gov/24867468/" style="text-decoration: none;">PubMed</a>

* **[Neuroendocrine neoplasms of the distal jejunum and ileum].**  
  Anlauf M, Sipos B, Boeck I, Baldus SE, Heikaus S, Krausch M, Knoefel WT, Begum N, **Goretzki P**, Schott M, Auernhammer CJ, Cremer B, Rinke A, Ezziddin S, Fottner C, Popperl G, Lahner H, Horsch D, Gabbert HE, Komminoth P, Perren A, Kloppel G, Wiedenmann B, Pavel M, Pape U. *Der Pathologe, 35(3):283-93; quiz 294*. <a href="https://pubmed.ncbi.nlm.nih.gov/24671468/" style="text-decoration: none;">PubMed</a>

* **Prevalence of xerostomia and hyposalivation and their association with quality of life in elderly patients in dependence on dental status and prosthetic rehabilitation: a pilot study.**  
  Hahnel S, **Schwarz S**, Zeman F, Schafer L, Behr M. *Journal of dentistry, 42(6):664-70*. <a href="https://pubmed.ncbi.nlm.nih.gov/24632475/" style="text-decoration: none;">PubMed</a>

* **Functional long-term results after rectal cancer surgery--technique of the athermal mesorectal excision.**  
  Touloumtzidis A, Sostmann B, Hilgers N, Renter MA, Kuhn P, **Goretzki PE**, Lammers BJ. *International journal of colorectal disease, 29(3):285-92*. <a href="https://pubmed.ncbi.nlm.nih.gov/24306821/" style="text-decoration: none;">PubMed</a>

* **Multifocal papillary thyroid carcinoma--a consensus report of the European Society of Endocrine Surgeons (ESES).**  
  Iacobone M, Jansson S, Barczynski M, **Goretzki P**. *Langenbeck's archives of surgery, 399(2):141-54*. <a href="https://pubmed.ncbi.nlm.nih.gov/24263684/" style="text-decoration: none;">PubMed</a>

* **Photophysics and electrochemistry of a platinum-acetylide disubstituted perylenediimide.**  
  Llewellyn BA, Slater AG, **Goretzki G**, Easun TL, Sun XZ, Davies ES, Argent SP, Lewis W, Beeby A, George MW, Champness NR. *Dalton transactions (Cambridge, England : 2003), 43(1):85-94*. <a href="https://pubmed.ncbi.nlm.nih.gov/24077462/" style="text-decoration: none;">PubMed</a>

* **Hedgehog-signaling is upregulated in non-producing human adrenal adenomas and antagonism of hedgehog-signaling inhibits proliferation of NCI-H295R cells and an immortalized primary human adrenal cell line.**  
  Werminghaus P, Haase M, Hornsby PJ, Schinner S, Schott M, Malendowicz LK, Lammers BJ, **Goretzki PE**, Muller-Mattheis V, Markus Giessing, Willenberg HS. *The Journal of steroid biochemistry and molecular biology, 139:7-15*. <a href="https://pubmed.ncbi.nlm.nih.gov/24063979/" style="text-decoration: none;">PubMed</a>

* **Risk for late-onset blood-culture proven sepsis in very-low-birth weight infants born small for gestational age: a large multicenter study from the German Neonatal Network.**  
  Troger B, Gopel W, Faust K, Muller T, Jorch G, Felderhoff-Muser U, Gortner L, Heitmann F, Hoehn T, Kribs A, Laux R, Roll C, Emeis M, Mogel M, Siegel J, Vochem M, von der Wense A, Wieg C, Herting E, Hartel C. *The Pediatric infectious disease journal, 33(3):238-43*. <a href="https://pubmed.ncbi.nlm.nih.gov/24030351/" style="text-decoration: none;">PubMed</a>

* **[Neuroendocrine tumours of the GI tract--data from the German NET Registry].**  
  Begum N, Maasberg S, Plockinger U, Anlauf M, Rinke A, Popperl G, Lehnert H, Izbicki JR, Krausch M, Vashist YK, Raffel A, Burk CG, Hoffmann J, **Goretzki P**, Pape UF. *Zentralblatt fur Chirurgie, 139(3):276-83*. <a href="https://pubmed.ncbi.nlm.nih.gov/23042103/" style="text-decoration: none;">PubMed</a>


## 2013

* **[Intraoperative neuromonitoring in thyroid surgery. Recommendations of the Surgical Working Group for Endocrinology].**  
  Dralle H, Lorenz K, Schabram P, Musholt TJ, Dotzenrath C, **Goretzki PE**, Kussmann J, Niederle B, Nies C, Schabram J, Scheuba C, Simon D, Steinmuller T, Trupka A. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 84(12):1049-56*. <a href="https://pubmed.ncbi.nlm.nih.gov/24337220/" style="text-decoration: none;">PubMed</a>

* **Is the covering of the resection margin after distal pancreatectomy advantageous?**  
  Akca A, **Goretzki PE**, Wirowski D, Renter MA, Bolke E, Matuschek C, Gerber PA, Lammers BJ. *European journal of medical research, 18(1):33*. <a href="https://pubmed.ncbi.nlm.nih.gov/24073931/" style="text-decoration: none;">PubMed</a>

* **Use of analgesic and sedative drugs in VLBW infants in German NICUs from 2003-2010.**  
  Mehler K, Oberthuer A, Haertel C, Herting E, Roth B, Goepel W. *European journal of pediatrics, 172(12):1633-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/23877637/" style="text-decoration: none;">PubMed</a>

* **Polymorphisms in FTO and MAF Genes and Birth Weight, BMI, Ponderal Index, Weight Gain in a Large Cohort of Infants with a Birth Weight below 1500 Grams.**  
  Haller S, Spiegler J, Hemmelmann C, Kuster H, Vochem M, Moller J, Muller D, Kribs A, Hoehn T, Hartel C, Herting E, Gopel W. *PloS one, 8(6):e66331*. <a href="https://pubmed.ncbi.nlm.nih.gov/23840443/" style="text-decoration: none;">PubMed</a>

* **Failed surgery in primary hyperparathyroidism - what has changed with time.**  
  Wirowski D, **Goretzki PE**, Schwarz K, Lammers BJ, Dotzenrath C, Roher HD. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 121(6):323-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/23512418/" style="text-decoration: none;">PubMed</a>

* **German Association of Endocrine Surgeons practice guideline for the surgical management of malignant thyroid tumors.**  
  Dralle H, Musholt TJ, Schabram J, Steinmuller T, Frilling A, Simon D, **Goretzki PE**, Niederle B, Scheuba C, Clerici T, Hermann M, Kussmann J, Lorenz K, Nies C, Schabram P, Trupka A, Zielke A, Karges W, Luster M, Schmid KW, Vordermark D, Schmoll HJ, Muhlenberg R, Schober O, Rimmele H, Machens A. *Langenbeck's archives of surgery, 398(3):347-75*. <a href="https://pubmed.ncbi.nlm.nih.gov/23456424/" style="text-decoration: none;">PubMed</a>

* **Pregnancy risk factors for very premature delivery: what role do hypertension, obesity and diabetes play?**  
  Spiegler J, Stichtenoth G, Weichert J, Konig IR, Schlaud M, V D Wense A, Olbertz D, Gurth H, Schiffmann JH, Bohnhorst B, Gortner L, Herting E, Gopel W. *Archives of gynecology and obstetrics, 288(1):57-64*. <a href="https://pubmed.ncbi.nlm.nih.gov/23400353/" style="text-decoration: none;">PubMed</a>

* **[Surgical therapy for thyroid gland malignancies].**  
  **Goretzki PE**, Schwarz K, Lammers B. *HNO, 61(1):71-81; quiz 82*. <a href="https://pubmed.ncbi.nlm.nih.gov/23325061/" style="text-decoration: none;">PubMed</a>

* **Very low birth weight infants after discharge: What do parents describe?**  
  Spiegler J, Schlaud M, Konig IR, Teig N, Hubert M, Herting E, Gopel W. *Early human development, 89(5):343-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/23266151/" style="text-decoration: none;">PubMed</a>

* **IONM-guided goiter surgery leading to two-stage thyroidectomy--indication and results.**  
  Melin M, Schwarz K, Lammers BJ, **Goretzki PE**. *Langenbeck's archives of surgery, 398(3):411-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/23179319/" style="text-decoration: none;">PubMed</a>

* **Cinacalcet effects on the perioperative course of patients with secondary hyperparathyroidism.**  
  Wirowski D, **Goretzki PE**, Schwarz K, Lammers BJ. *Langenbeck's archives of surgery, 398(1):131-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/23007384/" style="text-decoration: none;">PubMed</a>

* **Partial pancreaticoduodenectomy can provide cure for duodenal gastrinoma associated with multiple endocrine neoplasia type 1.**  
  Lopez CL, Falconi M, Waldmann J, Boninsegna L, Fendrich V, **Goretzki PK**, Langer P, Kann PH, Partelli S, Bartsch DK. *Annals of surgery, 257(2):308-14*. <a href="https://pubmed.ncbi.nlm.nih.gov/22580937/" style="text-decoration: none;">PubMed</a>


## 2012

* **Causes of delivery and outcomes of very preterm twins stratified to zygosity.**  
  Spiegler J, Hartel C, Schulz L, von Wurmb-Schwark N, Hoehn T, Kribs A, Kuster H, Siegel J, Wieg C, Weichert J, Herting E, Gopel W. *Twin research and human genetics : the official journal of the International Society for Twin Studies, 15(4):532-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/22853920/" style="text-decoration: none;">PubMed</a>

* **Epidemic microclusters of blood-culture proven sepsis in very-low-birth weight infants: experience of the German Neonatal Network.**  
  Hartel C, Faust K, Avenarius S, Bohnhorst B, Emeis M, Gebauer C, Groneck P, Heitmann F, Hoehn T, Hubert M, Kribs A, Kuster H, Laux R, Mogel M, Muller D, Olbertz D, Roll C, Siegel J, Stein A, Vochem M, Weller U, von der Wense A, Wieg C, Wintgens J, Hemmelmann C, Simon A, Herting E, Gopel W. *PloS one, 7(6):e38304*. <a href="https://pubmed.ncbi.nlm.nih.gov/22768043/" style="text-decoration: none;">PubMed</a>

* **Conservation and divergence between cytoplasmic and muscle-specific actin capping proteins: insights from the crystal structure of cytoplasmic Cap32/34 from Dictyostelium discoideum.**  
  Eckert C, **Goretzki A**, Faberova M, Kollmar M. *BMC structural biology, 12:12*. <a href="https://pubmed.ncbi.nlm.nih.gov/22657106/" style="text-decoration: none;">PubMed</a>

* **Correspondence (letter to the editor): Incomplete picture.**  
  Starke AA, **Goretzki PE**, Saddig C. *Deutsches Arzteblatt international, 109(4):66; author reply 66-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/22334829/" style="text-decoration: none;">PubMed</a>

* **[Surgery for neuroendocrine tumors of the gastroenteropancreatic system (GEP-NET)].**  
  **Goretzki PE**, Starke A, Akca A, Lammers BJ. *Der Internist, 53(2):152-60*. <a href="https://pubmed.ncbi.nlm.nih.gov/22290318/" style="text-decoration: none;">PubMed</a>

* **ATP-binding cassette member A3 (E292V) gene mutation and pulmonary morbidity in very-low-birth-weight infants.**  
  Hartel C, Felderhoff-Muser U, Gebauer C, Hoehn T, Kribs A, Laux R, Moller J, Segerer H, Teig N, von der Wense A, Wieg C, Stichtenoth G, Herting E, Gopel W. *Acta paediatrica (Oslo, Norway : 1992), 101(4):380-3*. <a href="https://pubmed.ncbi.nlm.nih.gov/22145626/" style="text-decoration: none;">PubMed</a>

* **[Continuous intraoperative neuromonitoring of the recurrent laryngeal nerve in thyroid surgery (CIONM) - Where are we now? An update to the European Symposium of Continuous Neuromonitoring in Thyroid Surgery].**  
  Schneider R, Lamade W, Hermann M, **Goretzki P**, Timmermann W, Hauss J, Leinung S. *Zentralblatt fur Chirurgie, 137(1):88-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/21365536/" style="text-decoration: none;">PubMed</a>


## 2011

* **Evaluation of Biocompatibility of Alloplastic Materials: Development of a Tissue Culture In Vitro Test System.**  
  Gerullis H, Georgas E, Eimer C, **Goretzki PE**, Lammers BJ, Klosterhalfen B, Boros M, Wishahi M, Heusch G, Otto T. *Surgical technology international, 21:21-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/22504966/" style="text-decoration: none;">PubMed</a>

* **Combination effects of digalloylresveratrol with arabinofuranosylcytosine and difluorodeoxycytidine in human leukemia and pancreatic cancer cells.**  
  Saiko P, Graser G, Madlener S, **Schwarz S**, Krupitza G, Jaeger W, Somepalli V, Golakoti T, Fritzer-Szekeres M, Szekeres T. *Nucleosides, nucleotides & nucleic acids, 30(12):1190-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/22132974/" style="text-decoration: none;">PubMed</a>

* **Avoidance of mechanical ventilation by surfactant treatment of spontaneously breathing preterm infants (AMV): an open-label, randomised, controlled trial.**  
  Gopel W, Kribs A, Ziegler A, Laux R, Hoehn T, Wieg C, Siegel J, Avenarius S, von der Wense A, Vochem M, Groneck P, Weller U, Moller J, Hartel C, Haller S, Roth B, Herting E. *Lancet (London, England), 378(9803):1627-34*. <a href="https://pubmed.ncbi.nlm.nih.gov/21963186/" style="text-decoration: none;">PubMed</a>

* **[Diagnostics and treatment in functional pancreatic neuroendocrine tumours].**  
  Mussig K, Dudziak K, Horger M, Anlauf M, **Goretzki PE**. *Deutsche medizinische Wochenschrift (1946), 136(24):1319-30*. <a href="https://pubmed.ncbi.nlm.nih.gov/21656454/" style="text-decoration: none;">PubMed</a>

* **German Association of Endocrine Surgeons practice guidelines for the surgical treatment of benign thyroid disease.**  
  Musholt TJ, Clerici T, Dralle H, Frilling A, **Goretzki PE**, Hermann MM, Kussmann J, Lorenz K, Nies C, Schabram J, Schabram P, Scheuba C, Simon D, Steinmuller T, Trupka AW, Wahl RA, Zielke A, Bockisch A, Karges W, Luster M, Schmid KW. *Langenbeck's archives of surgery, 396(5):639-49*. <a href="https://pubmed.ncbi.nlm.nih.gov/21424798/" style="text-decoration: none;">PubMed</a>

* **Hormone replacement after thyroid and parathyroid surgery: Importance of postoperative hypocalcemia.**  
  Wirowski D, Schwarz K, Lammers BJ, **Goretzki PE**. *Deutsches Arzteblatt international, 108(8):133; author reply 134*. <a href="https://pubmed.ncbi.nlm.nih.gov/21403810/" style="text-decoration: none;">PubMed</a>

* **Extra-adrenal pheochromocytoma in the organ of Zuckerkandl: diagnosis and treatment strategies.**  
  Kahraman D, **Goretzki PE**, Szangolies M, Schade H, Schmidt M, Kobe C. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 119(7):436-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/21374546/" style="text-decoration: none;">PubMed</a>

* **Tumor necrosis factor-alpha promoter -308 G/A polymorphism and susceptibility to sepsis in very-low-birth-weight infants.**  
  Hartel C, Hemmelmann C, Faust K, Gebauer C, Hoehn T, Kribs A, Laux R, Nikischin W, Segerer H, Teig N, von der Wense A, Wieg C, Herting E, Gopel W. *Critical care medicine, 39(5):1190-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/21317641/" style="text-decoration: none;">PubMed</a>


## 2010

* **Water-jet dissection in rectal cancer surgery: surgical and oncological outcomes.**  
  Touloumtzidis A, Kuhn P, **Goretzki PE**, Lammers BJ. *Surgical technology international, 20:115-23*. <a href="https://pubmed.ncbi.nlm.nih.gov/21082555/" style="text-decoration: none;">PubMed</a>

* **Substantial deficiencies.**  
  **Goretzki P**, Lammers B, Touloumtzidis A, Otto T. *Deutsches Arzteblatt international, 107(33):573; author reply 574*. <a href="https://pubmed.ncbi.nlm.nih.gov/20830282/" style="text-decoration: none;">PubMed</a>

* **[Insulinoma associated with pregnancy].**  
  Akca A, Mann K, Starke A, Lammers BJ, **Goretzki PE**. *Deutsche medizinische Wochenschrift (1946), 135(30):1484-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/20648406/" style="text-decoration: none;">PubMed</a>

* **[Pancreatic hyperinsulinism--changes of the clinical picture and importance of differences in sporadic disease course (experience with 144 patients operated in the period 1986-2009)].**  
  **Goretzki P**, Starke A, Lammers B, Schwarz K, Roher HD. *Zentralblatt fur Chirurgie, 135(3):218-25*. <a href="https://pubmed.ncbi.nlm.nih.gov/20560122/" style="text-decoration: none;">PubMed</a>

* **Diagnosis and treatment of small follicular thyroid carcinomas.**  
  Clerici T, Kolb W, Beutner U, Bareck E, Dotzenrath C, Kull C, Niederle B. *The British journal of surgery, 97(6):839-44*. <a href="https://pubmed.ncbi.nlm.nih.gov/20473996/" style="text-decoration: none;">PubMed</a>

* **The impact of intraoperative neuromonitoring (IONM) on surgical strategy in bilateral thyroid diseases: is it worth the effort?**  
  **Goretzki PE**, Schwarz K, Brinkmann J, Wirowski D, Lammers BJ. *World journal of surgery, 34(6):1274-84*. <a href="https://pubmed.ncbi.nlm.nih.gov/20143072/" style="text-decoration: none;">PubMed</a>

* **Supramolecular assemblies formed on an epitaxial graphene superstructure.**  
  Pollard AJ, Perkins EW, Smith NA, Saywell A, **Goretzki G**, Phillips AG, Argent SP, Sachdev H, Muller F, Hufner S, Gsell S, Fischer M, Schreck M, Osterwalder J, Greber T, Berner S, Champness NR, Beton PH. *Angewandte Chemie (International ed. in English), 49(10):1794-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/20135656/" style="text-decoration: none;">PubMed</a>

* **Solubilized derivatives of perylenetetracarboxylic dianhydride (PTCDA) adsorbed on highly oriented pyrolytic graphite.**  
  Russell JC, Blunt MO, **Goretzki G**, Phillips AG, Champness NR, Beton PH. *Langmuir : the ACS journal of surfaces and colloids, 26(6):3972-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/20017490/" style="text-decoration: none;">PubMed</a>


## 2009

* **Clinical predictors and algorithm for the genetic diagnosis of pheochromocytoma patients.**  
  Erlic Z, Rybicki L, Peczkowska M, Golcher H, Kann PH, Brauckhoff M, Mussig K, Muresan M, Schaffler A, Reisch N, Schott M, Fassnacht M, Opocher G, Klose S, Fottner C, Forrer F, Plockinger U, Petersenn S, Zabolotny D, Kollukch O, Yaremchuk S, Januszewicz A, Walz MK, Eng C, Neumann HP. *Clinical cancer research : an official journal of the American Association for Cancer Research, 15(20):6378-85*. <a href="https://pubmed.ncbi.nlm.nih.gov/19825962/" style="text-decoration: none;">PubMed</a>

* **Building multistate redox-active architectures using metal-complex functionalized perylene bis-imides.**  
  **Goretzki G**, Davies ES, Argent SP, Warren JE, Blake AJ, Champness NR. *Inorganic chemistry, 48(21):10264-74*. <a href="https://pubmed.ncbi.nlm.nih.gov/19799381/" style="text-decoration: none;">PubMed</a>

* **Less pain intensity after lichtenstein-repair by using BioGlue for mesh fixation.**  
  Bar A, Sauer T, Bohnert N, **Goretzki PE**, Lammers BJ. *Surgical technology international, 18:125-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/19579199/" style="text-decoration: none;">PubMed</a>

* **Implementing the general use of dissection devices in thyroid surgery from prospective randomized trial to daily use.**  
  **Goretzki PE**, Schwarz K, Lammers BJ. *Surgical technology international, 18:86-92*. <a href="https://pubmed.ncbi.nlm.nih.gov/19579193/" style="text-decoration: none;">PubMed</a>

* **Does multiple gland disease in primary hyperparathyroidism correlate with age or sex?**  
  Wirowski D, Lammers BJ, Pohl P, Schwarz K, **Goretzki PE**. *Langenbeck's archives of surgery, 394(5):885-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/19533167/" style="text-decoration: none;">PubMed</a>

* **Nestin as a marker in the classification of adrenocortical tumors.**  
  Lachenmayer A, Lichtenauer UD, Cox T, Schott M, Malendowicz LK, **Goretzki PE**, Cupisti K, Scherbaum WA, Bornstein SR, Willenberg HS. *Hormone and metabolic research = Hormon- und Stoffwechselforschung = Hormones et metabolisme, 41(5):397-401*. <a href="https://pubmed.ncbi.nlm.nih.gov/19294612/" style="text-decoration: none;">PubMed</a>

* **Self-assembly of a pyridine-terminated thiol monolayer on Au(111).**  
  Silien C, Buck M, **Goretzki G**, Lahaye D, Champness NR, Weidner T, Zharnikov M. *Langmuir : the ACS journal of surfaces and colloids, 25(2):959-67*. <a href="https://pubmed.ncbi.nlm.nih.gov/19138159/" style="text-decoration: none;">PubMed</a>

* **[Indication and performance of endocrine surgery. The significance of molecular genetic examination].**  
  **Goretzki PE**, Wirowski D, Schwarz K, Pohl P, Bohner H, Starke A, Lammers BJ. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 80(2):122-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/19096808/" style="text-decoration: none;">PubMed</a>


## 2008

* **Bis-morpholine-substituted perylene bisimides: impact of isomeric arrangement on electrochemical and spectroelectrochemical properties.**  
  **Goretzki G**, Davies ES, Argent SP, Alsindi WZ, Blake AJ, Warren JE, McMaster J, Champness NR. *The Journal of organic chemistry, 73(22):8808-14*. <a href="https://pubmed.ncbi.nlm.nih.gov/18855454/" style="text-decoration: none;">PubMed</a>

* **Functionalized supramolecular nanoporous arrays for surface templating.**  
  Perdigao LM, Saywell A, Fontes GN, Staniec PA, **Goretzki G**, Phillips AG, Champness NR, Beton PH. *Chemistry (Weinheim an der Bergstrasse, Germany), 14(25):7600-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/18666294/" style="text-decoration: none;">PubMed</a>

* **A left nonrecurrent inferior laryngeal nerve in a patient with right-sided aorta, truncus arteriosus communis, and an aberrant left innominate artery.**  
  Fellmer PT, Bohner H, Wolf A, Roher HD, **Goretzki PE**. *Thyroid : official journal of the American Thyroid Association, 18(6):647-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/18578615/" style="text-decoration: none;">PubMed</a>

* **Thyroid metastases of renal cell carcinoma: clinical course in 45 patients undergoing surgery. Assessment of factors affecting patients' survival.**  
  Iesalnieks I, Winter H, Bareck E, Sotiropoulos GC, **Goretzki PE**, Klinkhammer-Schalke M, Brockner S, Trupka A, Anthuber M, Rupprecht H, Raab M, Meyer W, Reichmann F, Kastel M, Mayr M, Braun W, Schlitt HJ, Agha A. *Thyroid : official journal of the American Thyroid Association, 18(6):615-24*. <a href="https://pubmed.ncbi.nlm.nih.gov/18578610/" style="text-decoration: none;">PubMed</a>

* **Presentation of 6 cases with parathyroid cysts and discussion of the literature.**  
  Wirowski D, Wicke C, Bohner H, Lammers BJ, Pohl P, Schwarz K, **Goretzki PE**. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 116(8):501-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/18523916/" style="text-decoration: none;">PubMed</a>

* **[Routine measurement of serum calcitonin in patients with nodular thyroid disorders?].**  
  Dietlein M, Wieler H, Schmidt M, Schwab R, **Goretzki PE**, Schicha H. *Nuklearmedizin. Nuclear medicine, 47(2):65-72*. <a href="https://pubmed.ncbi.nlm.nih.gov/18392315/" style="text-decoration: none;">PubMed</a>

* **Is it meaningful to preserve a palsied recurrent laryngeal nerve?**  
  Chi SY, Lammers B, Boehner H, Pohl P, **Goretzki PE**. *Thyroid : official journal of the American Thyroid Association, 18(3):363-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/18303959/" style="text-decoration: none;">PubMed</a>

* **Differentiation of insulin secretion patterns in insulinoma.**  
  Saddig C, **Goretzki PE**, Starke AA. *World journal of surgery, 32(5):918-29*. <a href="https://pubmed.ncbi.nlm.nih.gov/18259806/" style="text-decoration: none;">PubMed</a>

* **Consensus guidelines for the management of patients with digestive neuroendocrine tumours: well-differentiated colon and rectum tumour/carcinoma.**  
  Ramage JK, **Goretzki PE**, Manfredi R, Komminoth P, Ferone D, Hyrdel R, Kaltsas G, Kelestimur F, Kvols L, Scoazec JY, Garcia MI, Caplin ME. *Neuroendocrinology, 87(1):31-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/18097130/" style="text-decoration: none;">PubMed</a>

* **[Abdominal preoperation. No contraindication for laparoscopic transabdominal adrenalectomy].**  
  Pohl PP, Meyer A, Lammers BJ, **Goretzki PE**. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 79(6):571-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/17879074/" style="text-decoration: none;">PubMed</a>


## 2007

* **Developing effective screening strategies in multiple endocrine neoplasia type 1 (MEN 1) on the basis of clinical and sequencing data of German patients with MEN 1.**  
  Schaaf L, Pickel J, Zinner K, Hering U, Hofler M, **Goretzki PE**, Spelsberg F, Raue F, von zur Muhlen A, Gerl H, Hensen J, Bartsch DK, Rothmund M, Schneyer U, Dralle H, Engelbach M, Karges W, Stalla GK, Hoppner W. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 115(8):509-17*. <a href="https://pubmed.ncbi.nlm.nih.gov/17853334/" style="text-decoration: none;">PubMed</a>

* **Age-related penetrance of endocrine tumours in multiple endocrine neoplasia type 1 (MEN1): a multicentre study of 258 gene carriers.**  
  Machens A, Schaaf L, Karges W, Frank-Raue K, Bartsch DK, Rothmund M, Schneyer U, **Goretzki P**, Raue F, Dralle H. *Clinical endocrinology, 67(4):613-22*. <a href="https://pubmed.ncbi.nlm.nih.gov/17590169/" style="text-decoration: none;">PubMed</a>


## 2006

* **Well-differentiated pancreatic nonfunctioning tumors/carcinoma.**  
  Falconi M, Plockinger U, Kwekkeboom DJ, Manfredi R, Korner M, Kvols L, Pape UF, Ricke J, **Goretzki PE**, Wildi S, Steinmuller T, Oberg K, Scoazec JY. *Neuroendocrinology, 84(3):196-211*. <a href="https://pubmed.ncbi.nlm.nih.gov/17312380/" style="text-decoration: none;">PubMed</a>

* **A case of catecholamine and glucocorticoid excess syndrome due to a corticotropin-secreting paraganglioma.**  
  Willenberg HS, Feldkamp J, Lehmann R, Schott M, **Goretzki PE**, Scherbaum WA. *Annals of the New York Academy of Sciences, 1073:52-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/17102072/" style="text-decoration: none;">PubMed</a>

* **Coincidence of insulinoma with nesidioblastosis or false diagnosis in 2 of 15 patients?**  
  **Goretzki PE**, Starkel A. *The American journal of surgical pathology, 30(7):919-20; author reply 920-1*. <a href="https://pubmed.ncbi.nlm.nih.gov/16819338/" style="text-decoration: none;">PubMed</a>

* **Dianhydride-amine hydrogen bonded perylene tetracarboxylic dianhydride and tetraaminobenzene rows.**  
  Ma J, Rogers BL, Humphry MJ, Ring DJ, **Goretzki G**, Champness NR, Beton PH. *The journal of physical chemistry. B, 110(25):12207-10*. <a href="https://pubmed.ncbi.nlm.nih.gov/16800538/" style="text-decoration: none;">PubMed</a>

* **Islet hyperplasia in adults: challenge to preoperatively diagnose non-insulinoma pancreatogenic hypoglycemia syndrome.**  
  Starke A, Saddig C, Kirch B, Tschahargane C, **Goretzki P**. *World journal of surgery, 30(5):670-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/16555022/" style="text-decoration: none;">PubMed</a>


## 2005

* **Letter Re: Id1 gene expression in hyperplastic and neoplastic thyroid tissues.**  
  Rao AS, **Goretzki PE**, Kohrle J, Brabant G. *The Journal of clinical endocrinology and metabolism, 90(10):5906*. <a href="https://pubmed.ncbi.nlm.nih.gov/16207897/" style="text-decoration: none;">PubMed</a>

* **Do Germans keep patients too long in hospital? A prospective randomized trial.**  
  Dotzenrath CM, Cupisti K, Raffel A, Aust B, Yang Q, Kruger B, Ohmann C, Roher HD, **Goretzki PE**. *World journal of surgery, 29(9):1189-93*. <a href="https://pubmed.ncbi.nlm.nih.gov/16088423/" style="text-decoration: none;">PubMed</a>

* **Radiosynoviorthesis in osteoarthritis and other disorders with concomitant synovitis in comparison to rheumatoid arthritis.**  
  Rau H, Franke C, **Goretzki G**, Spitz J. *Cancer biotherapy & radiopharmaceuticals, 20(3):349-55*. <a href="https://pubmed.ncbi.nlm.nih.gov/15989483/" style="text-decoration: none;">PubMed</a>

* **[New aspects in hernia surgery].**  
  Lammers BJ, **Goretzki PE**, Otto T. *Der Urologe. Ausg. A, 44(7):774-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/15937683/" style="text-decoration: none;">PubMed</a>

* **Malignant metastatic insulinoma-postoperative treatment and follow-up.**  
  Starke A, Saddig C, Mansfeld L, Koester R, Tschahargane C, Czygan P, **Goretzki P**. *World journal of surgery, 29(6):789-93*. <a href="https://pubmed.ncbi.nlm.nih.gov/15880279/" style="text-decoration: none;">PubMed</a>

* **Monitoring of the hypoglossal nerve during general anesthesia.**  
  Bohner H, Terorde N, **Goretzki PE**. *Journal of vascular surgery, 41(4):734*. <a href="https://pubmed.ncbi.nlm.nih.gov/15874945/" style="text-decoration: none;">PubMed</a>

* **Coincidence of multiple endocrine neoplasia types 1 and 2: mutations in the RET protooncogene and MEN1 tumor suppressor gene in a family presenting with recurrent primary hyperparathyroidism.**  
  Frank-Raue K, Rondot S, Hoeppner W, **Goretzki P**, Raue F, Meng W. *The Journal of clinical endocrinology and metabolism, 90(7):4063-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/15870131/" style="text-decoration: none;">PubMed</a>

* **Photochemical hydrosilylation of 11-undecenyltriethylammonium bromide with hydrogen-terminated Si surfaces for the development of robust microcantilever sensors for Cr(VI).**  
  Boiadjiev VI, Brown GM, Pinnaduwage LA, **Goretzki G**, Bonnesen PV, Thundat T. *Langmuir : the ACS journal of surfaces and colloids, 21(4):1139-42*. <a href="https://pubmed.ncbi.nlm.nih.gov/15697249/" style="text-decoration: none;">PubMed</a>


## 2004

* **Convenient and versatile subcellular extraction procedure, that facilitates classical protein expression profiling and functional protein analysis.**  
  Abdolzade-Bavil A, Hayes S, **Goretzki L**, Kroger M, Anders J, Hendriks R. *Proteomics, 4(5):1397-405*. <a href="https://pubmed.ncbi.nlm.nih.gov/15188407/" style="text-decoration: none;">PubMed</a>

* **[Adenocarcinoma of the esophagogastric junction: prognostic factors and results of primary surgery].**  
  Bosing NM, Heise JW, **Goretzki PE**, Sarbia M, Roher HD. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 75(11):1088-97*. <a href="https://pubmed.ncbi.nlm.nih.gov/15168031/" style="text-decoration: none;">PubMed</a>

* **Therapeutic strategy of papillary cystic and solid neoplasm (PCSN): a rare non-endocrine tumor of the pancreas in children.**  
  Raffel A, Cupisti K, Krausch M, Braunstein S, Trobs B, **Goretzki PE**, Willnow U. *Surgical oncology, 13(1):1-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/15145028/" style="text-decoration: none;">PubMed</a>

* **[Economic restraints shorten the length of hospital stay: thyroid operation as a model case].**  
  Raffel A, Cupisti K, Dotzenrath B, Kruger B, Ohmann C, Schulte KM, **Goretzki PE**, Roher HD. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 75(7):702-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/15138657/" style="text-decoration: none;">PubMed</a>

* **Pheochromocytoma during pregnancy: laparoscopic and conventional surgical treatment of two cases.**  
  Wolf A, **Goretzki PE**, Rohrborn A, Feldkamp J, Simon D, Scherbaum WA, Roher HD. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 112(2):98-101*. <a href="https://pubmed.ncbi.nlm.nih.gov/15031775/" style="text-decoration: none;">PubMed</a>

* **[Multicenter study of radiosynoviorthesis. Clinical outcome in osteoarthritis and other disorders with concomitant synovitis in comparison with rheumatoid arthritis].**  
  Rau H, Lohmann K, Franke C, **Goretzki G**, Lemb MA, Muller J, Panholzer PJ, Stelling E, Spitz J. *Nuklearmedizin. Nuclear medicine, 43(2):57-62*. <a href="https://pubmed.ncbi.nlm.nih.gov/15029266/" style="text-decoration: none;">PubMed</a>


## 2003

* **Spindle epithelial tumour with thymus-like differentiation (SETTLE) of the thyroid gland with widespread metastases in a 13-year-old girl.**  
  Raffel A, Cupisti K, Rees M, Janig U, Bernbeck B, Jazbec J, **Goretzki PE**, Gobel U. *Clinical oncology (Royal College of Radiologists (Great Britain)), 15(8):490-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/14690006/" style="text-decoration: none;">PubMed</a>

* **Angiographic ablation of an ectopic mediastinal hyperplastic parathyroid gland using a left internal mammary artery coronary bypass.**  
  Ernst S, Cupisti K, Kemper J, Dotzenrath C, **Goretzki PE**, Furst G. *AJR. American journal of roentgenology, 181(1):95-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/12818837/" style="text-decoration: none;">PubMed</a>

* **3D QSAR analyses-guided rational design of novel ligands for the (alpha4)2(beta2)3 nicotinic acetylcholine receptor.**  
  Gohlke H, **Schwarz S**, Gundisch D, Tilotta MC, Weber A, Wegge T, Seitz G. *Journal of medicinal chemistry, 46(11):2031-48*. <a href="https://pubmed.ncbi.nlm.nih.gov/12747776/" style="text-decoration: none;">PubMed</a>

* **Operative treatment of renal autonomous hyperparathyroidism: cause of persistent or recurrent disease in 304 patients.**  
  Dotzenrath C, Cupisti K, **Goretzki E**, Mondry A, Vossough A, Grabensee B, Roher HD. *Langenbeck's archives of surgery, 387(9-10):348-54*. <a href="https://pubmed.ncbi.nlm.nih.gov/12536330/" style="text-decoration: none;">PubMed</a>


## 2002

* **Germline MSH2 and MLH1 mutational spectrum in HNPCC families from Poland and the Baltic States.**  
  Kurzawski G, Suchy J, Kladny J, Safranow K, Jakubowska A, Elsakov P, Kucinskas V, Gardovski J, Irmejs A, Sibul H, Huzarski T, Byrski T, Debniak T, Cybulski C, Gronwald J, Oszurek O, Clark J, Gozdz S, Niepsuj S, Slomski R, Plawski A, Lacka-Wojciechowska A, Rozmiarek A, Fiszer-Maliszewska L, Bebenek M, Sorokin D, Stawicka M, Godlewski D, Richter P, Brozek I, Wysocka B, Jawien A, Banaszkiewicz Z, Kowalczyk J, Czudowska D, **Goretzki PE**, Moeslein G, Lubinski J. *Journal of medical genetics, 39(10):E65*. <a href="https://pubmed.ncbi.nlm.nih.gov/12362047/" style="text-decoration: none;">PubMed</a>

* **[Kasabach-Merritt syndrome in giant hemangioma of the liver. A case report].**  
  Vogel T, Lammers B, von Herbay A, Kunz BM, Donner AJ, Furst G, **Goretzki PE**. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 73(7):729-32*. <a href="https://pubmed.ncbi.nlm.nih.gov/12242984/" style="text-decoration: none;">PubMed</a>

* **Effects of the putatively oncogenic protein kinase Calpha D294G mutation on enzymatic activity and cell growth and its occurrence in human thyroid neoplasias.**  
  Assert R, Kotter R, Schiemann U, **Goretzki P**, Pfeiffer AF. *Hormone and metabolic research = Hormon- und Stoffwechselforschung = Hormones et metabolisme, 34(6):311-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/12173071/" style="text-decoration: none;">PubMed</a>

* **Importance of lymph node metastases in follicular thyroid cancer.**  
  Witte J, **Goretzki PE**, Dieken J, Simon D, Roher HD. *World journal of surgery, 26(8):1017-22*. <a href="https://pubmed.ncbi.nlm.nih.gov/12045860/" style="text-decoration: none;">PubMed</a>

* **Adrenal involvement in multiple endocrine neoplasia type 1.**  
  Langer P, Cupisti K, Bartsch DK, Nies C, **Goretzki PE**, Rothmund M, Roher HD. *World journal of surgery, 26(8):891-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/12016472/" style="text-decoration: none;">PubMed</a>

* **Synthesis and nicotinic binding studies on enantiopure diazine analogues of the novel (2-chloro-5-pyridyl)-9-azabicyclo[4.2.1]non-2-ene UB-165.**  
  Gohlke H, Gundisch D, **Schwarz S**, Seitz G, Tilotta MC, Wegge T. *Journal of medicinal chemistry, 45(5):1064-72*. <a href="https://pubmed.ncbi.nlm.nih.gov/11855986/" style="text-decoration: none;">PubMed</a>

* **Therapy of suspected intrathoracic parathyroid adenomas. Experiences using open transthoracic approach and video-assisted thoracoscopic surgery.**  
  Cupisti K, Dotzenrath C, Simon D, Roher HD, **Goretzki PE**. *Langenbeck's archives of surgery, 386(7):488-93*. <a href="https://pubmed.ncbi.nlm.nih.gov/11819104/" style="text-decoration: none;">PubMed</a>

* **Syntheses and evaluation of pyridazine and pyrimidine containing bioisosteres of (+/-)-pyrido[3.4-b]homotropane and pyrido-[3.4-b]tropane as novel nAChR ligands.**  
  Gundisch D, Kampchen T, **Schwarz S**, Seitz G, Siegl J, Wegge T. *Bioorganic & medicinal chemistry, 10(1):1-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/11738601/" style="text-decoration: none;">PubMed</a>


## 2001

* **[Congenital diaphragmatic hernia in a polytrauma patient].**  
  Harms JM, Bohner H, Wenserski F, Rohrborn A, Muller E, **Goretzki PE**. *Der Unfallchirurg, 104(11):1107-10*. <a href="https://pubmed.ncbi.nlm.nih.gov/11760344/" style="text-decoration: none;">PubMed</a>

* **The microbiology of postoperative peritonitis.**  
  Roehrborn A, Thomas L, Potreck O, Ebener C, Ohmann C, **Goretzki PE**, Roher HD. *Clinical infectious diseases : an official publication of the Infectious Diseases Society of America, 33(9):1513-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/11568851/" style="text-decoration: none;">PubMed</a>

* **Soluble Fas is increased in hyperthyroidism independent of the underlying thyroid disease.**  
  Feldkamp J, Pascher E, Schott M, **Goretzki P**, Seissler J, Scherbaum WA. *The Journal of clinical endocrinology and metabolism, 86(9):4250-3*. <a href="https://pubmed.ncbi.nlm.nih.gov/11549657/" style="text-decoration: none;">PubMed</a>

* **Parathyroid carcinoma: problems in diagnosis and the need for radical surgery even in recurrent disease.**  
  Dotzenrath C, **Goretzki PE**, Sarbia M, Cupisti K, Feldkamp J, Roher HD. *European journal of surgical oncology : the journal of the European Society of Surgical Oncology and the British Association of Surgical Oncology, 27(4):383-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/11417985/" style="text-decoration: none;">PubMed</a>

* **Is there any consensus in diagnostic and operative strategy with respect to medullary thyroid cancer? A questionnaire answered by 73 endocrine surgical units.**  
  Dotzenrath C, **Goretzki PE**, Cupisti K, Simon D, Witte J, Yang Q, Ohmann C, Roher HD. *Langenbeck's archives of surgery, 386(1):47-52*. <a href="https://pubmed.ncbi.nlm.nih.gov/11405089/" style="text-decoration: none;">PubMed</a>

* **Long-term biochemical results after operative treatment of primary hyperparathyroidism associated with multiple endocrine neoplasia types I and IIa: is a more or less extended operation essential?**  
  Dotzenrath C, Cupisti K, **Goretzki PE**, Yang Q, Simon D, Ohmann C, Roher HD. *The European journal of surgery = Acta chirurgica, 167(3):173-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/11316400/" style="text-decoration: none;">PubMed</a>

* **Clinical management of malignant adrenal tumors.**  
  Kopf D, **Goretzki PE**, Lehnert H. *Journal of cancer research and clinical oncology, 127(3):143-55*. <a href="https://pubmed.ncbi.nlm.nih.gov/11260859/" style="text-decoration: none;">PubMed</a>

* **[When is total thyroidectomy indicated? What applies in thyroid carcinoma: radical surgery according to findings].**  
  **Goretzki PE**, Witte J. *MMW Fortschritte der Medizin, 143(1-2):32-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/11216010/" style="text-decoration: none;">PubMed</a>

* **Malignant endocrine tumors in patients with MEN 1 disease.**  
  Dotzenrath C, **Goretzki PE**, Cupisti K, Yang Q, Simon D, Roher HD. *Surgery, 129(1):91-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/11150038/" style="text-decoration: none;">PubMed</a>

* **Follicular thyroid cancer cells: a model of metastatic tumor in vitro (review).**  
  Hoelting T, **Goretzki PE**, Duh QY. *Oncology reports, 8(1):3-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/11115561/" style="text-decoration: none;">PubMed</a>


## 2000

* **Surgical treatment of postoperative, incidentally diagnosed small sporadic C-cell carcinomas of the thyroid.**  
  Cupisti K, Simon D, Wolf A, Gerharz CD, **Goretzki PE**, Dotzenrath C, Witte J, Roher HD. *Langenbeck's archives of surgery, 385(8):526-30*. <a href="https://pubmed.ncbi.nlm.nih.gov/11201009/" style="text-decoration: none;">PubMed</a>

* **[Stomach saving duodenopancreatectomy. Indications and contraindications. The most important surgical steps].**  
  Roher HD, Heise JW, **Goretzki PE**. *Zentralblatt fur Chirurgie, 125(12):961-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/11190613/" style="text-decoration: none;">PubMed</a>

* **["Field cancerization"--an additional phenomenon in development of colon tumors? K-ras codon 12 mutations in normal colonic mucosa of patients with colorectal neoplasms].**  
  Aivado M, Gynes M, Gorelov V, Schmidt WU, Roher HD, **Goretzki PE**. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 71(10):1230-4; discussion 1234-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/11077584/" style="text-decoration: none;">PubMed</a>

* **Surgery for Graves' disease: total versus subtotal thyroidectomy-results of a prospective randomized trial.**  
  Witte J, **Goretzki PE**, Dotzenrath C, Simon D, Felis P, Neubauer M, Roher HD. *World journal of surgery, 24(11):1303-11*. <a href="https://pubmed.ncbi.nlm.nih.gov/11038198/" style="text-decoration: none;">PubMed</a>

* **Therapeutic experience of 65 cases with organic hyperinsulinism.**  
  Hellman P, **Goretzki P**, Simon D, Dotzenrath C, Roher HD. *Langenbeck's archives of surgery, 385(5):329-36*. <a href="https://pubmed.ncbi.nlm.nih.gov/11026704/" style="text-decoration: none;">PubMed</a>

* **Gastric cancer: which patients benefit from systematic lymphadenectomy?**  
  Bosing NM, **Goretzki PE**, Roher HD. *European journal of surgical oncology : the journal of the European Society of Surgical Oncology and the British Association of Surgical Oncology, 26(5):498-505*. <a href="https://pubmed.ncbi.nlm.nih.gov/11016473/" style="text-decoration: none;">PubMed</a>

* **Binding of the NG2 proteoglycan to kringle domains modulates the functional properties of angiostatin and plasmin(ogen).**  
  **Goretzki L**, Lombardo CR, Stallcup WB. *The Journal of biological chemistry, 275(37):28625-33*. <a href="https://pubmed.ncbi.nlm.nih.gov/10889192/" style="text-decoration: none;">PubMed</a>

* **Growth regulation of thyroid and thyroid tumors in humans.**  
  **Goretzki PE**, Simon D, Dotzenrath C, Schulte KM, Roher HD. *World journal of surgery, 24(8):913-22*. <a href="https://pubmed.ncbi.nlm.nih.gov/10865035/" style="text-decoration: none;">PubMed</a>

* **Effects of a novel corticotropin-releasing-hormone receptor type I antagonist on human adrenal function.**  
  Willenberg HS, Bornstein SR, Hiroi N, Path G, **Goretzki PE**, Scherbaum WA, Chrousos GP. *Molecular psychiatry, 5(2):137-41*. <a href="https://pubmed.ncbi.nlm.nih.gov/10822340/" style="text-decoration: none;">PubMed</a>

* **Lack of MEN1 gene mutations in 27 sporadic insulinomas.**  
  Cupisti K, Hoppner W, Dotzenrath C, Simon D, Berndt I, Roher HD, **Goretzki PE**. *European journal of clinical investigation, 30(4):325-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/10759881/" style="text-decoration: none;">PubMed</a>


## 1999

* **cAMP-synthesis in a medullary thyroid carcinoma cell line: response to adrenergic agents and prostaglandines.**  
  Mertens PR, **Goretzki PE**, Keck E. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 107(8):488-95*. <a href="https://pubmed.ncbi.nlm.nih.gov/10612479/" style="text-decoration: none;">PubMed</a>

* **[Complications in thyroid surgery. Incidence and therapy].**  
  Roher HD, **Goretzki PE**, Hellmann P, Witte J. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 70(9):999-1010*. <a href="https://pubmed.ncbi.nlm.nih.gov/10501664/" style="text-decoration: none;">PubMed</a>

* **Urokinase-type plasminogen activator binding to its receptor stimulates tumor cell migration by enhancing integrin-mediated signal transduction.**  
  Yebra M, **Goretzki L**, Pfeifer M, Mueller BM. *Experimental cell research, 250(1):231-40*. <a href="https://pubmed.ncbi.nlm.nih.gov/10388537/" style="text-decoration: none;">PubMed</a>

* **Expression of wild-type ret, ret/PTC and ret/PTC variants in papillary thyroid carcinoma in Germany.**  
  Mayr B, Potter E, **Goretzki P**, Ruschoff J, Dietmaier W, Hoang-Vu C, Dralle H, Brabant G. *Langenbeck's archives of surgery, 384(1):54-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/10367631/" style="text-decoration: none;">PubMed</a>

* **Studies of oncogenes and tumor-suppressor genes in human thyroid carcinomas, and their clinical implications.**  
  **Goretzki PE**, Dotzenrath C, Simon D, Roher HD. *Langenbeck's archives of surgery, 384(1):1-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/10367623/" style="text-decoration: none;">PubMed</a>

* **High-affinity binding of basic fibroblast growth factor and platelet-derived growth factor-AA to the core protein of the NG2 proteoglycan.**  
  **Goretzki L**, Burg MA, Grako KA, Stallcup WB. *The Journal of biological chemistry, 274(24):16831-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/10358027/" style="text-decoration: none;">PubMed</a>

* **An in vitro model for peroxisome proliferation utilizing primary hepatocytes in sandwich culture.**  
  Hildebrand H, Schmidt U, Kempka G, Jacob R, Ahr HJ, Ebener C, **Goretzki PE**, Bader A. *Toxicology in vitro : an international journal published in association with BIBRA, 13(2):265-73*. <a href="https://pubmed.ncbi.nlm.nih.gov/20654484/" style="text-decoration: none;">PubMed</a>


## 1998

* **[Tumor necrosis factor-alpha (TNF-alpha) gene polymorphism in surgical intensive care patients with SIRS].**  
  Dumon K, Rossbach C, Harms B, Gorelov V, Gross-Weege W, Schneider EM, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 115(Suppl I):387-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/14518281/" style="text-decoration: none;">PubMed</a>

* **[Protein expression of hMSH2 and hMLH1 in HNPCC: evaluation of a prescreening method].**  
  Vossen S, Moslein G, Katzer M, Gabbert HE, Muller W, Wirtz C, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 115:1461-3*. <a href="https://pubmed.ncbi.nlm.nih.gov/9931912/" style="text-decoration: none;">PubMed</a>

* **[Surgical procedure for hyperthyroidism in children and adolescents].**  
  Witte J, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 115:1048-50*. <a href="https://pubmed.ncbi.nlm.nih.gov/9931786/" style="text-decoration: none;">PubMed</a>

* **[Pelvic exenteration from the surgical viewpoint].**  
  **Goretzki PE**, Goebell PJ, Vogel T, Schnurch HG, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 115:246-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/9931619/" style="text-decoration: none;">PubMed</a>

* **[Geographical differences of thyroid carcinoma and basic molecular principles].**  
  **Goretzki PE**, Witte J, Dotzenrath C, Schulte KM, Simon D, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 115:200-2*. <a href="https://pubmed.ncbi.nlm.nih.gov/9931611/" style="text-decoration: none;">PubMed</a>

* **Genetic and biochemical screening for endocrine disease.**  
  **Goretzki PE**, Hoppner W, Dotzenrath C, Clark OH, Simon D, Cupisti K, Schulte H, Roher HD. *World journal of surgery, 22(12):1202-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/9841744/" style="text-decoration: none;">PubMed</a>

* **Low-density-lipoprotein-receptor-related protein (LRP) interacts with a GTP-binding protein.**  
  **Goretzki L**, Mueller BM. *The Biochemical journal, 336 ( Pt 2)(Pt 2):381-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/9820815/" style="text-decoration: none;">PubMed</a>

* **Rare loss of heterozygosity of the MTS1 and MTS2 tumor suppressor genes in differentiated human thyroid cancer.**  
  Schulte KM, Staudt S, Niederacher D, Finken-Eigen M, Kohrer K, **Goretzki PE**, Roher HD. *Hormone and metabolic research = Hormon- und Stoffwechselforschung = Hormones et metabolisme, 30(9):549-54*. <a href="https://pubmed.ncbi.nlm.nih.gov/9808321/" style="text-decoration: none;">PubMed</a>

* **Regulation of the HGF-receptor c-met in the thyroid gland.**  
  Schulte KM, Antoch G, Ellrichmann M, Finken-Eigen M, Kohrer K, Simon D, **Goretzki PE**, Roher HD. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 106(4):310-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/9792464/" style="text-decoration: none;">PubMed</a>

* **[Significance of oncogenes in development and prognosis of differentiated thyroid gland carcinomas].**  
  **Goretzki PE**, Schulte KM. *Der Internist, 39(6):584-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/9677513/" style="text-decoration: none;">PubMed</a>

* **Therapeutic management of rare malignant pancreatic tumors in children.**  
  Vossen S, **Goretzki PE**, Goebel U, Willnow U. *World journal of surgery, 22(8):879-82*. <a href="https://pubmed.ncbi.nlm.nih.gov/9673563/" style="text-decoration: none;">PubMed</a>

* **Chance and risk of controlling rabies in large-scale and long-term immunized fox populations.**  
  Tischendorf L, Thulke HH, Staubach C, Muller MS, Jeltsch F, **Goretzki J**, Selhorst T, Muller T, Schluter H, Wissel C. *Proceedings. Biological sciences, 265(1399):839-46*. <a href="https://pubmed.ncbi.nlm.nih.gov/9633109/" style="text-decoration: none;">PubMed</a>

* **Reoperative surgery for organic hyperinsulinism: indications and operative strategy.**  
  Simon D, Starke A, **Goretzki PE**, Roeher HD. *World journal of surgery, 22(7):666-71; discussion 671-2*. <a href="https://pubmed.ncbi.nlm.nih.gov/9606279/" style="text-decoration: none;">PubMed</a>

* **Redifferentiation therapy with retinoids: therapeutic option for advanced follicular and papillary thyroid carcinoma.**  
  Simon D, Koehrle J, Reiners C, Boerner AR, Schmutzler C, Mainz K, **Goretzki PE**, Roeher HD. *World journal of surgery, 22(6):569-74*. <a href="https://pubmed.ncbi.nlm.nih.gov/9597930/" style="text-decoration: none;">PubMed</a>

* **Functional retinoid and thyroid hormone receptors in human thyroid-carcinoma cell lines and tissues.**  
  schmutzler C, Brtko J, Winzer R, Jakobs TC, Meissner-Weigl J, Simon D, **Goretzki PE**, Kohrle J. *International journal of cancer, 76(3):368-76*. <a href="https://pubmed.ncbi.nlm.nih.gov/9579574/" style="text-decoration: none;">PubMed</a>

* **[From limited to extensive radical operation in thyroid carcinoma].**  
  Simon D, **Goretzki PE**, Roher HD. *Praxis, 87(11):383-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/9562814/" style="text-decoration: none;">PubMed</a>

* **Expression of Ret/PTC1, -2, -3, -delta3 and -4 in German papillary thyroid carcinoma.**  
  Mayr B, Potter E, **Goretzki P**, Ruschoff J, Dietmaier W, Hoang-Vu C, Dralle H, Brabant G. *British journal of cancer, 77(6):903-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/9528832/" style="text-decoration: none;">PubMed</a>


## 1997

* **Receptor-mediated endocytosis of urokinase-type plasminogen activator is regulated by cAMP-dependent protein kinase.**  
  **Goretzki L**, Mueller BM. *Journal of cell science, 110 ( Pt 12):1395-402*. <a href="https://pubmed.ncbi.nlm.nih.gov/9217325/" style="text-decoration: none;">PubMed</a>

* **ret/PTC-1, -2, and -3 oncogene rearrangements in human thyroid carcinomas: implications for metastatic potential?**  
  Mayr B, Brabant G, **Goretzki P**, Ruschoff J, Dietmaier W, Dralle H. *The Journal of clinical endocrinology and metabolism, 82(4):1306-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/9100614/" style="text-decoration: none;">PubMed</a>

* **Inhibition of the interaction of urokinase-type plasminogen activator (uPA) with its receptor (uPAR) by synthetic peptides.**  
  Burgle M, Koppitz M, Riemer C, Kessler H, Konig B, Weidle UH, Kellermann J, Lottspeich F, Graeff H, Schmitt M, **Goretzki L**, Reuning U, Wilhelm O, Magdolen V. *Biological chemistry, 378(3-4):231-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/9165076/" style="text-decoration: none;">PubMed</a>

* **Differential loss of heterozygosity in familial, sporadic, and uremic hyperparathyroidism.**  
  Farnebo F, Teh BT, Dotzenrath C, Wassif WS, Svensson A, White I, Betz R, **Goretzki P**, Sandelin K, Farnebo LO, Larsson C. *Human genetics, 99(3):342-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/9050920/" style="text-decoration: none;">PubMed</a>

* **[Surgical therapy of neuroendocrine tumors of the thymus].**  
  Cupisti K, Dotzenrath C, Simon D, **Goretzki PE**, Roher HD. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 68(2):136-40*. <a href="https://pubmed.ncbi.nlm.nih.gov/9156979/" style="text-decoration: none;">PubMed</a>

* **[Special diagnostic and therapeutic aspects of insulinoma].**  
  Roher HD, Simon D, Starke A, **Goretzki PE**. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 68(2):116-21*. <a href="https://pubmed.ncbi.nlm.nih.gov/9156976/" style="text-decoration: none;">PubMed</a>

* **[Does rationing of intensive care beds lead to premature ward return with preventable complications?].**  
  Ebener C, **Goretzki PE**, Grabitz K, Elbertzhagen D, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 114:1396-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/9574440/" style="text-decoration: none;">PubMed</a>

* **[Does decreased preoperative diagnosis in goiter surgery present special challenges for the surgeon? What stays indispensible? What is superfluous?].**  
  Witte J, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 114:1145-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/9574360/" style="text-decoration: none;">PubMed</a>

* **[Instrumental diagnosis for therapeutic decisions--what is possible and desirable, what is essential and what is superfluous--in goiter surgery].**  
  Roher HD, Simon D, **Goretzki PE**. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 114:387-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/9574162/" style="text-decoration: none;">PubMed</a>

* **[Guidelines in oncologic surgery--malignant thyroid tumors].**  
  Roher HD, Simon D, **Goretzki PE**. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 114:142-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/9574114/" style="text-decoration: none;">PubMed</a>

* **[Results of selective venous site-specific catheterization in occult C-cell carcinoma of the thyroid gland].**  
  Cupisti K, Simon D, Dotzenrath C, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie, 382(6):295-301*. <a href="https://pubmed.ncbi.nlm.nih.gov/9498199/" style="text-decoration: none;">PubMed</a>

* **Surgery for Graves disease in childhood and adolescence.**  
  Witte J, **Goretzki PE**, Roher HD. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 105 Suppl 4:58-60*. <a href="https://pubmed.ncbi.nlm.nih.gov/9439918/" style="text-decoration: none;">PubMed</a>

* **[Conventional operation for therapy of incidentaloma].**  
  Simon D, **Goretzki PE**, Roher HD. *Zentralblatt fur Chirurgie, 122(6):473-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/9334115/" style="text-decoration: none;">PubMed</a>

* **[Surgical concept in sporadic and familial pheochromocytoma].**  
  **Goretzki PE**, Simon D, Roher HD. *Zentralblatt fur Chirurgie, 122(6):467-72*. <a href="https://pubmed.ncbi.nlm.nih.gov/9334114/" style="text-decoration: none;">PubMed</a>

* **[Significance of lymph node metastases of differentiated thyroid gland carcinomas and C-cell carcinomas for prognosis--a meta-analysis].**  
  Witte J, Schlotmann U, Simon D, Dotzenrath C, Ohmann C, **Goretzki PE**. *Zentralblatt fur Chirurgie, 122(4):259-65*. <a href="https://pubmed.ncbi.nlm.nih.gov/9221636/" style="text-decoration: none;">PubMed</a>

* **Iodide or L-thyroxine to prevent recurrent goiter in an iodine-deficient area: prospective sonographic study.**  
  Feldkamp J, Seppel T, Becker A, Klisch A, Schlaghecke R, **Goretzki PE**, Roher HD. *World journal of surgery, 21(1):10-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/8943171/" style="text-decoration: none;">PubMed</a>


## 1996

* **A frequent mutation/polymorphism in tumor suppressor gene INK4B (MTS-2) in papillary and medullary thyroid cancer.**  
  **Goretzki PE**, Gorelov V, Dotzenrath C, Witte J, Roeher HD. *Surgery, 120(6):1081-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/8957499/" style="text-decoration: none;">PubMed</a>

* **Microsatellite instability and mutation analysis of hMSH2 and hMLH1 in patients with sporadic, familial and hereditary colorectal cancer.**  
  Moslein G, Tester DJ, Lindor NM, Honchel R, Cunningham JM, French AJ, Halling KC, Schwab M, **Goretzki P**, Thibodeau SN. *Human molecular genetics, 5(9):1245-52*. <a href="https://pubmed.ncbi.nlm.nih.gov/8872463/" style="text-decoration: none;">PubMed</a>

* **Allelic loss of the retinoblastoma tumor suppressor gene: a marker for aggressive parathyroid tumors?**  
  Dotzenrath C, Teh BT, Farnebo F, Cupisti K, Svensson A, Toell A, **Goretzki P**, Larsson C. *The Journal of clinical endocrinology and metabolism, 81(9):3194-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/8784068/" style="text-decoration: none;">PubMed</a>

* **Incidence of regional recurrence guiding radicality in differentiated thyroid carcinoma.**  
  Simon D, **Goretzki PE**, Witte J, Roher HD. *World journal of surgery, 20(7):860-6; discussion 866*. <a href="https://pubmed.ncbi.nlm.nih.gov/8678963/" style="text-decoration: none;">PubMed</a>

* **[Endocrine pancreatic tumors].**  
  **Goretzki PE**, Simon D, Starke A, Roher HD. *Therapeutische Umschau. Revue therapeutique, 53(5):408-12*. <a href="https://pubmed.ncbi.nlm.nih.gov/8685860/" style="text-decoration: none;">PubMed</a>

* **Systematic mutational analysis of the receptor-binding region of the human urokinase-type plasminogen activator.**  
  Magdolen V, Rettenberger P, Koppitz M, **Goretzki L**, Kessler H, Weidle UH, Konig B, Graeff H, Schmitt M, Wilhelm O. *European journal of biochemistry, 237(3):743-51*. <a href="https://pubmed.ncbi.nlm.nih.gov/8647121/" style="text-decoration: none;">PubMed</a>

* **[Is total thyroidectomy and lymph node excision as therapy of differentiated thyroid gland carcinomas in childhood justified?].**  
  Witte J, **Goretzki PE**, Simon D, Roher HD. *Langenbecks Archiv fur Chirurgie. Supplement. Kongressband. Deutsche Gesellschaft fur Chirurgie. Kongress, 113:196-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/9101829/" style="text-decoration: none;">PubMed</a>

* **Molecular genetics of primary and secondary hyperparathyroidism.**  
  Dotzenrath C, **Goretzki PE**, Farnebo F, Teh BT, Weber G, Roher HD, Larsson C. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 104 Suppl 4:105-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/8981014/" style="text-decoration: none;">PubMed</a>

* **INT-2 gene amplification in differentiated human thyroid cancer.**  
  Schulte KM, Niederacher D, An HX, Staudt S, Simon D, Beckmann MW, **Goretzki PE**. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 104 Suppl 4:101-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/8981013/" style="text-decoration: none;">PubMed</a>

* **A modified PCR-based method for rapid non-radioactive detection of clinically important pathogens.**  
  Gorelov VN, Dumon K, Barteneva NS, Roher HD, **Goretzki PE**. *Microbiology and immunology, 40(9):611-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/8908604/" style="text-decoration: none;">PubMed</a>

* **[Ulcer surgery '96--choice of methods in an emergency].**  
  Roher HD, Imhof M, **Goretzki PE**, Ohmann C. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 67(1):20-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/8851671/" style="text-decoration: none;">PubMed</a>

* **[Results of surgical therapy in renal hyperparathyroidism. Follow-up of 143 patients].**  
  Dotzenrath C, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie, 381(1):46-50*. <a href="https://pubmed.ncbi.nlm.nih.gov/8717175/" style="text-decoration: none;">PubMed</a>

* **Distribution of Gs-alpha activating mutations in human thyroid tumors measured by subcloning.**  
  Gorelov VN, Gyenes M, Neser F, Roher HD, **Goretzki PE**. *Journal of cancer research and clinical oncology, 122(8):453-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/8698744/" style="text-decoration: none;">PubMed</a>


## 1995

* **[Preventive radical surgery of C-cell carcinoma in MEN-II syndrome based on genetic screening].**  
  Roher HD, Simon D, **Goretzki PE**, Hoppner W, Lederbogen S, Seppel T. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 66(12):1196-202*. <a href="https://pubmed.ncbi.nlm.nih.gov/8582162/" style="text-decoration: none;">PubMed</a>

* **A competitive chromogenic assay to study the functional interaction of urokinase-type plasminogen activator with its receptor.**  
  Rettenberger P, Wilhelm O, Oi H, Weidle UH, **Goretzki L**, Koppitz M, Lottspeich F, Konig B, Pessara U, Kramer MD, et al.. *Biological chemistry Hoppe-Seyler, 376(10):587-94*. <a href="https://pubmed.ncbi.nlm.nih.gov/8590627/" style="text-decoration: none;">PubMed</a>

* **[Differentiated thyroid gland carcinomas in "autonomous adenomas" in childhood].**  
  Muller A, **Goretzki P**, Witte J, Gerharz P, Roher HD. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 66(10):1018-20*. <a href="https://pubmed.ncbi.nlm.nih.gov/8529444/" style="text-decoration: none;">PubMed</a>

* **N-3 but not N-6 fatty acids reduce the expression of the combined adhesion and scavenger receptor CD36 in human monocytic cells.**  
  Pietsch A, Weber C, **Goretzki M**, Weber PC, Lorenz RL. *Cell biochemistry and function, 13(3):211-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/7554100/" style="text-decoration: none;">PubMed</a>

* **Prognostic value of the cysteine proteases cathepsins B and cathepsin L in human breast cancer.**  
  Thomssen C, Schmitt M, **Goretzki L**, Oppelt P, Pache L, Dettmar P, Janicke F, Graeff H. *Clinical cancer research : an official journal of the American Association for Cancer Research, 1(7):741-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/9816040/" style="text-decoration: none;">PubMed</a>

* **[Current aspects of surgery of the parathyroid glands].**  
  Dotzenrath C, **Goretzki PE**, Simon D, Witte J, Roher HD. *Journal de chirurgie, 132(5):270-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/7642736/" style="text-decoration: none;">PubMed</a>

* **Expression of the human urokinase-type plasminogen activator receptor in E. coli and Chinese hamster ovary cells: purification of the recombinant proteins and generation of polyclonal antibodies in chicken.**  
  Magdolen V, Rettenberger P, Lopens A, Oi H, Lottspeich F, Kellermann J, Creutzburg S, **Goretzki L**, Weidle UH, Wilhelm O, et al.. *Electrophoresis, 16(5):813-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/7588568/" style="text-decoration: none;">PubMed</a>

* **Retinoic acid inhibits basal and interferon-gamma-induced expression of intercellular adhesion molecule 1 in monocytic cells.**  
  Weber C, Calzada-Wack JC, **Goretzki M**, Pietsch A, Johnson JP, Ziegler-Heitbrock HW. *Journal of leukocyte biology, 57(3):401-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/7533818/" style="text-decoration: none;">PubMed</a>

* **Molecular analysis of mutations in thyroid tumors with TGGE.**  
  Simon D, **Goretzki PE**, Schafer C, Gorelov V, Ebeling B, Roher HD. *Experimental and clinical endocrinology & diabetes : official journal, German Society of Endocrinology [and] German Diabetes Association, 103(5):275-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/8536054/" style="text-decoration: none;">PubMed</a>

* **Overexpression of Gs alpha subunit in thyroid tumors bearing a mutated Gs alpha gene.**  
  Gorelov VN, Dumon K, Barteneva NS, Palm D, Roher HD, **Goretzki PE**. *Journal of cancer research and clinical oncology, 121(4):219-24*. <a href="https://pubmed.ncbi.nlm.nih.gov/7751320/" style="text-decoration: none;">PubMed</a>


## 1994

* **[Surgical treatment of esophageal perforations].**  
  **Goretzki PE**, Horstmann O, Verreet PR, Rohr HD. *Journal de chirurgie, 131(10):440-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/7860679/" style="text-decoration: none;">PubMed</a>

* **Diminished expression of the alpha 5 beta 1 integrin (fibronectin receptor) by invasive clones of a human follicular thyroid cancer cell line.**  
  Demeure MJ, Doffek KM, Rezaee M, **Goretzki PE**, Wilson SD. *World journal of surgery, 18(4):569-75; discussion 575-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/7725746/" style="text-decoration: none;">PubMed</a>

* **Significance of P53 in human thyroid tumors.**  
  Simon D, **Goretzki PE**, Gorelev V, Ebling B, Reishaus E, Lyons J, Haubruck H, Roher HD. *World journal of surgery, 18(4):535-40; discussion 540-1*. <a href="https://pubmed.ncbi.nlm.nih.gov/7725741/" style="text-decoration: none;">PubMed</a>

* **A method to increase the sensitivity of mutation specific oligonucleotide hybridization using asymmetric polymerase-chain reaction (PCR).**  
  Gorelov VN, Roher HD, **Goretzki PE**. *Biochemical and biophysical research communications, 200(1):365-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/8166707/" style="text-decoration: none;">PubMed</a>

* **[Surgical strategy in persistence and recurrence in surgery of primary hyperparathyroidism].**  
  Dotzenrath C, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie, 379(4):218-23*. <a href="https://pubmed.ncbi.nlm.nih.gov/7934579/" style="text-decoration: none;">PubMed</a>


## 1993

* **Persistent hypertension after successful adrenal operation.**  
  Simon D, **Goretzki PE**, Lollert A, Roher HD. *Surgery, 114(6):1189-95*. <a href="https://pubmed.ncbi.nlm.nih.gov/8256225/" style="text-decoration: none;">PubMed</a>

* **Principals of limited or radical surgery for differentiated thyroid cancer.**  
  Roeher HD, Simon D, Witte J, **Goretzki PE**. *Thyroidology, 5(3):93-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/7524636/" style="text-decoration: none;">PubMed</a>

* **Surgical reintervention for differentiated thyroid cancer.**  
  **Goretzki PE**, Simon D, Frilling A, Witte J, Reiners C, Grussendorf M, Horster FA, Roher HD. *The British journal of surgery, 80(8):1009-12*. <a href="https://pubmed.ncbi.nlm.nih.gov/8402050/" style="text-decoration: none;">PubMed</a>

* **Renal hyperparathyroidism following kidney transplantation.**  
  Dotzenrath C, **Goretzki PE**, Roher HD. *Annali italiani di chirurgia, 64(4):381-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/8154661/" style="text-decoration: none;">PubMed</a>

* **Expression of the human TSH receptor in a human thyroid carcinoma cell line that lacks an endogenous TSH receptor: growth inhibition by cAMP.**  
  Derwahl M, Kuemmel M, **Goretzki P**, Schatz H, Broecker M. *Biochemical and biophysical research communications, 191(3):1131-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/8385451/" style="text-decoration: none;">PubMed</a>

* **Saturation of tumour cell surface receptors for urokinase-type plasminogen activator by amino-terminal fragment and subsequent effect on reconstituted basement membranes invasion.**  
  Kobayashi H, Ohi H, Shinohara H, Sugimura M, Fujii T, Terao T, Schmitt M, **Goretzki L**, Chucholowski N, Janicke F, et al.. *British journal of cancer, 67(3):537-44*. <a href="https://pubmed.ncbi.nlm.nih.gov/8382511/" style="text-decoration: none;">PubMed</a>

* **[Surgical therapy of secondary hyperparathyroidism after kidney transplantation].**  
  Dotzenrath C, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie, 378(2):121-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/8474295/" style="text-decoration: none;">PubMed</a>

* **[Experience with presymptomatic screening of patients with C cell carcinoma of the thyroid gland].**  
  Frilling A, Roher HD, Ponder BA, **Goretzki PE**, Schlaghecke R, Reiners C. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 64(1):28-35*. <a href="https://pubmed.ncbi.nlm.nih.gov/8094664/" style="text-decoration: none;">PubMed</a>


## 1992

* **[Surgical treatment of carcinoid tumors of the intestine].**  
  Simon D, **Goretzki PE**, Branscheid D, Roher JD. *Journal de chirurgie, 129(10):449-55*. <a href="https://pubmed.ncbi.nlm.nih.gov/1294589/" style="text-decoration: none;">PubMed</a>

* **Mutational activation of RAS and GSP oncogenes in differentiated thyroid cancer and their biological implications.**  
  **Goretzki PE**, Lyons J, Stacy-Phipps S, Rosenau W, Demeure M, Clark OH, McCormick F, Roher HD, Bourne HR. *World journal of surgery, 16(4):576-81; discussion 581-2*. <a href="https://pubmed.ncbi.nlm.nih.gov/1413827/" style="text-decoration: none;">PubMed</a>

* **Invasion by cultured human follicular thyroid cancer correlates with increased beta 1 integrins and production of proteases.**  
  Demeure MJ, Damsky CH, Elfman F, **Goretzki PE**, Wong MG, Clark OH. *World journal of surgery, 16(4):770-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/1384245/" style="text-decoration: none;">PubMed</a>

* **Effective activation of the proenzyme form of the urokinase-type plasminogen activator (pro-uPA) by the cysteine protease cathepsin L.**  
  **Goretzki L**, Schmitt M, Mann K, Calvete J, Chucholowski N, Kramer M, Gunzler WA, Janicke F, Graeff H. *FEBS letters, 297(1-2):112-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/1551416/" style="text-decoration: none;">PubMed</a>

* **Flow cytofluorometry in tumour cell receptor analysis. Survey of the literature and recent developments concerning the urokinase-type plasminogen activator (uPA).**  
  Chucholowski N, Schmitt M, **Goretzki L**, Schuren E, Moniwa N, Weidle U, Kramer M, Wagner B, Janicke F, Graeff H. *Biochemical Society transactions, 20(1):208-16*. <a href="https://pubmed.ncbi.nlm.nih.gov/1321752/" style="text-decoration: none;">PubMed</a>

* **G-protein mutations in thyroid tumors.**  
  **Goretzki PE**, Simon D, Roher HD. *Experimental and clinical endocrinology, 100(1-2):14-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/1468509/" style="text-decoration: none;">PubMed</a>


## 1991

* **Management of primary hyperparathyroidism caused by multiple gland disease.**  
  **Goretzki PE**, Dotzenrath C, Roeher HD. *World journal of surgery, 15(6):693-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/1685044/" style="text-decoration: none;">PubMed</a>

* **Mutation of the p53 gene in a differentiated human thyroid carcinoma cell line, but not in primary thyroid tumours.**  
  Wright PA, Lemoine NR, **Goretzki PE**, Wyllie FS, Bond J, Hughes C, Roher HD, Williams ED, Wynford-Thomas D. *Oncogene, 6(9):1693-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/1923534/" style="text-decoration: none;">PubMed</a>

* **Fluorescent probes as tools to assess the receptor for the urokinase-type plasminogen activator on tumor cells.**  
  Schmitt M, Chucholowski N, Busch E, Hellmann D, Wagner B, **Goretzki L**, Janicke F, Gunzler WA, Graeff H. *Seminars in thrombosis and hemostasis, 17(3):291-302*. <a href="https://pubmed.ncbi.nlm.nih.gov/1724574/" style="text-decoration: none;">PubMed</a>

* **Surgery for immunogenic hyperthyroidism.**  
  Roher HD, Horster FA, Frilling A, **Goretzki PE**, Witte J. *Experimental and clinical endocrinology, 97(2-3):292-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/1915647/" style="text-decoration: none;">PubMed</a>

* **Correlated abnormalities of transforming growth factor-beta 1 response and p53 expression in thyroid epithelial cell transformation.**  
  Wyllie FS, Dawson T, Bond JA, **Goretzki P**, Game S, Prime S, Wynford-Thomas D. *Molecular and cellular endocrinology, 76(1-3):13-21*. <a href="https://pubmed.ncbi.nlm.nih.gov/1820969/" style="text-decoration: none;">PubMed</a>

* **Cathepsin B efficiently activates the soluble and the tumor cell receptor-bound form of the proenzyme urokinase-type plasminogen activator (Pro-uPA).**  
  Kobayashi H, Schmitt M, **Goretzki L**, Chucholowski N, Calvete J, Kramer M, Gunzler WA, Janicke F, Graeff H. *The Journal of biological chemistry, 266(8):5147-52*. <a href="https://pubmed.ncbi.nlm.nih.gov/1900515/" style="text-decoration: none;">PubMed</a>

* **[Morphology and function-based surgery in different forms of hyperthyroidism].**  
  Roher HD, Horster FA, Frilling A, **Goretzki PE**. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 62(3):176-81*. <a href="https://pubmed.ncbi.nlm.nih.gov/1709844/" style="text-decoration: none;">PubMed</a>

* **Biological and clinical relevance of the urokinase-type plasminogen activator (uPA) in breast cancer.**  
  Schmitt M, **Goretzki L**, Janicke F, Calvete J, Eulitz M, Kobayashi H, Chucholowski N, Graeff H. *Biomedica biochimica acta, 50(4-6):731-41*. <a href="https://pubmed.ncbi.nlm.nih.gov/1801751/" style="text-decoration: none;">PubMed</a>


## 1990

* **Actin architecture of cultured human thyroid cancer cells: predictor of differentiation?**  
  Demeure MJ, Hughes-Fulford M, **Goretzki PE**, Duh QY, Clark OH. *Surgery, 108(6):986-92; discussion 992-3*. <a href="https://pubmed.ncbi.nlm.nih.gov/2247845/" style="text-decoration: none;">PubMed</a>

* **West Germany: still an underdeveloped country in the diagnosis and early treatment of primary hyperparathyroidism?**  
  Dotzenrath C, **Goretzki PE**, Roher HD. *World journal of surgery, 14(5):660-1; discussion 662*. <a href="https://pubmed.ncbi.nlm.nih.gov/2238668/" style="text-decoration: none;">PubMed</a>

* **[Subtotal thyroid gland resection as therapy for thyrotoxic crises].**  
  Frilling A, **Goretzki PE**, Horster FA, Grussendorf M, Roher HD. *Deutsche medizinische Wochenschrift (1946), 115(19):735-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/2338059/" style="text-decoration: none;">PubMed</a>

* **The influence of surgery on endocrine ophthalmopathy.**  
  Frilling A, **Goretzki PE**, Grussendorf M, Erbsloh M, Roher HD. *World journal of surgery, 14(3):442-5; discussion 445-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/2368450/" style="text-decoration: none;">PubMed</a>

* **[Acute pancreatitis as the initial manifestation of primary hyperparathyroidism in children].**  
  Dotzenrath C, Raunest J, **Goretzki PE**, Schmitt-Graff A, Roher HD. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 61(4):329-30*. <a href="https://pubmed.ncbi.nlm.nih.gov/2347271/" style="text-decoration: none;">PubMed</a>

* **[Iodine-induced crisis in thyroid autonomy: a surgical emergency].**  
  Frilling A, **Goretzki PE**, Roher HD. *Acta medica Austriaca, 17 Suppl 1:86-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/2389645/" style="text-decoration: none;">PubMed</a>

* **[Progress in the diagnosis and therapy of C cell carcinoma of the thyroid gland].**  
  Frilling A, **Goretzki PE**, Roher HD. *Langenbecks Archiv fur Chirurgie, 375(6):333-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/2280648/" style="text-decoration: none;">PubMed</a>

* **[Therapeutic strategy and prognosis of malignant struma].**  
  Roher HD, **Goretzki PE**, Frilling A. *Zentralblatt fur Chirurgie, 115(7):389-97*. <a href="https://pubmed.ncbi.nlm.nih.gov/2195806/" style="text-decoration: none;">PubMed</a>

* **Growth regulation of normal thyroids and thyroid tumors in man.**  
  **Goretzki PE**, Frilling A, Simon D, Roeher HD. *Recent results in cancer research. Fortschritte der Krebsforschung. Progres dans les recherches sur le cancer, 118:48-63*. <a href="https://pubmed.ncbi.nlm.nih.gov/2173080/" style="text-decoration: none;">PubMed</a>


## 1989

* **[Various strategies in the diagnosis and therapy of thyroid gland cancer. Results of a survey].**  
  **Goretzki PE**, Frilling A, Ohmann C, Wins L, Grussendorf M, Roher HD. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 60(6):398-402*. <a href="https://pubmed.ncbi.nlm.nih.gov/2758893/" style="text-decoration: none;">PubMed</a>

* **[Intrathoracic struma].**  
  Roher HD, **Goretzki PE**, Wahl RA, Frilling A. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 60(6):384-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/2667906/" style="text-decoration: none;">PubMed</a>

* **[Indications for and basic principles of the surgical treatment of thyroid diseases].**  
  Roher HD, **Goretzki PE**, Frilling A. *Der Radiologe, 29(3):119-24*. <a href="https://pubmed.ncbi.nlm.nih.gov/2652183/" style="text-decoration: none;">PubMed</a>

* **Effect of near total thyroidectomy on ophthalmopathy in patients with Graves' disease.**  
  Grussendorf M, Inanc Y, **Goretzki PE**, Roher HD, Horster FA, Kruskemper HL. *Developments in ophthalmology, 20:86-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/2591630/" style="text-decoration: none;">PubMed</a>

* **The importance of screening for medullary thyroid carcinoma in families of patients with MEN 2.**  
  Frilling A, **Goretzki PE**, Bastian L, Roeher HD. *Henry Ford Hospital medical journal, 37(3-4):122-3*. <a href="https://pubmed.ncbi.nlm.nih.gov/2576942/" style="text-decoration: none;">PubMed</a>


## 1988

* **Thyroid-stimulating antibodies of Graves' disease in thyroid cancer.**  
  Roher HD, **Goretzki PE**, Frilling A. *The New England journal of medicine, 319(25):1669-70*. <a href="https://pubmed.ncbi.nlm.nih.gov/3200284/" style="text-decoration: none;">PubMed</a>

* **A new siliconized-glass fiber as support for protein-chemical analysis of electroblotted proteins.**  
  Eckerskorn C, Mewes W, **Goretzki H**, Lottspeich F. *European journal of biochemistry, 176(3):509-19*. <a href="https://pubmed.ncbi.nlm.nih.gov/2458922/" style="text-decoration: none;">PubMed</a>

* **[Persistent postoperative hyperthyroidism--a surgical problem].**  
  Frilling A, **Goretzki PE**, Roher HD. *Wiener medizinische Wochenschrift (1946), 138(18):450-1*. <a href="https://pubmed.ncbi.nlm.nih.gov/3188552/" style="text-decoration: none;">PubMed</a>

* **Classical versus lower segment caesarean section for very-low-birthweight infants.**  
  Dietl JA, **Goretzki K**, Hirsch HA. *Lancet (London, England), 1(8596):1227*. <a href="https://pubmed.ncbi.nlm.nih.gov/2897041/" style="text-decoration: none;">PubMed</a>

* **[Acute abdomen in unrecognized hyperparathyroidism].**  
  **Goretzki PE**, Becker H, Dotzenrath C, Grussendorf M, Roher HD. *Wiener klinische Wochenschrift, 100(11):373-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/3407198/" style="text-decoration: none;">PubMed</a>

* **[Subtotal thyroidectomy as a superior therapeutic principle in the treatment of endocrine ophthalmopathy: a comparison with retrobulbar irradiation and thyrostatic therapy alone].**  
  Grussendorf M, Horster FA, Inanc Y, **Goretzki PE**, Roher HD, Kruskemper HL. *Wiener klinische Wochenschrift, 100(11):355-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/3407195/" style="text-decoration: none;">PubMed</a>

* **[Reinterventions for hyperparathyroidism in multiple endocrine adenopathy].**  
  Roher HD, **Goretzki PE**, Dotzenrath C. *Wiener klinische Wochenschrift, 100(11):364-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/2900582/" style="text-decoration: none;">PubMed</a>

* **[Reintervention in C-cell carcinoma].**  
  Wahl RA, **Goretzki PE**, Branscheid D, Vietmeier P, Roher HD. *Wiener klinische Wochenschrift, 100(11):348-51*. <a href="https://pubmed.ncbi.nlm.nih.gov/2900580/" style="text-decoration: none;">PubMed</a>

* **[Echinococcus cyst of the thyroid gland. A case report].**  
  Dotzenrath C, Burring KF, **Goretzki PE**. *Der Chirurg; Zeitschrift fur alle Gebiete der operativen Medizen, 59(2):106-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/3359872/" style="text-decoration: none;">PubMed</a>

* **[Fetal outcome following cesarean section in premature labor. Isthmocorpus longitudinal section or isthmian transverse section?].**  
  Dietl J, Dannecker G, **Goretzki K**, Hirsch HA. *Geburtshilfe und Frauenheilkunde, 48(1):13-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/3350321/" style="text-decoration: none;">PubMed</a>

* **[Autotransplantation of parathyroid gland tissue--on the indications for total parathyroidectomy with autotransplantation].**  
  **Goretzki PE**. *Langenbecks Archiv fur Chirurgie, 373(6):323-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/3210848/" style="text-decoration: none;">PubMed</a>


## 1987

* **Nerve growth factor (NGF) sensitizes human medullary thyroid carcinoma (hMTC) cells for cytostatic therapy in vitro.**  
  **Goretzki PE**, Wahl RA, Becker R, Koller C, Branscheid D, Grussendorf M, Roeher HD. *Surgery, 102(6):1035-42*. <a href="https://pubmed.ncbi.nlm.nih.gov/3686343/" style="text-decoration: none;">PubMed</a>

* **Observation of carboxylic groups in the lattice of sintered Ba2YCu3O7-y high-Tc superconductors.**  
  Parmigiani F, Chiarello G, Ripamonti N, **Goretzki H**, Roll U. *Physical review. B, Condensed matter, 36(13):7148-7150*. <a href="https://pubmed.ncbi.nlm.nih.gov/9942447/" style="text-decoration: none;">PubMed</a>

* **Management of goiter and thyroid nodules in an area of endemic goiter.**  
  Roher HD, **Goretzki PE**. *The Surgical clinics of North America, 67(2):233-49*. <a href="https://pubmed.ncbi.nlm.nih.gov/3551146/" style="text-decoration: none;">PubMed</a>

* **[Surgery of metastases of differentiated thyroid cancers].**  
  Roher HD, **Goretzki PE**, Wahl RA. *Langenbecks Archiv fur Chirurgie, 371(2):103-13*. <a href="https://pubmed.ncbi.nlm.nih.gov/3669824/" style="text-decoration: none;">PubMed</a>

* **Pathophysiological and clinical implications of thyroid growth-stimulating immunoglobulins: evidence for their intrathyroidal production.**  
  Schatz H, Ludwig I, Wiss F, **Goretzki PE**. *Acta endocrinologica. Supplementum, 281:334-41*. <a href="https://pubmed.ncbi.nlm.nih.gov/3475914/" style="text-decoration: none;">PubMed</a>

* **Adenylate cyclase stimulation and [3H]thymidine incorporation in human thyroid tissues and thyrocyte cultures: the effect of IgG preparation from patients with different thyroid disorders.**  
  **Goretzki PE**, West M, Koob R, Koller C, Joseph K, Roher HD. *Acta endocrinologica. Supplementum, 281:281-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/3475910/" style="text-decoration: none;">PubMed</a>

* **Thyrotropin (TSH) stimulates cell growth and DNA synthesis in monolayer cultures of human thyrocytes independent of the adenylate-cyclase system.**  
  **Goretzki PE**, Koob R, Koller C, Roher HD. *Acta endocrinologica. Supplementum, 281:273-80*. <a href="https://pubmed.ncbi.nlm.nih.gov/3475909/" style="text-decoration: none;">PubMed</a>


## 1986

* **The effect of thyrotropin and cAMP on DNA synthesis and cell growth of human thyrocytes in monolayer culture.**  
  **Goretzki PE**, Koob R, Koller T, Simon R, Branscheid D, Clark OH, Roher HD. *Surgery, 100(6):1053-61*. <a href="https://pubmed.ncbi.nlm.nih.gov/3024342/" style="text-decoration: none;">PubMed</a>

* **Tetanus toxin: primary structure, expression in E. coli, and homology with botulinum toxins.**  
  Eisel U, Jarausch W, **Goretzki K**, Henschen A, Engels J, Weller U, Hudel M, Habermann E, Niemann H. *The EMBO journal, 5(10):2495-502*. <a href="https://pubmed.ncbi.nlm.nih.gov/3536478/" style="text-decoration: none;">PubMed</a>

* **Screening for bacterial Fc-receptor activity on nitrocellulose membranes.**  
  Lammler C, Schaufuss P, **Goretzki K**, Blobel H. *Journal of immunological methods, 90(1):47-50*. <a href="https://pubmed.ncbi.nlm.nih.gov/3519777/" style="text-decoration: none;">PubMed</a>

* **[The cold thyroid nodule].**  
  Roher HD, **Goretzki P**. *Langenbecks Archiv fur Chirurgie, 369:191-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/3807517/" style="text-decoration: none;">PubMed</a>


## 1985

* **Thyroid function after surgery for autonomous and non-autonomous nodular endemic goitre--effect of iodide-substitution.**  
  Wahl RA, Joseph K, Bogner E, Ohmann C, **Goretzki P**, Roher HD. *Klinische Wochenschrift, 63(17):812-20*. <a href="https://pubmed.ncbi.nlm.nih.gov/3903336/" style="text-decoration: none;">PubMed</a>

* **Low T3 syndrome in patients following major surgery.**  
  Tsuchiya A, **Goretzki PE**, Gramse M, Joseph K, Wahl RA. *The Japanese journal of surgery, 15(4):285-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/4057698/" style="text-decoration: none;">PubMed</a>

* **Indication for operation of patients with autonomously functioning thyroid tissue in endemic goiter areas.**  
  **Goretzki PE**, Wahl RA, Branscheid D, Joseph K, Tsuchiya A, Roher HD. *World journal of surgery, 9(1):149-55*. <a href="https://pubmed.ncbi.nlm.nih.gov/3838608/" style="text-decoration: none;">PubMed</a>

* **Estrogen and thyroid-stimulating hormone (TSH) receptors in neoplastic and nonneoplastic human thyroid tissue.**  
  Clark OH, Gerend PL, Davis M, **Goretzki PE**, Hoffman PG Jr. *The Journal of surgical research, 38(2):89-96*. <a href="https://pubmed.ncbi.nlm.nih.gov/2982058/" style="text-decoration: none;">PubMed</a>

* **[Radicality principles in operations on malignant thyroid tumors].**  
  Wahl RA, **Goretzki PE**, Joseph K, Roher HD. *Langenbecks Archiv fur Chirurgie, 366:61-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/4058200/" style="text-decoration: none;">PubMed</a>

* **Enzymatic hydrolysis of tetanus toxin by intrinsic and extrinsic proteases. Characterization of the fragments by monoclonal antibodies.**  
  **Goretzki K**, Habermann E. *Medical microbiology and immunology, 174(3):139-50*. <a href="https://pubmed.ncbi.nlm.nih.gov/3897815/" style="text-decoration: none;">PubMed</a>


## 1984

* **Absence of high-affinity binding sites for beta-adrenergic blockers and lack of adenyl cyclase stimulation to beta-adrenergic stimulators in most normal and adenomatous human thyroid tissues.**  
  **Goretzki PE**, Wahl RA, Branscheid D, Roeher HD. *Surgery, 96(6):1001-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/6150553/" style="text-decoration: none;">PubMed</a>


## 1983

* **Serum and tissue T3 levels in endotoxin shock in the rat.**  
  **Goretzki PE**, Cavalieri RR, Wahl RA, Ellis RJ, McMahon FA, Goldfien S, Gerend P, Clark OH. *The Journal of surgical research, 35(3):211-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/6887841/" style="text-decoration: none;">PubMed</a>

* **Characterization of the thyrotropin receptor-adenylate cyclase system in neoplastic human thyroid tissue.**  
  Clark OH, Gerend PL, **Goretzki P**, Nissenson RA. *The Journal of clinical endocrinology and metabolism, 57(1):140-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/6304132/" style="text-decoration: none;">PubMed</a>

* **Surgery for hyperthyroidism: indications, pretreatment, operative strategy and results.**  
  Roeher HD, Wahl R, **Goretzki P**, Branscheid D. *Annales chirurgiae et gynaecologiae, 72(3):101-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/6194731/" style="text-decoration: none;">PubMed</a>

* **Monoclonal antibodies against tetanus toxin and toxoid.**  
  Ahnert-Hilger G, Bizzini B, **Goretzki K**, Muller H, Volckers C, Habermann E. *Medical microbiology and immunology, 172(2):123-35*. <a href="https://pubmed.ncbi.nlm.nih.gov/6193402/" style="text-decoration: none;">PubMed</a>


## 1982

* **Coexistence of hyperthyroidism and thyroid cancer.**  
  Wahl RA, **Goretzki P**, Meybier H, Nitschke J, Linder M, Roher HD. *World journal of surgery, 6(4):385-90*. <a href="https://pubmed.ncbi.nlm.nih.gov/7123975/" style="text-decoration: none;">PubMed</a>


## 1981

* **Prophylaxis of recurrent goiter by high-dose L-thyroxine.**  
  **Goretzki P**, Roeher HD, Horeyseck G. *World journal of surgery, 5(6):855-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/7342482/" style="text-decoration: none;">PubMed</a>


## 1979

* **[Mode of action and tolerance of a single weekly dose of 1 mg of L-thyroxine for the prevention of recurrence of goiter].**  
  Roher HD, **Goretzki P**, Horeyseck G. *Chirurgisches Forum fur experimentelle und klinische Forschung:251-4*. <a href="https://pubmed.ncbi.nlm.nih.gov/456135/" style="text-decoration: none;">PubMed</a>




defaultdict(list,
            {'2023': [{'PMID': '36670127',
               'OWN': 'NLM',
               'STAT': 'MEDLINE',
               'DCOM': '20230124',
               'LR': '20230316',
               'IS': '2045-2322 (Electronic) 2045-2322 (Linking)',
               'VI': '13',
               'IP': '1',
               'DP': '2023 Jan 20',
               'TI': 'A comparison of pediatric inflammatory multisystem syndrome temporarily-associated with SARS-CoV-2 and Kawasaki disease.',
               'PG': '1173',
               'LID': '10.1038/s41598-022-26832-5 [doi] 1173',
               'AB': 'The connection between Pediatric Inflammatory Multisystem Syndrome (PIMS) and Kawasaki Disease (KD) is not yet fully understood. Using the same national registry, clinical features and outcome of children hospitalized in Germany, and Innsbruck (Austria) were compared. Reported to the registry were 395 PIMS and 69 KD hospitalized patients. Patient age in PIMS cases was higher than in KD case